AInewsbot.ipynb

- Automate collecting daily AI news
- Open URLs of news sites specififed in `sources` dict (sources.yaml) using Selenium and Firefox
- Save HTML of each URL in htmldata directory
- Extract URLs from all files, create a pandas dataframe with url, title, src
- Use ChatGPT to filter only AI-related headlines by sending a prompt and formatted table of headlines
- Use SQLite to filter headlines previously seen 
- OPENAI_API_KEY should be in the environment or in a .env file
  
Alternative manual workflow to get HTML files if necessary
- Use Chrome, open e.g. Tech News bookmark folder, right-click and open all bookmarks in new window
- on Google News, make sure switch to AI tab
- on Google News, Feedly, Reddit, scroll to additional pages as desired
- Use SingleFile extension, 'save all tabs'
- Move files to htmldata directory
- Run lower part of notebook to process the data


todos:
langgraph
1. initialize
2. fetch sources
3. scrape news story urls from web pages
4. filter headlines
    - map e.g. google news headlines, url shorteners to 'true' URL
    - unique URLs
    - unique headlines
    - AI-relevant headlines we haven't seen before based on SQLite history
5. perform topic analysis on headlines, and ordering by topic
    - identify a topic name for each clusters
6. summarize individual pages as bullet points
    - todo, add cluster topics when summarizing
    - put into vector db with site, all topics, evergreen, assitned, and cluster topic 
7. from bullet points, extract top 10-20 most common themes and stories of the day in order of importance

loop through the top topics
for top topics retrieve from vector store and summarize
combine summaries
rewrite until it's good

- include the topics from clusters
- for each topic, top topic, examine each bullet point and determine if it is in that topic
- filter aidf to only stories that belong to these top topics
- send to a final summary prompt label only the matching top topics
- iterate until it looks good. 
8. topic analysis of bullet points, categorize bullet points as belonging to particular themes
9. for each theme, make a summary and links. Here we want to iterate to improve summaries per specific criteria.
10. combine themes and send.

In [1]:
# import sys
# del sys.modules['ainb_utilities']


In [2]:
from datetime import datetime
import os
import yaml
import dotenv
import sqlite3
import unicodedata
import json
import pickle
from collections import Counter
import shutil

import numpy as np
import pandas as pd
import umap
# import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN

# import bs4
import requests
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import trafilatura

import multiprocessing
from concurrent.futures import ThreadPoolExecutor, as_completed
import asyncio
import aiohttp

from IPython.display import HTML, Image, Markdown, display
import markdown

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

from langchain.vectorstores import Chroma
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings

from openai import OpenAI

from ainb_const import (DOWNLOAD_DIR, LOWCOST_MODEL, MODEL, CANONICAL_TOPICS,
                        SOURCECONFIG, FILTER_PROMPT, TOPIC_PROMPT,
                        SUMMARIZE_SYSTEM_PROMPT, SUMMARIZE_USER_PROMPT, FINAL_SUMMARY_PROMPT, TOP_CATEGORIES_PROMPT,
                        MAX_INPUT_TOKENS, MAX_OUTPUT_TOKENS, MAX_RETRIES, TEMPERATURE)
from ainb_utilities import (log, delete_files, filter_unseen_urls_db, insert_article, 
                            nearest_neighbor_sort, agglomerative_cluster_sort, traveling_salesman_sort_scipy,
                            unicode_to_ascii, send_gmail)
from ainb_webscrape import (get_driver, quit_drivers, launch_drivers, get_file, get_url, parse_file, 
                            get_og_tags, get_path_from_url, trimmed_href, process_source_queue_factory, 
                            process_url_queue_factory, DRIVERS)
from ainb_llm import (paginate_df, process_pages, fetch_pages, fetch_openai, fetch_all_summaries, 
                      fetch_openai_summary, trunc_tokens, categorize_headline)


import asyncio
# need this to run async in jupyter since it already has an asyncio event loop running
import nest_asyncio
nest_asyncio.apply()


# Initialize

In [3]:
# OpenAI API module
client = OpenAI()

# Or can use REST API directly
API_URL = 'https://api.openai.com/v1/chat/completions'

headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {os.getenv("OPENAI_API_KEY")}',
}


In [4]:
#  load sources to scrape from sources.yaml
with open(SOURCECONFIG, "r") as stream:
    try:
        sources = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

log(f"Load {len(sources)} sources from {SOURCECONFIG}")

# make a reverse dict to map output file titles to source names
sources_reverse = {}
for k, v in sources.items():
    log(f"{k} -> {v['url']} -> {v['title']}.html")
    v['sourcename'] = k
    # map filename (title) to source name
    sources_reverse[v['title']] = k

log(f"Mapped {len(sources_reverse)} source page titles to sources")


2024-07-15 08:41:13,298 - AInewsbot - INFO - Load 17 sources from sources.yaml
2024-07-15 08:41:13,298 - AInewsbot - INFO - Ars Technica -> https://arstechnica.com/ -> Ars Technica.html
2024-07-15 08:41:13,298 - AInewsbot - INFO - Bloomberg Tech -> https://www.bloomberg.com/technology -> Bloomberg Technology - Bloomberg.html
2024-07-15 08:41:13,298 - AInewsbot - INFO - Business Insider -> https://www.businessinsider.com/tech -> Tech - Business Insider.html
2024-07-15 08:41:13,299 - AInewsbot - INFO - FT Tech -> https://www.ft.com/technology -> Technology.html
2024-07-15 08:41:13,299 - AInewsbot - INFO - Feedly AI -> https://feedly.com/i/aiFeeds?options=eyJsYXllcnMiOlt7InBhcnRzIjpbeyJpZCI6Im5scC9mL3RvcGljLzMwMDAifV0sInNlYXJjaEhpbnQiOiJ0ZWNobm9sb2d5IiwidHlwZSI6Im1hdGNoZXMiLCJzYWxpZW5jZSI6ImFib3V0In1dLCJidW5kbGVzIjpbeyJ0eXBlIjoic3RyZWFtIiwiaWQiOiJ1c2VyLzYyZWViYjlmLTcxNTEtNGY5YS1hOGM3LTlhNTdiODIwNTMwOC9jYXRlZ29yeS9HYWRnZXRzIn1dfQ -> Discover and Add New Feedly AI Feeds.html
2024-07-15 08:4

20

In [5]:
sources

{'Ars Technica': {'include': ['^https://arstechnica.com/(\\w+)/(\\d+)/(\\d+)/'],
  'title': 'Ars Technica',
  'url': 'https://arstechnica.com/',
  'sourcename': 'Ars Technica'},
 'Bloomberg Tech': {'include': ['^https://www.bloomberg.com/news/'],
  'title': 'Bloomberg Technology - Bloomberg',
  'url': 'https://www.bloomberg.com/technology',
  'sourcename': 'Bloomberg Tech'},
 'Business Insider': {'exclude': ['^https://www.insider.com',
   '^https://www.passionfroot.me'],
  'title': 'Tech - Business Insider',
  'url': 'https://www.businessinsider.com/tech',
  'sourcename': 'Business Insider'},
 'FT Tech': {'include': ['https://www.ft.com/content/'],
  'title': 'Technology',
  'url': 'https://www.ft.com/technology',
  'sourcename': 'FT Tech'},
 'Feedly AI': {'exclude': ['^https://feedly.com',
   '^https://s1.feedly.com',
   '^https://blog.feedly.com'],
  'scroll': 5,
  'initial_sleep': 30,
  'title': 'Discover and Add New Feedly AI Feeds',
  'url': 'https://feedly.com/i/aiFeeds?options=e

In [6]:
sources_reverse


{'Ars Technica': 'Ars Technica',
 'Bloomberg Technology - Bloomberg': 'Bloomberg Tech',
 'Tech - Business Insider': 'Business Insider',
 'Technology': 'FT Tech',
 'Discover and Add New Feedly AI Feeds': 'Feedly AI',
 'Google News - Technology - Artificial intelligence': 'Google News',
 'Hacker News Page 1': 'Hacker News',
 'Hacker News Page 2': 'Hacker News 2',
 'HackerNoon - read, write and learn about any technology': 'HackerNoon',
 'Technology - The New York Times': 'NYT Tech',
 'top scoring links _ multi': 'Reddit',
 'Techmeme': 'Techmeme',
 'The Register_ Enterprise Technology News and Analysis': 'The Register',
 'Artificial Intelligence - The Verge': 'The Verge',
 'AI News _ VentureBeat': 'VentureBeat',
 'Technology - WSJ.com': 'WSJ Tech',
 'Technology - The Washington Post': 'WaPo Tech'}

In [7]:
# determine files already in htmldata directory
# List all paths in the directory matching today's date
nfiles = 50
files = [os.path.join(DOWNLOAD_DIR, file)
         for file in os.listdir(DOWNLOAD_DIR)]
# Get the current date
today = datetime.now()
year, month, day = today.year, today.month, today.day
datestr = datetime.now().strftime("%m_%d_%Y")

# filter files only
files = [file for file in files if os.path.isfile(file)]

# Sort files by modification time and take top 50
files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
file = files[:nfiles]

# filter files by with today's date ending in .html
files = [
    file for file in files if datestr in file and file.endswith(".html")]
log(len(files))
for file in files:
    log(file)

saved_pages = []
for file in files:
    filename = os.path.basename(file)
    # locate date like '01_14_2024' in filename
    position = filename.find(" (" + datestr)
    basename = filename[:position]
    # match to source name
    sourcename = sources_reverse.get(basename)
    if sourcename is None:
        log(f"Skipping {basename}, no sourcename metadata")
        continue
    sources[sourcename]['latest'] = file
    saved_pages.append((sourcename, file))

2024-07-15 08:41:17,778 - AInewsbot - INFO - 17
2024-07-15 08:41:17,778 - AInewsbot - INFO - htmldata/Technology - The Washington Post (07_15_2024 07_52_47 AM).html
2024-07-15 08:41:17,779 - AInewsbot - INFO - htmldata/Technology - WSJ.com (07_15_2024 07_52_41 AM).html
2024-07-15 08:41:17,779 - AInewsbot - INFO - htmldata/AI News _ VentureBeat (07_15_2024 07_52_40 AM).html
2024-07-15 08:41:17,779 - AInewsbot - INFO - htmldata/Discover and Add New Feedly AI Feeds (07_15_2024 07_52_36 AM).html
2024-07-15 08:41:17,780 - AInewsbot - INFO - htmldata/Artificial Intelligence - The Verge (07_15_2024 07_52_30 AM).html
2024-07-15 08:41:17,780 - AInewsbot - INFO - htmldata/top scoring links _ multi (07_15_2024 07_52_29 AM).html
2024-07-15 08:41:17,780 - AInewsbot - INFO - htmldata/The Register_ Enterprise Technology News and Analysis (07_15_2024 07_52_19 AM).html
2024-07-15 08:41:17,781 - AInewsbot - INFO - htmldata/Techmeme (07_15_2024 07_52_09 AM).html
2024-07-15 08:41:17,781 - AInewsbot - INFO

# Fetch and save source pages

In [8]:
# Fetch HTML files from sources

# empty download directory
delete_files(DOWNLOAD_DIR)

# save each file specified from sources
num_browsers = 3
log(f"Saving HTML files using {num_browsers} browsers")

# Create a queue for multiprocessing and populate it 
queue = multiprocessing.Queue()
for item in sources.values():
    queue.put(item)
    
# Function to take the queue and pop entries off and process until none are left
# lets you create an array of functions with different args
callable = process_source_queue_factory(queue)

saved_pages = launch_drivers(num_browsers, callable)


2024-07-15 08:41:22,417 - AInewsbot - INFO - Saving HTML files using 3 browsers
2024-07-15 08:41:22,438 - AInewsbot - INFO - get_driver - 64568 Initializing webdriver
2024-07-15 08:41:22,440 - AInewsbot - INFO - get_driver - 64568 Initializing webdriver
2024-07-15 08:41:22,441 - AInewsbot - INFO - get_driver - 64568 Initializing webdriver
2024-07-15 08:41:41,532 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-15 08:41:41,532 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-15 08:41:41,533 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-15 08:41:41,533 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-15 08:41:41,534 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-15 08:41:41,534 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-15 08:42:42,598 - AInewsbot - INFO - get_driver - Initialized webdriver
2024-07-15 08:42:42,694 - AInewsbot - INFO - Proces

2024-07-15 08:43:28,057 - AInewsbot - INFO - get_files(Google News - Technology - Artificial intelligence) - Saving Google News - Technology - Artificial intelligence (07_15_2024 08_43_28 AM).html as utf-8
2024-07-15 08:43:28,061 - AInewsbot - INFO - Processing NYT Tech
2024-07-15 08:43:28,062 - AInewsbot - INFO - get_files(Technology - The New York Times) - starting get_files https://www.nytimes.com/section/technology
2024-07-15 08:43:36,007 - AInewsbot - INFO - get_files(Discover and Add New Feedly AI Feeds) - Loading additional infinite scroll items
2024-07-15 08:43:37,306 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrom

2024-07-15 08:44:22,427 - AInewsbot - INFO - get_files(Technology - WSJ.com) - Saving Technology - WSJ.com (07_15_2024 08_44_22 AM).html as utf-8
2024-07-15 08:44:22,428 - AInewsbot - INFO - Quit webdriver
2024-07-15 08:44:27,276 - AInewsbot - INFO - Message: Unable to locate element: //meta[@http-equiv='Content-Type']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:192:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:510:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

2024-07-15 08:44:27,276 - AInewsbot - INFO - get_files(Technology - The Washington Post) - Saving Technology - The Washington Post (07_15_2024 08_44_27 AM).html as utf-8
2024-07-15 08:44:27,279 - AInewsbot - INFO - Quit webdriver


In [9]:
log(f"Saved {len(saved_pages)} pages")

print(len(saved_pages))
for sourcename, page in saved_pages:
    sources[sourcename]['latest'] = page
    log(f"{sourcename} -> {page}")
    

2024-07-15 08:44:29,742 - AInewsbot - INFO - Saved 17 pages
2024-07-15 08:44:29,743 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-15 08:44:29,744 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-15 08:44:29,744 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-15 08:44:29,745 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-15 08:44:29,745 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-15 08:44:29,746 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-15 08:44:29,747 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-15 08:44:29,747 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-15 08:44:29,747 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-15 08:44:29,748 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-15 08:44:29,748 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-15 08:44:29,748 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-15 08:44:29,748 - AInewsbot - INFO - {sourcename} -> {page}
2024-07-15 08:44:29,749 - AInewsbot - INFO - {sourcename

17


# Extract news URLs from saved pages

In [10]:
# Parse news URLs and titles from downloaded HTML files
log("Parsing html files")
all_urls = []
for sourcename, filename in saved_pages:
    log(sourcename +' -> ' + filename)
    log(f"{sourcename}", "parse loop")
    links = parse_file(sources[sourcename])
    log(f"{len(links)} links found", "parse loop")
    all_urls.extend(links)

log(f"found {len(all_urls)} links", "parse loop")

# make a pandas dataframe of all the links found
orig_df = (
    pd.DataFrame(all_urls)
    .groupby("url")
    .first()
    .reset_index()
    .sort_values("src")[["src", "title", "url"]]
    .reset_index(drop=True)
    .reset_index(drop=False)
    .rename(columns={"index": "id"})
)
orig_df.head()


2024-07-15 08:44:29,790 - AInewsbot - INFO - Parsing html files
2024-07-15 08:44:29,796 - AInewsbot - INFO - Ars Technica -> htmldata/Ars Technica (07_15_2024 08_42_54 AM).html
2024-07-15 08:44:29,798 - AInewsbot - INFO - parse loop - Ars Technica
2024-07-15 08:44:29,850 - AInewsbot - INFO - parse_file - found 252 raw links
2024-07-15 08:44:29,853 - AInewsbot - INFO - parse_file - found 30 filtered links
2024-07-15 08:44:29,854 - AInewsbot - INFO - parse loop - 30 links found
2024-07-15 08:44:29,854 - AInewsbot - INFO - FT Tech -> htmldata/Technology (07_15_2024 08_43_05 AM).html
2024-07-15 08:44:29,854 - AInewsbot - INFO - parse loop - FT Tech
2024-07-15 08:44:29,887 - AInewsbot - INFO - parse_file - found 464 raw links
2024-07-15 08:44:29,893 - AInewsbot - INFO - parse_file - found 111 filtered links
2024-07-15 08:44:29,894 - AInewsbot - INFO - parse loop - 111 links found
2024-07-15 08:44:29,895 - AInewsbot - INFO - Hacker News -> htmldata/Hacker News Page 1 (07_15_2024 08_43_16 AM)

,id,src,title,url
0,0,Ars Technica,Animals use physics? Let us count the ways,https://arstechnica.com/science/2024/07/animal...
1,1,Ars Technica,"German Navy still uses 8-inch floppy disks, wo...",https://arstechnica.com/gadgets/2024/07/german...
2,2,Ars Technica,New app releases for Apple Vision Pro have fal...,https://arstechnica.com/gadgets/2024/07/new-ap...
3,3,Ars Technica,$500 aluminum version of the Analogue Pocket l...,https://arstechnica.com/gaming/2024/07/500-alu...
4,4,Ars Technica,Arm tweaks AMD’s FSR to bring battery-saving G...,https://arstechnica.com/gaming/2024/07/arm-get...


In [11]:
# map google news headlines to redirect
def get_google_news_redirects(orig_df):
    redirect_dict = {}
    for row in orig_df.itertuples():
        parsed_url = urlparse(row.url)
        netloc = parsed_url.netloc
        if netloc == 'news.google.com':
            logstr = netloc + " -> "
            response = requests.get(row.url, allow_redirects=False)
            # The URL to which it would have redirected
            redirect_url = response.headers.get('Location')
            redirect_dict[row.url] = redirect_url
            parsed_url2 = urlparse(redirect_url)
            netloc2 = parsed_url2.netloc
            if netloc2 == 'news.google.com':
#                 logstr += netloc2 + " -> "
                response = requests.get(redirect_url, allow_redirects=False)
            # The URL to which it would have redirected
                redirect_url = response.headers.get('Location')
                redirect_dict[row.url] = redirect_url
                logstr += redirect_url
            log(logstr, "get_google_news_redirects")
    return redirect_dict

redirect_dict = get_google_news_redirects(orig_df)

orig_df['actual_url'] = orig_df['url'].apply(lambda url: redirect_dict.get(url, url))


2024-07-15 08:44:58,078 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.techradar.com/televisions/a-big-deal-get-a-75-inch-samsung-8k-tv-for-half-off-in-this-early-prime-day-deal
2024-07-15 08:44:58,281 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://seekingalpha.com/news/4123718-seeking-alphas-top-quant-picks-in-information-technology-as-q2-earnings-approach
2024-07-15 08:44:58,585 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.defensenews.com/global/europe/2024/07/15/how-europes-next-generation-combat-jet-aims-to-catch-the-ai-wave/
2024-07-15 08:44:58,825 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.benzinga.com/news/24/07/39748729/nvidia-to-rally-more-than-33-here-are-10-top-analyst-forecasts-for-friday
2024-07-15 08:44:59,167 - AInewsbot - INFO - get_google_news_redirects - news.go

2024-07-15 08:45:08,269 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.ign.com/articles/once-human-dev-struggles-to-cope-with-huge-number-of-players-as-steam-hit-exceeds-expectations
2024-07-15 08:45:08,636 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.forbes.com/sites/davealtavilla/2024/07/10/dell-xps-13-powered-by-snapdragon-x-elite-tested-a-very-strong-start/
2024-07-15 08:45:08,946 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.theguardian.com/world/article/2024/jul/09/spain-sentences-15-school-children-over-ai-generated-naked-images
2024-07-15 08:45:09,201 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.creativebloq.com/tech/laptops/save-dollar500-on-brand-new-macbook-pro-m3-pro-now-at-record-low-price
2024-07-15 08:45:09,539 - AInewsbot - INFO - get_google_news_redirects - ne

2024-07-15 08:45:19,093 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://hbr.org/sponsored/2024/07/the-ai-advantage-revolutionizing-company-culture-for-unmatched-business-success
2024-07-15 08:45:19,386 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.zdnet.com/article/samsungs-upcoming-xr-headset-was-the-biggest-product-teaser-at-unpacked-july-2024/
2024-07-15 08:45:19,623 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.reuters.com/technology/tsmc-second-quarter-profit-seen-jumping-30-surging-ai-chip-demand-2024-07-15/
2024-07-15 08:45:19,953 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.investing.com/news/stock-market-news/5-big-analyst-ai-moves-tesla-and-smci-stocks-downgraded-3518257
2024-07-15 08:45:20,208 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google

2024-07-15 08:45:28,531 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.youtube.com/watch?v=IaBi-m9s2jM
2024-07-15 08:45:28,869 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.gamesradar.com/platforms/xbox/with-game-pass-price-hikes-sparking-debate-indie-developers-are-rising-up-to-remind-us-how-vital-the-xbox-subscription-can-be-to-the-tune-of-staggering-numbers/
2024-07-15 08:45:29,109 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.gamesradar.com/games/survival/hours-before-launch-hyped-survival-game-once-human-reminds-players-about-its-seasonal-server-wipes-to-provide-a-fairer-more-relaxed-and-freer-gaming-experience/
2024-07-15 08:45:29,412 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.gamesradar.com/games/baldur-s-gate/baldurs-gate-3-ceo-says-he-was-fighting-with-embassies-to-r

2024-07-15 08:45:37,361 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.pushsquare.com/news/2024/07/nexon-will-update-the-first-descendants-in-game-icons-following-plagiarism-complaints
2024-07-15 08:45:37,588 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.eurogamer.net/the-legend-of-zelda-echoes-of-wisdom-esrb-rating-confirms-well-get-to-play-as-link-as-well-as-zelda
2024-07-15 08:45:37,933 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.notebookcheck.net/150-Nvidia-RTX-A6000-GPUs-power-the-Las-Vegas-Sphere-that-s-7-2TB-of-GDDR6-memory.862441.0.html
2024-07-15 08:45:38,252 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.forbes.com/sites/paultassi/2024/07/13/baldurs-gate-3s-larian-says-they-wanted-to-do-a-fallout-game-can-they-now/
2024-07-15 08:45:38,509 - AInewsbot - INFO - get_goog

...

2024-07-15 08:46:00,369 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.currentaffairs.org/news/the-only-ethical-model-for-ai-is-socialism
2024-07-15 08:46:00,594 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.androidpolice.com/oneplus-pad-2-retail-packaging-specs-price-leak/
2024-07-15 08:46:00,918 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.androidcentral.com/gaming/virtual-reality/news-weekly-july-13-2024
2024-07-15 08:46:01,154 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.androidauthority.com/google-photos-copies-my-week-feature-3460484/
2024-07-15 08:46:01,500 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.nytimes.com/2024/07/11/climate/kobold-zambia-copper-ai-mining.html
2024-07-15 08:46:01,752 - AInewsbot - INFO - get_g

2024-07-15 08:46:13,180 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.ft.com/content/7b8d2057-2687-45b3-bae4-1488a75ac5b2
2024-07-15 08:46:13,506 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.ign.com/articles/sony-extends-concord-beta-by-a-day
2024-07-15 08:46:13,752 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.wired.com/gallery/best-hearing-aids-for-seniors/
2024-07-15 08:46:14,070 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.buzzfeed.com/emmalord9/underrated-useful-thirtysomething-an
2024-07-15 08:46:14,315 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.androidauthority.com/android-ultra-hdr-camera-apps-3460456/
2024-07-15 08:46:14,627 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> h

2024-07-15 08:46:26,221 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.engadget.com/google-maps-speedometer-finally-comes-to-ios-and-carplay-185946210.html
2024-07-15 08:46:26,540 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://pro.thestreet.com/portfolio/wall-street-ups-price-targets-on-4-portfolio-stocks-our-take
2024-07-15 08:46:26,782 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://investorplace.com/2024/07/3-artificial-intelligence-stocks-set-to-mint-new-millionaires/
2024-07-15 08:46:27,097 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.pcmag.com/news/las-vegas-sphere-uses-150-nvidia-a6000-gpus-to-power-its-massive-display
2024-07-15 08:46:27,345 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.livescience.com/health/relationships/peopl

2024-07-15 08:46:37,555 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.cio.com/article/2509754/generative-ai-and-preparing-for-a-shift-to-skills-based-hiring.html
2024-07-15 08:46:37,794 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.scientificamerican.com/article/we-cannot-cede-control-of-weapons-to-artificial-intelligence/
2024-07-15 08:46:38,133 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.pymnts.com/news/payments-innovation/2024/discover-looks-to-ai-to-unlock-customer-experience/
2024-07-15 08:46:38,370 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.gizchina.com/2024/07/15/one-ui-6-1-1-update-brings-exciting-new-features-to-samsung-devices/
2024-07-15 08:46:38,667 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.forbes.com/sit

2024-07-15 08:46:49,224 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.helpnetsecurity.com/2024/07/10/jake-king-elastic-llms-security-risks/
2024-07-15 08:46:49,558 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.polygon.com/24196656/best-buy-black-friday-prime-day-gaming-deal-sale
2024-07-15 08:46:49,809 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.techradar.com/gaming/nyt-connections-today-answers-hints-15-july-2024
2024-07-15 08:46:50,141 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.gamingbible.com/news/gta-6-second-trailer-take-two-announcement-350153-20240712
2024-07-15 08:46:50,395 - AInewsbot - INFO - get_google_news_redirects - news.google.com -> news.google.com -> https://www.tomsguide.com/phones/iphones/you-can-now-play-pc-games-on-your-iphone
2024-07-15 08:46:50,746 -

In [ ]:
# # extracts all links from history where isAI=1
# # useful for training dimensionality reduction
# conn = sqlite3.connect('articles.db')
# c = conn.cursor()
# #  and timestamp > '2024-07-01' 
# query = "select * from news_articles where isAI=1 order by id"
# ai_history_df = pd.read_sql_query(query, conn)
# ai_history_df

# Filter URLs to new AI headlines only

In [12]:
# filter urls we've already seen in previous runs and saved in SQLite
filtered_df = filter_unseen_urls_db(orig_df, before_date='2024-07-14 09:00:00')
len(filtered_df)

2024-07-15 08:47:47,174 - AInewsbot - INFO - Existing URLs: 130230
2024-07-15 08:47:47,211 - AInewsbot - INFO - New URLs: 756


756

In [13]:
# use chatgpt to filter AI-related headlines using a prompt to OpenAI
print(FILTER_PROMPT)



You will act as a research assistant to categorize news articles based on their relevance
to the topic of artificial intelligence (AI). You will closely read the title of each story
to determine if it is primarily about AI based on the semanting meaning of the title and
the keywords and entities mentioned. The input headlines and outptu classifications will
be formatted as JSON objects.

Input Specification:
You will receive a list of news headlines formatted as JSON objects.
Each object will include an 'id' and a 'title'. For instance:
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 103,'title': 'Baby trapped in refrigerator eats own foot'},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
 ]

Classification Criteria:
Classify each story based

In [14]:
# make pages that fit in a reasonably sized (MAXPAGELEN or MAX_INPUT_TOKENS) prompt
pages = paginate_df(filtered_df)
log(f"Paginated {len(pages)} pages")


2024-07-15 08:47:51,785 - AInewsbot - INFO - Paginated 16 pages


20

In [15]:
# use REST API directly. OpenAI python API doesn't support concurrent requests from a single client
# this runs fast with async aiohttp and on gpt-3.5 (15 seconds vs 2 minutes synchronously with gpt-4o)
# the old API supported submitting multiple payloads in a single completion request
# current API supports a slow 'batch' submission https://platform.openai.com/docs/guides/rate-limits/usage-tiers
# there is a more complex example here - https://github.com/openai/openai-cookbook/blob/main/examples/api_request_parallel_processor.py

log("start classify")
enriched_urls = asyncio.run(fetch_pages(pages, prompt=FILTER_PROMPT))
log("end classify")

enriched_df = pd.DataFrame(enriched_urls)
print(len(enriched_df))
log("isAI", len(enriched_df.loc[enriched_df["isAI"]]))
log("not isAI", len(enriched_df.loc[~enriched_df["isAI"]]))
enriched_df.head()


2024-07-15 08:47:54,152 - AInewsbot - INFO - start classify
2024-07-15 08:47:54,157 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:47:54,180 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:47:54,181 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:47:54,181 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:47:54,182 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:47:54,182 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:47:54,183 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:47:54,183 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:47:54,184 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:47:54,184 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:47:54,185 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:47:54,185 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:47:54,185 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:47:54,185 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:47:54,186 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:47:54,186 - AInewsbot - IN

805


,id,isAI
0,0,False
1,28,False
2,30,False
3,31,False
4,32,False


In [16]:
# merge returned df with isAI column into original df on id column
merged_df = pd.merge(filtered_df, enriched_df, on="id", how="outer")
merged_df['date'] = datetime.now().date()
merged_df.head()


,id,src,title,url,actual_url,isAI,date
0,0,Ars Technica,Animals use physics? Let us count the ways,https://arstechnica.com/science/2024/07/animal...,https://arstechnica.com/science/2024/07/animal...,False,2024-07-15
1,28,Ars Technica,Here’s Audi’s next A5—and all the stuff we won...,https://arstechnica.com/cars/2024/07/heres-aud...,https://arstechnica.com/cars/2024/07/heres-aud...,False,2024-07-15
2,30,Bloomberg Tech,Sea’s Digital Bank Picks New Chief to Ward Off...,https://www.bloomberg.com/news/articles/2024-0...,https://www.bloomberg.com/news/articles/2024-0...,False,2024-07-15
3,31,Bloomberg Tech,Amazon Sold a Used Diaper. It Tanked a Mom-and...,https://www.bloomberg.com/news/articles/2024-0...,https://www.bloomberg.com/news/articles/2024-0...,False,2024-07-15
4,32,Bloomberg Tech,AutoNation Warns CDK Hack Put a Big Dent in Qu...,https://www.bloomberg.com/news/articles/2024-0...,https://www.bloomberg.com/news/articles/2024-0...,False,2024-07-15


In [17]:
# should be empty, shouldn't get back rows that don't match to existing
log(f"Unmatched response rows: {len(merged_df.loc[merged_df['src'].isna()])}")
# should be empty, should get back all rows from orig
log(f"Unmatched source rows: {len(merged_df.loc[merged_df['isAI'].isna()])}")


2024-07-15 08:48:10,858 - AInewsbot - INFO - Unmatched response rows: 0
2024-07-15 08:48:10,860 - AInewsbot - INFO - Unmatched source rows: 0


20

In [18]:
merged_df

,id,src,title,url,actual_url,isAI,date
0,0,Ars Technica,Animals use physics? Let us count the ways,https://arstechnica.com/science/2024/07/animal...,https://arstechnica.com/science/2024/07/animal...,False,2024-07-15
1,28,Ars Technica,Here’s Audi’s next A5—and all the stuff we won...,https://arstechnica.com/cars/2024/07/heres-aud...,https://arstechnica.com/cars/2024/07/heres-aud...,False,2024-07-15
2,30,Bloomberg Tech,Sea’s Digital Bank Picks New Chief to Ward Off...,https://www.bloomberg.com/news/articles/2024-0...,https://www.bloomberg.com/news/articles/2024-0...,False,2024-07-15
3,31,Bloomberg Tech,Amazon Sold a Used Diaper. It Tanked a Mom-and...,https://www.bloomberg.com/news/articles/2024-0...,https://www.bloomberg.com/news/articles/2024-0...,False,2024-07-15
4,32,Bloomberg Tech,AutoNation Warns CDK Hack Put a Big Dent in Qu...,https://www.bloomberg.com/news/articles/2024-0...,https://www.bloomberg.com/news/articles/2024-0...,False,2024-07-15
...,...,...,...,...,...,...,...
800,1243,VentureBeat,Psychological safety can be beneficial (to a p...,https://venturebeat.com/ai/psychological-safet...,https://venturebeat.com/ai/psychological-safet...,False,2024-07-15
801,1264,WaPo Tech,Amazon’s health clinic pushed a misleading acc...,https://www.washingtonpost.com/technology/2024...,https://www.washingtonpost.com/technology/2024...,False,2024-07-15
802,1274,WaPo Tech,"Make your job less painful, at least physically",https://www.washingtonpost.com/technology/2024...,https://www.washingtonpost.com/technology/2024...,False,2024-07-15
803,1280,WaPo Tech,‘BlueAnon’ conspiracy theories flood social me...,https://www.washingtonpost.com/technology/2024...,https://www.washingtonpost.com/technology/2024...,False,2024-07-15


In [19]:
# must do this after fixing google actualurl
merged_df['hostname']=merged_df['actual_url'].apply(lambda url: urlparse(url).netloc)
merged_df['site_name']

In [20]:
# update SQLite database with all seen URLs
conn = sqlite3.connect('articles.db')
cursor = conn.cursor()

for row in merged_df.itertuples():
    insert_article(conn, cursor, row.src, row.hostname, row.title,
                   row.url, row.actual_url, row.isAI, row.date)


In [21]:
# keep headlines that are related to AI
AIdf = merged_df.loc[merged_df["isAI"]==1] \
    .reset_index(drop=True)  \
    .reset_index()  \
    .drop(columns=["id"])  \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} AI headlines")
AIdf

2024-07-15 08:48:31,193 - AInewsbot - INFO - Found 236 AI headlines


,id,src,title,url,actual_url,isAI,date,hostname
0,0,Bloomberg Tech,Apple Is Top Pick at Morgan Stanley as AI Seen...,https://www.bloomberg.com/news/articles/2024-0...,https://www.bloomberg.com/news/articles/2024-0...,True,2024-07-15,www.bloomberg.com
1,1,Bloomberg Tech,Smartphone Market Grows 6% as AI and Discounts...,https://www.bloomberg.com/news/articles/2024-0...,https://www.bloomberg.com/news/articles/2024-0...,True,2024-07-15,www.bloomberg.com
2,2,Business Insider,Cathie Wood says she wouldn't have sold Nvidia...,https://www.businessinsider.com/cathie-wood-wo...,https://www.businessinsider.com/cathie-wood-wo...,True,2024-07-15,www.businessinsider.com
3,3,Business Insider,"The CEO of Cerebras, which could go public soo...",https://www.businessinsider.com/cerebras-nvidi...,https://www.businessinsider.com/cerebras-nvidi...,True,2024-07-15,www.businessinsider.com
4,4,Business Insider,AI startup founders discuss what it's like to ...,https://www.businessinsider.com/nvidia-startup...,https://www.businessinsider.com/nvidia-startup...,True,2024-07-15,www.businessinsider.com
...,...,...,...,...,...,...,...,...
231,231,Reddit,"In this age of misinformation and AI growth, i...",https://www.reddit.com/r/ChatGPT/comments/1e3a...,https://www.reddit.com/r/ChatGPT/comments/1e3a...,True,2024-07-15,www.reddit.com
232,232,Reddit,Did Chatgpt just create a word for me?,https://www.reddit.com/r/ChatGPT/comments/1e33...,https://www.reddit.com/r/ChatGPT/comments/1e33...,True,2024-07-15,www.reddit.com
233,233,Techmeme,"Q&A with Imran Ahmed, founder of the Center fo...",https://www.wired.com/story/dial-up-imran-ahmed/,https://www.wired.com/story/dial-up-imran-ahmed/,True,2024-07-15,www.wired.com
234,234,The Register,"Honey, I shrunk the LLM! A beginner's guide to...",https://www.theregister.com/2024/07/14/quantiz...,https://www.theregister.com/2024/07/14/quantiz...,True,2024-07-15,www.theregister.com


In [22]:
# map title to ascii characters to avoid some dupes with e.g. different quote symbols

def unicode_to_ascii(input_string):
    # Normalize the Unicode string to NFKD form
    normalized_string = unicodedata.normalize('NFKD', input_string)
    
    # Encode to ASCII bytes, ignoring characters that cannot be converted
    ascii_bytes = normalized_string.encode('ascii', 'ignore')
    
    # Convert bytes back to a string
    ascii_string = ascii_bytes.decode('ascii')
    
    return ascii_string

AIdf['title'] = AIdf['title'].apply(unicode_to_ascii)


In [23]:
# dedupe identical URLs, Google News and Feedly might have same URLs with different headlines
AIdf = AIdf.sort_values("src") \
    .groupby("url") \
    .first() \
    .reset_index(drop=False) \
    .drop(columns=['id']) \
    .reset_index() \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} unique AI URLs")


2024-07-15 08:48:45,477 - AInewsbot - INFO - Found 217 unique AI URLs


20

In [24]:
# dedupe identical headlines
AIdf['title_clean'] = AIdf['title'].map(lambda s: "".join(s.split()))
AIdf = AIdf.sort_values("src") \
    .groupby("title_clean") \
    .first() \
    .reset_index(drop=True) \
    .drop(columns=['id']) \
    .reset_index() \
    .rename(columns={'index': 'id'})

log(f"Found {len(AIdf)} unique AI headlines")


2024-07-15 08:48:53,188 - AInewsbot - INFO - Found 206 unique AI headlines


20

### Get site names and update site names based on URL

In [25]:
# get site_name
conn = sqlite3.connect('articles.db')
c = conn.cursor()
#  and timestamp > '2024-07-01' 
query = "select * from sites"
sites_df = pd.read_sql_query(query, conn)
sites_dict = {row.hostname:row.site_name for row in sites_df.itertuples()}

sites_df

,id,hostname,site_name
0,1,247wallst.com,247wallst
1,2,3quarksdaily.com,3 Quarks Daily
2,3,9to5google.com,9to5Google
3,4,9to5mac.com,9to5Mac
4,5,9to5toys.com,9to5Toys
...,...,...,...
536,538,www.ultimate-guitar.com,Ultimate Guitar
537,539,www.13newsnow.com,13News Now
538,540,smilingrobo.necrozmalabs.com,Smiling Robo
539,541,www.askmen.com,AskMen


In [26]:
AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, ""))
AIdf.loc[AIdf['site_name']==""]

,id,url,src,title,actual_url,isAI,date,hostname,site_name
9,9,https://www.reformer.com/health/ai-better-at-p...,Feedly AI,AI Better at Predicting Progression to Alzheim...,https://www.reformer.com/health/ai-better-at-p...,True,2024-07-15,www.reformer.com,
12,12,https://religionunplugged.com/news/2024/7/15/a...,Feedly AI,AI Ripe For Worship Music Planning And Admin H...,https://religionunplugged.com/news/2024/7/15/a...,True,2024-07-15,religionunplugged.com,
43,43,https://zenodo.org/records/12740116,Hacker News 2,CURLoRA: Stable LLM Fine-Tuning and Catastroph...,https://zenodo.org/records/12740116,True,2024-07-15,zenodo.org,
62,62,https://www.raspberrypi.org/blog/empowering-un...,Feedly AI,Empowering undergraduate computer science stud...,https://www.raspberrypi.org/blog/empowering-un...,True,2024-07-15,www.raspberrypi.org,
69,69,https://newhampshirebulletin.com/2024/07/15/fr...,Feedly AI,"From precalculus to Gatsby, state offers schoo...",https://newhampshirebulletin.com/2024/07/15/fr...,True,2024-07-15,newhampshirebulletin.com,
95,95,https://next.thenextweb.com/news/europe-univer...,Feedly AI,How Europes universities are using AI to battl...,https://next.thenextweb.com/news/europe-univer...,True,2024-07-15,next.thenextweb.com,
112,112,https://sg.finance.yahoo.com/news/apple-inc-na...,Feedly AI,Is Apple Inc (NASDAQ:AAPL) Aswath Damodarans B...,https://sg.finance.yahoo.com/news/apple-inc-na...,True,2024-07-15,sg.finance.yahoo.com,
202,202,https://www.utilitydive.com/spons/you-cant-hav...,Feedly AI,You cant have an AI strategy without a data st...,https://www.utilitydive.com/spons/you-cant-hav...,True,2024-07-15,www.utilitydive.com,


In [27]:
async def get_site_name(session, row):
    cat_prompt = f"""
based on this url and your knowledge of the Web, what is the name of the site? https://{row.hostname}

return the response as a json object of the form {{"url": "www.yankodesign.com", "site_name": "Yanko Design"}}

    """
    try:
        messages=[
                  {"role": "user", "content": cat_prompt
                  }]

        payload = {"model":  LOWCOST_MODEL,
                   "response_format": {"type": "json_object"},
                   "messages": messages,
                   "temperature": 0
                   }
        response = await fetch_openai(session, payload)
        response_dict = json.loads(response["choices"][0]["message"]["content"])
        return response_dict
    except Exception as exc:
        print(exc)
                
tasks = []
async with aiohttp.ClientSession() as session:
    for row in AIdf.loc[AIdf['site_name']==""].itertuples():
        task = asyncio.create_task(get_site_name(session, row))
        tasks.append(task)
    responses = await asyncio.gather(*tasks)

responses


[{'url': 'www.reformer.com', 'site_name': 'The Brattleboro Reformer'},
 {'url': 'https://religionunplugged.com', 'site_name': 'Religion Unplugged'},
 {'url': 'https://zenodo.org', 'site_name': 'Zenodo'},
 {'url': 'www.raspberrypi.org', 'site_name': 'Raspberry Pi'},
 {'url': 'https://newhampshirebulletin.com',
  'site_name': 'New Hampshire Bulletin'},
 {'url': 'https://next.thenextweb.com', 'site_name': 'The Next Web'},
 {'url': 'https://sg.finance.yahoo.com', 'site_name': 'Yahoo Finance'},
 {'url': 'www.utilitydive.com', 'site_name': 'Utility Dive'}]

In [28]:
# update site_dict from responses
new_urls = []
for r in responses:
    if r['url'].startswith('https://'):
        r['url'] = r['url'][8:]
    new_urls.append(r['url'])
    sites_dict[r['url']] = r['site_name']
    print(r['url'], r['site_name'])

AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, hostname))



www.reformer.com The Brattleboro Reformer
religionunplugged.com Religion Unplugged
zenodo.org Zenodo
www.raspberrypi.org Raspberry Pi
newhampshirebulletin.com New Hampshire Bulletin
next.thenextweb.com The Next Web
sg.finance.yahoo.com Yahoo Finance
www.utilitydive.com Utility Dive


In [29]:
site_update_str = ",\n".join([f"('{url}', '{sites_dict[url]}')" for url in new_urls])
print(site_update_str)

('www.reformer.com', 'The Brattleboro Reformer'),
('religionunplugged.com', 'Religion Unplugged'),
('zenodo.org', 'Zenodo'),
('www.raspberrypi.org', 'Raspberry Pi'),
('newhampshirebulletin.com', 'New Hampshire Bulletin'),
('next.thenextweb.com', 'The Next Web'),
('sg.finance.yahoo.com', 'Yahoo Finance'),
('www.utilitydive.com', 'Utility Dive')


In [30]:
# update 
if site_update_str:
    sqlstr = f"""

    INSERT OR IGNORE INTO sites (hostname, site_name) VALUES 
    {site_update_str};
    """
    conn.execute(sqlstr)
    conn.commit()


# Topic analysis
Try to identify the top topics of the day, to help make a nice summary. 

1st approach - do dimensionality reduction on the headline embeddings with UMAP and cluster with DBSCAN.

2nd approach
 - extract topics from headline using a prompt
 - human canonicalizes topics
 - assign headlines to topics using a prompt
 
 The final summary is pretty inconsistent, would be nice to give chatgpt a prompt that would say, summarize these bullet points using this categorization.
 

### Fit dimensionality reduction model

In [31]:
# # train dimensionality reduction, only need to do this every few months and pickle the model to reflect new topics
# # extracts all links from history where isAI=1
# conn = sqlite3.connect('articles.db')
# c = conn.cursor()
# #  and timestamp > '2024-07-01' 
# query = "select * from news_articles where isAI=1 order by id desc limit 20000"
# ai_history_df = pd.read_sql_query(query, conn)
# len(ai_history_df)

In [32]:
# embedding_model="text-embedding-3-large"
# embedding_df_list = []
# pages = paginate_df(ai_history_df, maxpagelen=1000, max_input_tokens=8192)

# for p in pages:
#     response = client.embeddings.create(input=[obj['title'] for obj in p],
#                                         model=embedding_model)
#     embedding_df_list.append(pd.DataFrame([e.model_dump()['embedding'] for e in response.data]))

# embedding_df = pd.concat(embedding_df_list, axis=0, ignore_index=True)

# embedding_df.to_pickle("historical_embedding_df.pkl")


In [33]:
# # Initialize the UMAP reducer
# reducer = umap.UMAP(n_components=30)
# # Fit the reducer to the data without transforming
# reducer.fit(embedding_df)
# # Pickle the reducer
# with open('reducer.pkl', 'wb') as f:
#     pickle.dump(reducer, f)
# print("UMAP reducer pickled and saved as 'reducer.pkl'")

In [34]:
# attempt to extract top topics 
print(TOPIC_PROMPT)



You will act as a research assistant to extract topics from news headlines. You will extract topics, entities,
and keywords from news headlines formatted as JSON objects.

Input Specification:
You will receive a list of news headlines formatted as JSON objects.
Each object will include an 'id' and a 'title'. For instance:
[{'id': 97, 'title': 'AI to predict dementia, detect cancer'},
 {'id': 103,'title': 'Figure robot learns to make coffee by watching humans for 10 hours'},
 {'id': 105,'title': "Former Microsoft CEO Steve Ballmer is now just as rich as his former boss Bill Gates. Here's how he spends his billions."},
 {'id': 210,'title': 'ChatGPT removes, then reinstates a summarization assistant without explanation.'},
 {'id': 298,'title': 'The 5 most interesting PC monitors from CES 2024'},
]

Output Specification:
You will return a JSON object with the field 'topics' containing a flat list of classification results.
For each headline input, your output will be a JSON object contain

In [35]:
# get topics
pages = paginate_df(AIdf)

# apply this prompt to AI headlines
log("start topic extraction")
response = asyncio.run(fetch_pages(pages, prompt=TOPIC_PROMPT))
log("end topic extraction")

2024-07-15 08:49:44,340 - AInewsbot - INFO - start topic extraction
2024-07-15 08:49:44,342 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:49:44,343 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:49:44,343 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:49:44,344 - AInewsbot - INFO - sent 50 items 
2024-07-15 08:49:44,345 - AInewsbot - INFO - sent 6 items 
2024-07-15 08:49:46,438 - AInewsbot - INFO - got dict with 6 items 
2024-07-15 08:49:58,959 - AInewsbot - INFO - got dict with 50 items 
2024-07-15 08:50:00,099 - AInewsbot - INFO - got dict with 50 items 
2024-07-15 08:50:00,549 - AInewsbot - INFO - got dict with 50 items 
2024-07-15 08:50:01,279 - AInewsbot - INFO - got dict with 50 items 
2024-07-15 08:50:01,282 - AInewsbot - INFO - Processed 206 responses.
2024-07-15 08:50:01,283 - AInewsbot - INFO - end topic extraction


20

In [36]:
topic_df = pd.DataFrame(response)
topic_df = topic_df.rename(columns={'topics': 'extracted_topics'})
print(len(topic_df))
topic_df.head()

206


,id,extracted_topics
0,0,"[AI, data security, stability, fairness]"
1,1,"[Microsoft, AI, terminology]"
2,2,"[Nvidia, RTX A6000, GPUs, Las Vegas Sphere, GD..."
3,3,"[Artificial Intelligence, stocks, millionaires]"
4,4,"[AI, analyst moves, Tesla, SMCI, stocks]"


In [37]:
all_topics = [item.lower() for row in topic_df.itertuples() for item in row.extracted_topics]
item_counts = Counter(all_topics)
filtered_topics = [item for item in item_counts if item_counts[item] >= 2 and item not in {'technology', 'ai', 'artificial intelligence'}]
print(len(filtered_topics))
sorted(filtered_topics)


67


['ai bias',
 'ai chips rally',
 'ai demand',
 'ai model',
 'ai privacy concerns',
 'ai revolution',
 'ai-generated content',
 "alzheimer's prediction",
 'amd',
 'analyst forecasts',
 'apple',
 'battery cell design',
 'battlefield',
 'bill gates',
 'chatbots',
 'chatgpt',
 'copilot+',
 'earnings',
 'elon musk',
 'employers',
 'ethics',
 'europe',
 'future',
 'gemini',
 'gemini ai',
 'gen ai',
 'generative ai',
 'google',
 'hollywood',
 'iatse',
 'insurers',
 'investigation',
 'investment',
 'jeff bezos',
 'las vegas sphere',
 'legislation',
 'lego',
 'lg energy solution',
 'llm',
 'mac',
 'machine learning',
 'microsoft',
 'ndas',
 'nvidia',
 'nvidia stock',
 'openai',
 'pdf files',
 'privacy',
 'privacy concerns',
 'robot',
 'samsung',
 'scanning',
 'search feature',
 'seasonic',
 'silo ai acquisition',
 'skild ai',
 'smartphone market',
 'softbank',
 'stock',
 'stock rally',
 'stocks',
 'technology',
 'tesla',
 'text generation',
 'us financial watchdog',
 'verification',
 'whistleblo

In [38]:
topic_df['extracted_topics'] = topic_df['extracted_topics'].apply(lambda l: [t.title() for t in l if t.lower() in filtered_topics])

In [40]:
CANONICAL_TOPICS = [
    "Policy and regulation",
    "Economics",
    "Robots",
    "Autonomous vehicles",
    "LLMs",
    "Healthcare",
    "Fintech",
    "Education",
    "Entertainment",
    "Funding",
    "Mergers and acquisitions",
    "Deals",
    "IPOs",
    "Ethics",
    "Laws",
    "Cybersecurity",
    "AI doom",
    'Stocks',
    'Bubble',
    'Cryptocurrency',
    'Climate',
    'Energy',
    'Nuclear',
    'Scams',
    'Privacy',
    'Intellectual Property',
    'Code assistants',
    'Customer service',
    'Reinforcement Learning',
    'Open Source',
    'Language Models',
    'Military',
    'Semiconductor chips',
    'Sustainability',
    'Agriculture',
    'Gen AI',
    'Testing',
    'RAG',
    'Data',
    'Disinformation',
    'Deepfakes',
    'Virtual Reality',
    'Cognitive Science',
    'Smart Home',
    'Authors, Writing',
    'Books, Publishing',
    'TV, Film',
    'Music',
    'Art, Design',
    'Fashion',
    'Food, Drink',
    'Travel, Adventure',
    'Health, Fitness',
    'Sports',
    'Gaming',
    'Technology',
    'Science',
    'Politics',
    'Finance',
    'History',
    'Society, Culture',
    'Lifestyle, Travel',
    'Jobs, Careers, Labor Market',
    'Products',
    'Opinion',
    'Review',

    'Nvidia',
    'Google',
    'OpenAI',
    'Meta',
    'Apple',
    'Microsoft',
    'Perplexity',
    'Salesforce',
    'Uber',
    'AMD',
    'Netflix',
    'Disney',
    'Amazon',
    'Cloudflare',
    'Anthropic',
    'Cohere',
    'Baidu',
    'Big Tech',
    'Samsung',
    'Tesla',

    'ChatGPT',
    'WhatsApp',
    'Gemini',
    'Claude',
    'Copilot',

    'Elon Musk',
    'Bill Gates',
    'Sam Altman',
    'Mustafa Suleyman',
    'Sundar Pichai',
    'Yann LeCun',
    'Geoffrey Hinton',
    'Mark Zuckerberg',

    'China',
    'European Union',
    'UK',
    'Russia',
    'Japan',
    'India',
    'Korea',
    'Taiwan',
]


In [41]:
# evergreen topics to hopefully map healdines to canonical standardized topics
# review extracted topics and add
# you could try it with new cats or new cats + evergreen
# but probably look at new cats and human in the loop should add good new cats today to evergreen list
# new_cats = list(json.loads(response.choices[0].message.content).values())[0]
# categories = sorted(list(set(new_cats + evergreen)))
categories = sorted(CANONICAL_TOPICS)
for c in categories:
    print(c)

AI doom
AMD
Agriculture
Amazon
Anthropic
Apple
Art, Design
Authors, Writing
Autonomous vehicles
Baidu
Big Tech
Bill Gates
Books, Publishing
Bubble
ChatGPT
China
Claude
Climate
Cloudflare
Code assistants
Cognitive Science
Cohere
Copilot
Cryptocurrency
Customer service
Cybersecurity
Data
Deals
Deepfakes
Disinformation
Disney
Economics
Education
Elon Musk
Energy
Entertainment
Ethics
European Union
Fashion
Finance
Fintech
Food, Drink
Funding
Gaming
Gemini
Gen AI
Geoffrey Hinton
Google
Health, Fitness
Healthcare
History
IPOs
India
Intellectual Property
Japan
Jobs, Careers, Labor Market
Korea
LLMs
Language Models
Laws
Lifestyle, Travel
Mark Zuckerberg
Mergers and acquisitions
Meta
Microsoft
Military
Music
Mustafa Suleyman
Netflix
Nuclear
Nvidia
Open Source
OpenAI
Opinion
Perplexity
Policy and regulation
Politics
Privacy
Products
RAG
Reinforcement Learning
Review
Robots
Russia
Salesforce
Sam Altman
Samsung
Scams
Science
Semiconductor chips
Smart Home
Society, Culture
Sports
Stocks
Sundar Pich

In [42]:
[t for t in filtered_topics if t.lower() not in [u.lower() for u in CANONICAL_TOPICS]]

['las vegas sphere',
 'chatbots',
 "alzheimer's prediction",
 'ai demand',
 'nvidia stock',
 'battlefield',
 'silo ai acquisition',
 'smartphone market',
 'privacy concerns',
 'copilot+',
 'future',
 'employers',
 'ai bias',
 'generative ai',
 'llm',
 'gemini ai',
 'scanning',
 'pdf files',
 'ai-generated content',
 'verification',
 'search feature',
 'ai privacy concerns',
 'stock',
 'investment',
 'hollywood',
 'europe',
 'iatse',
 'mac',
 'insurers',
 'jeff bezos',
 'softbank',
 'ai model',
 'battery cell design',
 'lg energy solution',
 'machine learning',
 'stock rally',
 'analyst forecasts',
 'ai revolution',
 'seasonic',
 'earnings',
 'ai chips rally',
 'whistleblowers',
 'legislation',
 'skild ai',
 'us financial watchdog',
 'investigation',
 'ndas',
 'text generation',
 'lego',
 'robot']

In [43]:
# maybe try to set timeout in categorize_headline
catdict = dict()

async with aiohttp.ClientSession() as session:
    for i, row in enumerate(AIdf.itertuples()):
        tasks = []
        log(f"Categorizing headline {row.id+1} of {len(AIdf)}")
        h = row.title
        log(h)
        for c in categories:
            task = asyncio.create_task(categorize_headline(h, c, session))
            tasks.append(task)
        responses = await asyncio.gather(*tasks)
        catdict[row.id] = [item for sublist in responses for item in sublist]
        log(str(catdict[row.id]))
        
catdict

2024-07-15 08:53:10,126 - AInewsbot - INFO - Categorizing headline 1 of 206
2024-07-15 08:53:10,127 - AInewsbot - INFO - 'Everything is AI now': Amid AI reality check, agencies navigate data security, stability and fairness
2024-07-15 08:53:11,400 - AInewsbot - INFO - ['Data', 'Ethics', 'Technology']
2024-07-15 08:53:11,401 - AInewsbot - INFO - Categorizing headline 2 of 206
2024-07-15 08:53:11,402 - AInewsbot - INFO - 10 terms Microsoft wants everyone should know in AI era
2024-07-15 08:53:12,383 - AInewsbot - INFO - ['Big Tech', 'Microsoft', 'Technology']
2024-07-15 08:53:12,389 - AInewsbot - INFO - Categorizing headline 3 of 206
2024-07-15 08:53:12,395 - AInewsbot - INFO - 150 Nvidia RTX A6000 GPUs power the Las Vegas Sphere - that's 7.2TB of GDDR6 memory
2024-07-15 08:53:14,381 - AInewsbot - INFO - ['Data', 'Nvidia', 'Products', 'Semiconductor chips', 'Technology']
2024-07-15 08:53:14,383 - AInewsbot - INFO - Categorizing headline 4 of 206
2024-07-15 08:53:14,384 - AInewsbot - INFO

2024-07-15 08:53:46,789 - AInewsbot - INFO - After a day of running the same prompts through three models, comparing intuitiveness and performance, I have come to a conclusion:
2024-07-15 08:53:48,170 - AInewsbot - INFO - []
2024-07-15 08:53:48,171 - AInewsbot - INFO - Categorizing headline 29 of 206
2024-07-15 08:53:48,171 - AInewsbot - INFO - After initially rejecting it, Apple has approved the first PC emulator for iOS
2024-07-15 08:53:48,990 - AInewsbot - INFO - ['Apple', 'Big Tech', 'Products', 'Technology']
2024-07-15 08:53:48,992 - AInewsbot - INFO - Categorizing headline 30 of 206
2024-07-15 08:53:48,993 - AInewsbot - INFO - Amazon Prime Day 2024: Rufus AI Assistant Is Here but Is It Helpful?
2024-07-15 08:53:50,235 - AInewsbot - INFO - ['Amazon', 'Big Tech', 'Gen AI', 'Language Models', 'Opinion', 'Review', 'Robots', 'Smart Home', 'Technology']
2024-07-15 08:53:50,236 - AInewsbot - INFO - Categorizing headline 31 of 206
2024-07-15 08:53:50,237 - AInewsbot - INFO - Amgen Hasnt 

2024-07-15 08:56:16,414 - AInewsbot - INFO - Categorizing headline 53 of 206
2024-07-15 08:56:16,414 - AInewsbot - INFO - Court ordered penalties for 15 teens who created naked AI images of classmates
2024-07-15 08:56:17,599 - AInewsbot - INFO - ['Deepfakes', 'Ethics', 'Laws', 'Policy and regulation', 'Privacy', 'Society, Culture']
2024-07-15 08:56:17,601 - AInewsbot - INFO - Categorizing headline 54 of 206
2024-07-15 08:56:17,601 - AInewsbot - INFO - Defeated by A.I., a legend in the board game go warns: Get ready for whats next
2024-07-15 08:56:19,189 - AInewsbot - INFO - ['AI doom', 'Opinion', 'Robots', 'Technology']
2024-07-15 08:56:19,190 - AInewsbot - INFO - Categorizing headline 55 of 206
2024-07-15 08:56:19,191 - AInewsbot - INFO - Dell unveils world's first AI laptop with built-in Copilot+ on Snapdragon X
2024-07-15 08:56:20,163 - AInewsbot - INFO - ['Big Tech', 'Code assistants', 'Copilot', 'Deals', 'Gen AI', 'Products', 'Technology']
2024-07-15 08:56:20,164 - AInewsbot - INF

2024-07-15 08:58:29,167 - AInewsbot - INFO - Categorizing headline 77 of 206
2024-07-15 08:58:29,168 - AInewsbot - INFO - Google AI uses enough electricity in one second to charge seven electric cars
2024-07-15 08:58:30,296 - AInewsbot - INFO - ['Big Tech', 'Climate', 'Energy', 'Google', 'Sustainability', 'Technology']
2024-07-15 08:58:30,298 - AInewsbot - INFO - Categorizing headline 78 of 206
2024-07-15 08:58:30,299 - AInewsbot - INFO - Google Close to Its Biggest Acquisition Ever, Despite Antitrust Scrutiny
2024-07-15 08:58:32,011 - AInewsbot - INFO - ['Big Tech', 'Deals', 'Google', 'Mergers and acquisitions', 'Policy and regulation', 'Technology']
2024-07-15 08:58:32,013 - AInewsbot - INFO - Categorizing headline 79 of 206
2024-07-15 08:58:32,013 - AInewsbot - INFO - Google Gemini: AI-Driven Information Verification for Accuracy
2024-07-15 08:58:33,442 - AInewsbot - INFO - ['Big Tech', 'Disinformation', 'Gemini', 'Gen AI', 'Google', 'Language Models', 'Products', 'Robots', 'Technol

2024-07-15 08:59:05,569 - AInewsbot - INFO - How to make Logics new AI-powered Session keyboard Player work for you
2024-07-15 08:59:07,868 - AInewsbot - INFO - ['Gen AI', 'Products', 'Technology']
2024-07-15 08:59:07,869 - AInewsbot - INFO - Categorizing headline 104 of 206
2024-07-15 08:59:07,871 - AInewsbot - INFO - IATSE Members Press Leaders on AI Deal Terms as Contract Ratification Vote Begins
2024-07-15 08:59:13,379 - AInewsbot - INFO - ['Deals', 'Entertainment', 'Jobs, Careers, Labor Market', 'TV, Film']
2024-07-15 08:59:13,383 - AInewsbot - INFO - Categorizing headline 105 of 206
2024-07-15 08:59:13,384 - AInewsbot - INFO - IBM and Vapor IO to deliver hybrid multicloud networking for the age of AI
2024-07-15 08:59:14,373 - AInewsbot - INFO - ['Big Tech', 'Deals', 'Products', 'Technology']
2024-07-15 08:59:14,375 - AInewsbot - INFO - Categorizing headline 106 of 206
2024-07-15 08:59:14,375 - AInewsbot - INFO - I Generated 1,000 AI SongsHere Are My Top 6 Tips
2024-07-15 08:59:15

2024-07-15 09:01:31,008 - AInewsbot - INFO - Error: 'choices'
2024-07-15 09:01:31,445 - AInewsbot - INFO - ['Entertainment', 'TV, Film']
2024-07-15 09:01:31,447 - AInewsbot - INFO - Categorizing headline 130 of 206
2024-07-15 09:01:31,447 - AInewsbot - INFO - Nvidia GeForce RTX 50 series GPUs appear on Seasonic's official wattage calculator page
2024-07-15 09:01:40,089 - AInewsbot - INFO - ['Gaming', 'Nvidia', 'Products', 'Semiconductor chips', 'Technology']
2024-07-15 09:01:40,090 - AInewsbot - INFO - Categorizing headline 131 of 206
2024-07-15 09:01:40,091 - AInewsbot - INFO - Nvidia Stock Rises. How Earnings Could Reignite the AI Chips Rally.
2024-07-15 09:01:41,123 - AInewsbot - INFO - ['Finance', 'Nvidia', 'OpenAI', 'Semiconductor chips', 'Stocks', 'Technology']
2024-07-15 09:01:41,125 - AInewsbot - INFO - Categorizing headline 132 of 206
2024-07-15 09:01:41,126 - AInewsbot - INFO - Nvidia Stock Rising. Why Earnings Can Reignite the AI Chips Rally.
2024-07-15 09:01:42,169 - AInews

2024-07-15 09:02:29,404 - AInewsbot - INFO - ['AI doom', 'Big Tech', 'Cybersecurity', 'Data', 'Ethics', 'Privacy', 'Products', 'Technology']
2024-07-15 09:02:29,406 - AInewsbot - INFO - Categorizing headline 155 of 206
2024-07-15 09:02:29,407 - AInewsbot - INFO - Recall, Limitless, Gemini: inside the AI memory machines
2024-07-15 09:02:30,673 - AInewsbot - INFO - ['Gemini', 'Gen AI', 'LLMs', 'Technology']
2024-07-15 09:02:30,674 - AInewsbot - INFO - Categorizing headline 156 of 206
2024-07-15 09:02:30,675 - AInewsbot - INFO - Researchers Make AI-Generated Board Games Using CodeLLaMa
2024-07-15 09:02:32,401 - AInewsbot - INFO - ['Code assistants', 'Gaming', 'Gen AI', 'Language Models', 'Open Source', 'Products', 'RAG', 'Reinforcement Learning', 'Science', 'Technology']
2024-07-15 09:02:32,403 - AInewsbot - INFO - Categorizing headline 157 of 206
2024-07-15 09:02:32,404 - AInewsbot - INFO - Revolutionizing Alzheimers Care With Predictive Modeling
2024-07-15 09:02:34,000 - AInewsbot - INF

2024-07-15 09:03:25,119 - AInewsbot - INFO - Categorizing headline 181 of 206
2024-07-15 09:03:25,119 - AInewsbot - INFO - The Relevance Of Relevance: Hybrid AI In Financial Services
2024-07-15 09:03:27,063 - AInewsbot - INFO - ['Finance', 'Fintech', 'Technology']
2024-07-15 09:03:27,065 - AInewsbot - INFO - Categorizing headline 182 of 206
2024-07-15 09:03:27,068 - AInewsbot - INFO - Time for businesses to move past generative AI hype and find real value
2024-07-15 09:03:34,478 - AInewsbot - INFO - ['Gen AI', 'Technology']
2024-07-15 09:03:34,479 - AInewsbot - INFO - Categorizing headline 183 of 206
2024-07-15 09:03:34,480 - AInewsbot - INFO - Tips for Effectively Training Your Machine Learning Models
2024-07-15 09:03:35,547 - AInewsbot - INFO - ['Data', 'Meta', 'Technology']
2024-07-15 09:03:35,549 - AInewsbot - INFO - Categorizing headline 184 of 206
2024-07-15 09:03:35,550 - AInewsbot - INFO - U.S. chip designer aims to bring down AI prices pushed up by Nvidia
2024-07-15 09:03:49,3

{0: ['Data', 'Ethics', 'Technology'],
 1: ['Big Tech', 'Microsoft', 'Technology'],
 2: ['Data', 'Nvidia', 'Products', 'Semiconductor chips', 'Technology'],
 3: ['Finance', 'Nvidia', 'Stocks', 'Technology'],
 4: ['Elon Musk',
  'Finance',
  'Opinion',
  'Review',
  'Stocks',
  'Technology',
  'Tesla'],
 5: ['Jobs, Careers, Labor Market', 'Technology'],
 6: ['Science', 'Technology'],
 7: ['AI doom',
  'Big Tech',
  'Energy',
  'Gen AI',
  'Nvidia',
  'OpenAI',
  'Science',
  'Sustainability',
  'Technology'],
 8: ['AI doom',
  'Climate',
  'Economics',
  'Energy',
  'Science',
  'Sustainability',
  'Technology'],
 9: ['Cognitive Science',
  'Data',
  'Health, Fitness',
  'Healthcare',
  'Review',
  'Science',
  'Technology',
  'Testing'],
 10: ['Technology'],
 11: ['AI doom', 'Entertainment', 'TV, Film'],
 12: ['Technology'],
 13: ['Big Tech', 'Products', 'Science', 'Technology'],
 14: ['Big Tech',
  'Climate',
  'Data',
  'Energy',
  'Meta',
  'Sustainability',
  'Technology'],
 15: ['O

In [44]:
topic_df["assigned_topics"] = topic_df["id"].apply(lambda id: catdict.get(id, []))
topic_df


,id,extracted_topics,assigned_topics
0,0,[],"[Data, Ethics, Technology]"
1,1,[Microsoft],"[Big Tech, Microsoft, Technology]"
2,2,"[Nvidia, Las Vegas Sphere]","[Data, Nvidia, Products, Semiconductor chips, ..."
3,3,[Stocks],"[Finance, Nvidia, Stocks, Technology]"
4,4,"[Tesla, Stocks]","[Elon Musk, Finance, Opinion, Review, Stocks, ..."
...,...,...,...
201,201,"[Lego, Robot, Technology]","[Art, Design, Big Tech, Entertainment, Ethics,..."
202,202,[],"[Data, Technology]"
203,203,[],[]
204,204,[Technology],"[Gen AI, LLMs, Language Models, OpenAI]"


In [53]:
lcategories = set([c.lower() for c in categories])

In [54]:
def clean_topics(row):
    topics = [x.title() for x in row.extracted_topics if x.lower() not in {"technology", "ai", "artificial intelligence"}]
    assigned_topics = [x.title() for x in row.assigned_topics if x.lower() in lcategories]
    combined = sorted(list(set(topics + assigned_topics)))
    combined = [s.replace("Ai", "AI") for s in combined]
    combined = [s.replace("Genai", "Gen AI") for s in combined]
    combined = [s.replace("Openai", "OpenAI") for s in combined]
    return combined

topic_df["topics"] = topic_df.apply(clean_topics, axis=1)
topic_df["topic_str"] = topic_df.apply(lambda row: ", ".join(row.topics), axis=1)
topic_df

,id,extracted_topics,assigned_topics,topics,topic_str
0,0,[],"[Data, Ethics, Technology]","[Data, Ethics, Technology]","Data, Ethics, Technology"
1,1,[Microsoft],"[Big Tech, Microsoft, Technology]","[Big Tech, Microsoft, Technology]","Big Tech, Microsoft, Technology"
2,2,"[Nvidia, Las Vegas Sphere]","[Data, Nvidia, Products, Semiconductor chips, ...","[Data, Las Vegas Sphere, Nvidia, Products, Sem...","Data, Las Vegas Sphere, Nvidia, Products, Semi..."
3,3,[Stocks],"[Finance, Nvidia, Stocks, Technology]","[Finance, Nvidia, Stocks, Technology]","Finance, Nvidia, Stocks, Technology"
4,4,"[Tesla, Stocks]","[Elon Musk, Finance, Opinion, Review, Stocks, ...","[Elon Musk, Finance, Opinion, Review, Stocks, ...","Elon Musk, Finance, Opinion, Review, Stocks, T..."
...,...,...,...,...,...
201,201,"[Lego, Robot, Technology]","[Art, Design, Big Tech, Entertainment, Ethics,...","[Art, Design, Big Tech, Entertainment, Ethics,...","Art, Design, Big Tech, Entertainment, Ethics, ..."
202,202,[],"[Data, Technology]","[Data, Technology]","Data, Technology"
203,203,[],[],[],
204,204,[Technology],"[Gen AI, LLMs, Language Models, OpenAI]","[Gen AI, Language Models, Llms, OpenAI]","Gen AI, Language Models, Llms, OpenAI"


In [55]:
merged_df = pd.merge(AIdf, topic_df[["id", "topic_str"]], on="id", how="outer")
merged_df['title_topic_str'] = merged_df.apply(lambda row: f'{row.title} (Topics: {row.topic_str})', axis=1)
merged_df


,id,url,src,title,actual_url,isAI,date,hostname,site_name,topic_str,title_topic_str
0,0,https://digiday.com/marketing/everything-is-ai...,Feedly AI,"'Everything is AI now': Amid AI reality check,...",https://digiday.com/marketing/everything-is-ai...,True,2024-07-15,digiday.com,Digiday,"Data, Ethics, Technology","'Everything is AI now': Amid AI reality check,..."
1,1,https://news.google.com/articles/CBMiigFodHRwc...,Google News,10 terms Microsoft wants everyone should know ...,https://timesofindia.indiatimes.com/technology...,True,2024-07-15,news.google.com,Google News,"Big Tech, Microsoft, Technology",10 terms Microsoft wants everyone should know ...
2,2,https://news.google.com/articles/CBMifWh0dHBzO...,Google News,150 Nvidia RTX A6000 GPUs power the Las Vegas ...,https://www.notebookcheck.net/150-Nvidia-RTX-A...,True,2024-07-15,news.google.com,Google News,"Data, Las Vegas Sphere, Nvidia, Products, Semi...",150 Nvidia RTX A6000 GPUs power the Las Vegas ...
3,3,https://news.google.com/articles/CBMiYGh0dHBzO...,Google News,3 Artificial Intelligence Stocks Set to Mint N...,https://investorplace.com/2024/07/3-artificial...,True,2024-07-15,news.google.com,Google News,"Finance, Nvidia, Stocks, Technology",3 Artificial Intelligence Stocks Set to Mint N...
4,4,https://news.google.com/articles/CBMicGh0dHBzO...,Google News,5 big analyst AI moves: Tesla and SMCI stocks ...,https://www.investing.com/news/stock-market-ne...,True,2024-07-15,news.google.com,Google News,"Elon Musk, Finance, Opinion, Review, Stocks, T...",5 big analyst AI moves: Tesla and SMCI stocks ...
...,...,...,...,...,...,...,...,...,...,...,...
201,201,https://news.google.com/articles/CBMiK2h0dHBzO...,Google News,Would you want to chat with this creepy-lookin...,https://www.foxnews.com/video/6357757492112,True,2024-07-15,news.google.com,Google News,"Art, Design, Big Tech, Entertainment, Ethics, ...",Would you want to chat with this creepy-lookin...
202,202,https://www.utilitydive.com/spons/you-cant-hav...,Feedly AI,You cant have an AI strategy without a data st...,https://www.utilitydive.com/spons/you-cant-hav...,True,2024-07-15,www.utilitydive.com,Utility Dive,"Data, Technology",You cant have an AI strategy without a data st...
203,203,https://www.reddit.com/r/ChatGPT/comments/1e3h...,Reddit,"ah, a cultured AI indeed",https://www.reddit.com/r/ChatGPT/comments/1e3h...,True,2024-07-15,www.reddit.com,Reddit,,"ah, a cultured AI indeed (Topics: )"
204,204,https://www.reddit.com/r/ProgrammerHumor/comme...,Reddit,largeLanguageModels,https://www.reddit.com/r/ProgrammerHumor/comme...,True,2024-07-15,www.reddit.com,Reddit,"Gen AI, Language Models, Llms, OpenAI","largeLanguageModels (Topics: Gen AI, Language ..."


In [56]:
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    display(merged_df.loc[merged_df["topic_str"]==""][['title']])


,title
27,"After a day of running the same prompts through three models, comparing intuitiveness and performance, I have come to a conclusion:"
111,"In this age of misinformation and AI growth, it's more important now than ever to ensure we are able to distinguish between fabrication and reality. It may come as a shock but this is, in fact, AI."
203,"ah, a cultured AI indeed"


In [57]:
AIdf = merged_df

### Semantic sort

In [58]:
# use embeddings to sort headlines by semantical similarity
log(f"Fetching embeddings for {len(AIdf)} headlines")
embedding_model = 'text-embedding-3-large'
response = client.embeddings.create(input=AIdf['title_topic_str'].tolist(),
                                    model=embedding_model)
embedding_df = pd.DataFrame([e.model_dump()['embedding'] for e in response.data])

# sort of a traveling salesman sort
log(f"Sort with agglomerative cluster sort")
sorted_indices = agglomerative_cluster_sort(embedding_df)
AIdf['sort_order'] = sorted_indices

# do dimensionality reduction on embedding_df and cluster analysis
log(f"Perform dimensionality reduction")
with open("reducer.pkl", 'rb') as file:
    # Load the model from the file
    reducer = pickle.load(file)
reduced_data = reducer.transform(embedding_df)
log(f"Cluster with DBSCAN")
dbscan = DBSCAN(eps=0.4, min_samples=3)  # Adjust eps and min_samples as needed
AIdf['cluster'] = dbscan.fit_predict(reduced_data)
AIdf.loc[AIdf['cluster'] == -1, 'cluster'] = 999
    
# sort first by clusters found by DBSCAN, then by semantic ordering
AIdf = AIdf.sort_values(['cluster', 'sort_order']) \
    .reset_index(drop=True) \
    .reset_index() \
    .drop(columns=["id"]) \
    .rename(columns={'index': 'id'})

AIdf


2024-07-15 09:13:32,497 - AInewsbot - INFO - Fetching embeddings for 206 headlines
2024-07-15 09:13:33,114 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-15 09:13:33,662 - AInewsbot - INFO - Sort with agglomerative cluster sort


,id,url,src,title,actual_url,isAI,date,hostname,site_name,topic_str,title_topic_str,sort_order,cluster
0,0,https://www.fool.com/investing/2024/07/15/here...,Feedly AI,Here Is My TopArtificial Intelligence(AI) Stoc...,https://www.fool.com/investing/2024/07/15/here...,True,2024-07-15,www.fool.com,The Motley Fool,"Finance, Investment, Nvidia, Stock, Stocks, Te...",Here Is My TopArtificial Intelligence(AI) Stoc...,53,0
1,1,https://news.google.com/articles/CBMiYGh0dHBzO...,Google News,3 Artificial Intelligence Stocks Set to Mint N...,https://investorplace.com/2024/07/3-artificial...,True,2024-07-15,news.google.com,Google News,"Finance, Nvidia, Stocks, Technology",3 Artificial Intelligence Stocks Set to Mint N...,129,0
2,2,https://news.google.com/articles/CBMiamh0dHBzO...,Google News,Sterling Infrastructure: The Picks And Shovels...,https://seekingalpha.com/article/4703935-sterl...,True,2024-07-15,news.google.com,Google News,"Stocks, Technology",Sterling Infrastructure: The Picks And Shovels...,203,0
3,3,https://www.scmp.com/business/article/3270513/...,Feedly AI,Paris Olympics will use Alibabas AI-powered en...,https://www.scmp.com/business/article/3270513/...,True,2024-07-15,www.scmp.com,South China Morning Post,"Big Tech, Climate, Energy, Products, Sustainab...",Paris Olympics will use Alibabas AI-powered en...,30,1
4,4,https://news.google.com/articles/CBMiVGh0dHBzO...,Google News,A.I.s Insatiable Appetite for Energy,https://www.nytimes.com/2024/07/11/climate/art...,True,2024-07-15,news.google.com,Google News,"AI Doom, Big Tech, Energy, Gen AI, Nvidia, Ope...",A.I.s Insatiable Appetite for Energy (Topics: ...,116,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,201,https://news.google.com/articles/CBMiY2h0dHBzO...,Google News,People are falling in love with AI. Should we ...,https://www.livescience.com/health/relationshi...,True,2024-07-15,news.google.com,Google News,"AI Doom, Gen AI, Opinion, Technology",People are falling in love with AI. Should we ...,195,999
202,202,https://news.google.com/articles/CBMiiQFodHRwc...,Google News,mini PC identify management artificial intelli...,https://www.militaryaerospace.com/computers/ar...,True,2024-07-15,news.google.com,Google News,"Gen AI, Nvidia, Products, Technology",mini PC identify management artificial intelli...,198,999
203,203,https://www.marktechpost.com/2024/07/14/opengp...,Feedly AI,OpenGPT-X Team Publishes European LLM Leaderbo...,https://www.marktechpost.com/2024/07/14/opengp...,True,2024-07-15,www.marktechpost.com,MarkTechPost,"Cohere, Language Models, Llms, Open Source, Te...",OpenGPT-X Team Publishes European LLM Leaderbo...,201,999
204,204,https://www.techradar.com/phones/samsung-galax...,Feedly AI,Samsung's latest Galaxy AI features tipped to ...,https://www.techradar.com/phones/samsung-galax...,True,2024-07-15,www.techradar.com,TechRadar,"Big Tech, Korea, Products, Samsung, Technology",Samsung's latest Galaxy AI features tipped to ...,202,999


In [59]:
async def write_topic_name(session, topic_list_str, max_retries=3, model=LOWCOST_MODEL):

    TOPIC_WRITER_PROMPT = f"""
You are a topic writing assistant. I will provide a list of headlines with extracted topics in parentheses. 
Your task is to propose a name for a topic that very simply, clearly and accurately captures all the provided 
headlines in less than 7 words. You will output a JSON object with the key "topic_title".

Example Input:
In the latest issue of Caixins weekly magazine: CATL Bets on 'Skateboard Chassis' and Battery Swaps to Dispell Market Concerns (powered by AI) (Topics: Battery Swaps, Catl, China, Market Concerns, Skateboard Chassis)

AI, cheap EVs, future Chevy  the week (Topics: Chevy, Evs)

Electric Vehicles and AI: Driving the Consumer & World Forward (Topics: Consumer, Electric Vehicles, Technology)

Example Output:
{{"topic_title": "Electric Vehicles"}}

Task
Propose the name for the overall topic based on the following provided headlines and individual topics:

{topic_list_str}
"""

    for i in range(max_retries):
        try:
            messages=[
                      {"role": "user", "content": TOPIC_WRITER_PROMPT
                      }]

            payload = {"model":  model,
                       "response_format": {"type": "json_object"},
                       "messages": messages,
                       "temperature": 0
                       }
            response = await fetch_openai(session, payload)
            response_dict = json.loads(response["choices"][0]["message"]["content"])
            return response_dict

            break
        except Exception as exc:
            log(f"Error: {exc}")

    return {}
        

# show clusters
cluster_topics = []
with pd.option_context('display.max_rows', None, 'display.max_colwidth', None):
    async with aiohttp.ClientSession() as session:

        for i in range(30):
            tmpdf = AIdf.loc[AIdf['cluster']==i][["id", "title_topic_str"]]
            if len(tmpdf) ==0:
                break
            display(tmpdf)
            title_topic_str_list = ("\n\n".join(tmpdf['title_topic_str'].to_list()))
            cluster_topic = await write_topic_name(session, title_topic_str_list)
            cluster_topics.append(cluster_topic)
            print(cluster_topic)

    


,id,title_topic_str
0,0,"Here Is My TopArtificial Intelligence(AI) Stock to Buy Right Now (Topics: Finance, Investment, Nvidia, Stock, Stocks, Technology)"
1,1,"3 Artificial Intelligence Stocks Set to Mint New Millionaires (Topics: Finance, Nvidia, Stocks, Technology)"
2,2,"Sterling Infrastructure: The Picks And Shovels To The Future Of AI (NASDAQ:STRL) (Topics: Stocks, Technology)"


{'topic_title': 'AI Stocks'}


,id,title_topic_str
3,3,"Paris Olympics will use Alibabas AI-powered energy-saving tool to cut venue emissions (Topics: Big Tech, Climate, Energy, Products, Sustainability, Technology)"
4,4,"A.I.s Insatiable Appetite for Energy (Topics: AI Doom, Big Tech, Energy, Gen AI, Nvidia, OpenAI, Science, Sustainability, Technology)"
5,5,"AI's Rapid Growth Threatens Energy Industry, Economy, and Climate (Topics: AI Doom, Climate, Economics, Energy, Science, Sustainability, Technology)"
6,6,"AI demand puts more pressure on data centers' energy use. Here's how to make it sustainable (Topics: AI Demand, Big Tech, Climate, Data, Energy, Meta, Sustainability, Technology)"


{'topic_title': 'AI Energy Sustainability'}


,id,title_topic_str
7,7,"AI Better at Predicting Progression to Alzheimer's Than Standard Care (Topics: Alzheimer'S Prediction, Cognitive Science, Data, Health, Fitness, Healthcare, Review, Science, Technology, Testing)"
8,8,"Artificial intelligence outperforms clinical tests at predicting progress of Alzheimer's disease (Topics: Cognitive Science, Gen AI, Health, Fitness, Healthcare, Science, Technology, Testing)"
9,9,"Revolutionizing Alzheimers Care With Predictive Modeling (Topics: Cognitive Science, Data, Health, Fitness, Healthcare, Science, Technology)"
10,10,"AI tool is better at predicting Alzheimers disease than current tests (Topics: Alzheimer'S Prediction, Cognitive Science, Health, Fitness, Healthcare, Science, Technology, Testing)"
11,11,"AI predicts Alzheimers disease risk with remarkable accuracy through subtle speech patterns (Topics: Cognitive Science, Data, Health, Fitness, Healthcare, Language Models, Science, Technology, Testing)"


...

{'topic_title': 'AI-Powered Battery Design Innovation'}


,id,title_topic_str
77,77,"OpenAI says there are 5 'levels' for AI to reach human intelligence it's already almost at level 2 (Topics: OpenAI, Science, Technology)"
78,78,"OpenAI is building a new AI model under code name Strawberry Report (Topics: AI Model, Big Tech, Gen AI, Language Models, Llms, Meta, OpenAI, Science, Technology)"
79,79,"OpenAI is reportedly working on more advanced AI models capable of reasoning and deep research (Topics: Big Tech, Gen AI, Language Models, OpenAI, Rag, Science, Technology)"
80,80,"OpenAI Defines Five Steps From AI to AGI (Topics: Big Tech, Gen AI, Meta, OpenAI, Science, Technology)"
81,81,"Open AI is working on new reasoning AI technology (Topics: Big Tech, Cognitive Science, OpenAI, Products, Science, Technology)"


{'topic_title': 'Advanced AI Development'}


,id,title_topic_str
82,82,"Open AI reportedly stopped staffers from warning about security risks (Topics: AI Doom, Big Tech, Copilot, Cybersecurity, Ethics, Language Models, OpenAI, Policy And Regulation, Technology)"
83,83,"US financial watchdog urged to investigate NDAs at OpenAI | Whistleblowers say contracts include restrictions requiring staff to seek permission before contacting regulators (Topics: Big Tech, Ethics, Investigation, Ndas, OpenAI, Policy And Regulation, Technology, Us Financial Watchdog, Whistleblowers)"
84,84,"OpenAI whistleblowers ask SEC to investigate alleged restrictive non-disclosure agreements (Topics: Ethics, Language Models, OpenAI, Policy And Regulation, Sam Altman, Technology, Whistleblowers)"


{'topic_title': 'OpenAI Ethics Investigation'}


In [60]:
# we could extract top words using tfidf, something like 
# vectorizer = TfidfVectorizer(stop_words='english')
# tfidf_matrix = vectorizer.fit_transform(documents)
# feature_names = vectorizer.get_feature_names_out()
# topics = []
# for i in range(n_topics):
#     # Get the documents in this cluster
#     cluster_docs = [doc for doc, label in zip(documents, cluster_labels) if label == i]
#     cluster_metadatas = [meta for meta, label in zip(metadatas, cluster_labels) if label == i]

#     # Get the top words for this cluster based on TF-IDF scores
#     tfidf_scores = tfidf_matrix[cluster_labels == i].sum(axis=0).A1
#     top_word_indices = tfidf_scores.argsort()[-n_words_per_topic:][::-1]
#     top_words = [feature_names[index] for index in top_word_indices]


In [61]:
cluster_topic_list = [obj['topic_title'] for obj in cluster_topics]
cluster_topic_list

['AI Stocks',
 'AI Energy Sustainability',
 "AI Predicting Alzheimer's Progression",
 'AI in the Workplace',
 'Nvidia and AI Chips Investment',
 'AI Privacy Concerns',
 "Apple's AI Stock Performance",
 'AI in Business Technology',
 'Language Models',
 'ChatGPT Desktop App on Mac',
 'AI Chatbots and Language Models',
 'AI in Entertainment Industry',
 'AI in Healthcare',
 'Artificial Intelligence Funding for Robotics',
 'AI-Powered Battery Design Innovation',
 'Advanced AI Development',
 'OpenAI Ethics Investigation']

In [62]:
AIdf['cluster_name'] = AIdf['cluster'].apply(lambda i: cluster_topic_list[i] if i<len(cluster_topic_list) else "")


# Save and email headlines


In [63]:
html_str = ""
for row in AIdf.itertuples():
    log(f"[{row.Index}. {row.title} - {row.src}]({row.actual_url})")
    html_str += f'{row.Index}.<a href="{row.url}">{row.title} - {row.src}</a><br />\n'


2024-07-15 09:15:15,623 - AInewsbot - INFO - [0. Here Is My TopArtificial Intelligence(AI) Stock to Buy Right Now - Feedly AI](https://www.fool.com/investing/2024/07/15/heres-my-top-ai-stock-buy-now/)
2024-07-15 09:15:15,626 - AInewsbot - INFO - [1. 3 Artificial Intelligence Stocks Set to Mint New Millionaires - Google News](https://investorplace.com/2024/07/3-artificial-intelligence-stocks-set-to-mint-new-millionaires/)
2024-07-15 09:15:15,627 - AInewsbot - INFO - [2. Sterling Infrastructure: The Picks And Shovels To The Future Of AI (NASDAQ:STRL) - Google News](https://seekingalpha.com/article/4703935-sterling-infrastructure-the-picks-and-shovels-to-the-future-of-ai)
2024-07-15 09:15:15,628 - AInewsbot - INFO - [3. Paris Olympics will use Alibabas AI-powered energy-saving tool to cut venue emissions - Feedly AI](https://www.scmp.com/business/article/3270513/paris-olympics-will-use-alibabas-ai-powered-energy-saving-tool-cut-venue-emissions)
2024-07-15 09:15:15,629 - AInewsbot - INFO -

2024-07-15 09:15:15,642 - AInewsbot - INFO - [33. Google Gemini may be peeking into people's files, even when told not to - Google News](https://www.xda-developers.com/google-gemini-peeking-peoples-files/)
2024-07-15 09:15:15,642 - AInewsbot - INFO - [34. Google Gemini now allows users to double-check AI-generated information using Google search - Google News](https://indianexpress.com/article/technology/artificial-intelligence/google-gemini-check-ai-generated-information-using-google-search-9453995/)
2024-07-15 09:15:15,643 - AInewsbot - INFO - [35. Is Apple Inc (NASDAQ:AAPL) Aswath Damodarans Best AI Stock Pick? - Feedly AI](https://sg.finance.yahoo.com/news/apple-inc-nasdaq-aapl-aswath-214249845.html)
2024-07-15 09:15:15,643 - AInewsbot - INFO - [36. Apple Is Top Pick at Morgan Stanley as AI Seen Driving iPhone, iPad Upgrades - Bloomberg Tech](https://www.bloomberg.com/news/articles/2024-07-15/apple-is-top-pick-at-morgan-stanley-as-ai-seen-driving-iphone-ipad-upgrades)
2024-07-15 09

2024-07-15 09:15:15,652 - AInewsbot - INFO - [69. Jeff Bezos And Masayoshi Son's SoftBank Lead $300M Funding Round For Startup Working On Developing 'General Purpose Brain' For AI Systems - Feedly AI](https://www.benzinga.com/news/24/07/39766402/jeff-bezos-and-masayoshi-sons-softbank-lead-300m-funding-round-for-startup-working-on-developing-gen)
2024-07-15 09:15:15,652 - AInewsbot - INFO - [70. Skild AI Raises $300M Series A To Build A Scalable AI Foundation Model For Robotics - Google News](https://www.businesswire.com/news/home/20240709306400/en/Skild-AI-Raises-300M-Series-A-To-Build-A-Scalable-AI-Foundation-Model-For-Robotics)
2024-07-15 09:15:15,653 - AInewsbot - INFO - [71. New LG Energy Solution AI designs battery cells in less than a day - Google News](https://koreajoongangdaily.joins.com/news/2024-07-14/business/industry/New-LG-Energy-Solution-AI-designs-battery-cells-in-less-than-a-day/2089779)
2024-07-15 09:15:15,653 - AInewsbot - INFO - [72. Spotlight on Lightmatter, the Pho

2024-07-15 09:15:15,658 - AInewsbot - INFO - [103. Amazon Prime Day 2024: Rufus AI Assistant Is Here but Is It Helpful? - Feedly AI](https://www.techtimes.com/articles/306669/20240715/amazon-prime-day-2024-rufus-ai-assistant-here-helpful.htm)
2024-07-15 09:15:15,658 - AInewsbot - INFO - [104. Apple Intelligence: The Next Frontier in AI Development - Google News](https://investorplace.com/hypergrowthinvesting/2024/07/apple-intelligence-the-next-frontier-in-ai-development/)
2024-07-15 09:15:15,659 - AInewsbot - INFO - [105. Employers Find Openings to Share AI Bias Liability With Vendors - Feedly AI](https://news.bloomberglaw.com/daily-labor-report/employers-find-openings-to-share-ai-bias-liability-with-vendors)
2024-07-15 09:15:15,659 - AInewsbot - INFO - [106. I Generated 1,000 AI SongsHere Are My Top 6 Tips - Google News](https://www.howtogeek.com/i-generated-1000-ai-songs-here-are-my-top-tips/)
2024-07-15 09:15:15,659 - AInewsbot - INFO - [107. Nvidia GeForce RTX 50 series GPUs appear

2024-07-15 09:15:15,663 - AInewsbot - INFO - [138. Evolving scientific discovery by unifying data and background knowledge with AI Hilbert - Google News](https://www.nature.com/articles/s41467-024-50074-w)
2024-07-15 09:15:15,663 - AInewsbot - INFO - [139. From basement to battlefield: Ukrainian startups create low-cost robots to fight Russia - Feedly AI](https://www.newsday.com/news/nation/ukraine-russia-drones-robots-artificial-intelligence-y99911)
2024-07-15 09:15:15,663 - AInewsbot - INFO - [140. Dell unveils world's first AI laptop with built-in Copilot+ on Snapdragon X - Google News](https://www.jpost.com/brandblend/dell-unveils-worlds-first-ai-laptop-with-built-in-copilot-on-snapdragon-x-809735)
2024-07-15 09:15:15,663 - AInewsbot - INFO - [141. AMD: Building An AI Ecosystem (NASDAQ:AMD) - Google News](https://seekingalpha.com/article/4703737-amd-building-an-ai-ecosystem)
2024-07-15 09:15:15,663 - AInewsbot - INFO - [142. AMD Builds Out Its AI Software And Services Stack With Si

2024-07-15 09:15:15,668 - AInewsbot - INFO - [173. Hollywood Stars Take Sides on Using AI and Deepfakes in Films - Google News](https://digitalchew.com/2024/07/09/hollywood-stars-take-sides-on-using-ai-and-deepfakes-in-films/)
2024-07-15 09:15:15,668 - AInewsbot - INFO - [174. Would you want to chat with this creepy-looking Lego head powered by AI - Google News](https://www.foxnews.com/video/6357757492112)
2024-07-15 09:15:15,668 - AInewsbot - INFO - [175. Q&A: AI vs. the metaverseHow artificial intelligence might change the future of the internet - Google News](https://techxplore.com/news/2024-07-qa-ai-metaverse-artificial-intelligence.html)
2024-07-15 09:15:15,669 - AInewsbot - INFO - [176. Pinterest Outlines AI Background Generation Process for Product Shots - Google News](https://www.socialmediatoday.com/news/pinterest-outlines-ai-background-generation-process-for-product-shots/721304/)
2024-07-15 09:15:15,669 - AInewsbot - INFO - [177. Get Ready for More AI Mania This Earnings Sea

In [64]:
# save headlines
with open('headlines.html', 'w') as f:
    f.write(html_str)


In [65]:
# send mail
log("Sending headlines email")
subject = f'AI headlines {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


2024-07-15 09:15:17,438 - AInewsbot - INFO - Sending headlines email


# Save individual pages 

In [66]:
# fetch pages
# Create a queue for multiprocessing and populate it 
log("Queuing URLs for scraping")

queue = multiprocessing.Queue()
for row in AIdf.itertuples():
    queue.put((row.id, row.url, row.title))


2024-07-15 09:15:20,491 - AInewsbot - INFO - Queuing URLs for scraping


In [67]:
# scrape urls in queue asynchronously
num_browsers = 4

callable = process_url_queue_factory(queue)

log(f"fetching {len(AIdf)} pages using {num_browsers} browsers")
saved_pages = launch_drivers(num_browsers, callable)


2024-07-15 09:15:20,851 - AInewsbot - INFO - fetching 206 pages using 4 browsers
2024-07-15 09:15:20,854 - AInewsbot - INFO - get_driver - 64568 Initializing webdriver
2024-07-15 09:15:20,857 - AInewsbot - INFO - get_driver - 64568 Initializing webdriver
2024-07-15 09:15:20,858 - AInewsbot - INFO - get_driver - 64568 Initializing webdriver
2024-07-15 09:15:20,859 - AInewsbot - INFO - get_driver - 64568 Initializing webdriver
2024-07-15 09:15:41,862 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-15 09:15:41,863 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-15 09:15:41,864 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-15 09:15:41,865 - AInewsbot - INFO - get_driver - Initialized webdriver profile
2024-07-15 09:15:41,865 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-15 09:15:41,865 - AInewsbot - INFO - get_driver - Initialized webdriver service
2024-07-15 09:15:41,866 - AInewsbot - INFO 

2024-07-15 09:17:34,223 - AInewsbot - INFO - get_url(A.I.s Insatiable Appetite for Energy) - Saving A_I_s_Insatiable_Appetite_for_Energy_20240715_091734.html as utf-8
2024-07-15 09:17:34,227 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMiUmh0dHBzOi8vc2NpdGVjaGRhaWx5LmNvbS9yZXZvbHV0aW9uaXppbmctYWx6aGVpbWVycy1jYXJlLXdpdGgtcHJlZGljdGl2ZS1tb2RlbGluZy_SAQA
2024-07-15 09:17:34,227 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiUmh0dHBzOi8vc2NpdGVjaGRhaWx5LmNvbS9yZXZvbHV0aW9uaXppbmctYWx6aGVpbWVycy1jYXJlLXdpdGgtcHJlZGljdGl2ZS1tb2RlbGluZy_SAQA) - starting get_url https://news.google.com/articles/CBMiUmh0dHBzOi8vc2NpdGVjaGRhaWx5LmNvbS9yZXZvbHV0aW9uaXppbmctYWx6aGVpbWVycy1jYXJlLXdpdGgtcHJlZGljdGl2ZS1tb2RlbGluZy_SAQA
2024-07-15 09:17:36,026 - AInewsbot - INFO - get_url(AI demand puts more pressure on data centers' energy use. Here's how to make it sustainable) - Saving AI_demand_puts_more_pressure_on_data_centers__energy_use__Here_s_how_to_make_it_sustainabl

2024-07-15 09:17:57,176 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiRWh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL252aWRpYS1zdG9jay1wcmljZS1idXktc2VsbC0yODc0N2MzNdIBSWh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FtcC9hcnRpY2xlcy9udmlkaWEtc3RvY2stcHJpY2UtYnV5LXNlbGwtMjg3NDdjMzU) - starting get_url https://news.google.com/articles/CBMiRWh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL252aWRpYS1zdG9jay1wcmljZS1idXktc2VsbC0yODc0N2MzNdIBSWh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FtcC9hcnRpY2xlcy9udmlkaWEtc3RvY2stcHJpY2UtYnV5LXNlbGwtMjg3NDdjMzU
2024-07-15 09:18:06,430 - AInewsbot - INFO - get_url(Software workers across the globe are excited to get using Gen AI at work) - Saving Software_workers_across_the_globe_are_excited_to_get_using_Gen_AI_at_work_20240715_091806.html as utf-8
2024-07-15 09:18:06,434 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMic2h0dHBzOi8vaW52ZXN0b3JwbGFjZS5jb20vMjAyNC8wNy9haXMtZGlydHktc2VjcmV0LXdoeS1udmlkaWEtc3RvY2staW52ZXN0b3JzLXNob3VsZC1mZWFyLXRoZS1

2024-07-15 09:18:27,337 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMiamh0dHBzOi8vd3d3LnRlY2h6aW5lLmV1L25ld3MvcHJpdmFjeS1jb21wbGlhbmNlLzEyMjI3NC9nb29nbGUtZ2VtaW5pLXJlYWRzLWRyaXZlLWZpbGVzLXdpdGhvdXQtcGVybWlzc2lvbi_SAQA
2024-07-15 09:18:27,339 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiamh0dHBzOi8vd3d3LnRlY2h6aW5lLmV1L25ld3MvcHJpdmFjeS1jb21wbGlhbmNlLzEyMjI3NC9nb29nbGUtZ2VtaW5pLXJlYWRzLWRyaXZlLWZpbGVzLXdpdGhvdXQtcGVybWlzc2lvbi_SAQA) - starting get_url https://news.google.com/articles/CBMiamh0dHBzOi8vd3d3LnRlY2h6aW5lLmV1L25ld3MvcHJpdmFjeS1jb21wbGlhbmNlLzEyMjI3NC9nb29nbGUtZ2VtaW5pLXJlYWRzLWRyaXZlLWZpbGVzLXdpdGhvdXQtcGVybWlzc2lvbi_SAQA
2024-07-15 09:18:27,489 - AInewsbot - INFO - get_url(Is Now The Time To Look At Buying NVIDIA Corporation (NASDAQ:NVDA)?) - Saving Is_Now_The_Time_To_Look_At_Buying_NVIDIA_Corporation__NASDAQ_NVDA___20240715_091827.html as utf-8
2024-07-15 09:18:27,494 - AInewsbot - INFO - Processing https://news.google.com/article

2024-07-15 09:18:43,204 - AInewsbot - INFO - get_url(Actionable Threat Intelligence at Google Scale: Meet Google Threat Intelligence Powered by Gemini) - Saving Actionable_Threat_Intelligence_at_Google_Scale__Meet_Google_Threat_Intelligence_Powered_by_Gemini_20240715_091843.html as utf-8
2024-07-15 09:18:43,208 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMijgFodHRwczovL2luZGlhbmV4cHJlc3MuY29tL2FydGljbGUvdGVjaG5vbG9neS9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9nb29nbGUtZ2VtaW5pLWNoZWNrLWFpLWdlbmVyYXRlZC1pbmZvcm1hdGlvbi11c2luZy1nb29nbGUtc2VhcmNoLTk0NTM5OTUv0gGTAWh0dHBzOi8vaW5kaWFuZXhwcmVzcy5jb20vYXJ0aWNsZS90ZWNobm9sb2d5L2FydGlmaWNpYWwtaW50ZWxsaWdlbmNlL2dvb2dsZS1nZW1pbmktY2hlY2stYWktZ2VuZXJhdGVkLWluZm9ybWF0aW9uLXVzaW5nLWdvb2dsZS1zZWFyY2gtOTQ1Mzk5NS9saXRlLw
2024-07-15 09:18:43,209 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMijgFodHRwczovL2luZGlhbmV4cHJlc3MuY29tL2FydGljbGUvdGVjaG5vbG9neS9hcnRpZmljaWFsLWludGVsbGlnZW5jZS9nb29nbGUtZ2VtaW5pLWNoZWNrLWFpLWdlbmVyY

...

2024-07-15 09:24:36,828 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LnRoZWd1YXJkaWFuLmNvbS90ZWNobm9sb2d5L2FydGljbGUvMjAyNC9qdWwvMTQvYWlzLW9wcGVuaGVpbWVyLW1vbWVudC1hdXRvbm9tb3VzLXdlYXBvbnMtZW50ZXItdGhlLWJhdHRsZWZpZWxk0gF6aHR0cHM6Ly9hbXAudGhlZ3VhcmRpYW4uY29tL3RlY2hub2xvZ3kvYXJ0aWNsZS8yMDI0L2p1bC8xNC9haXMtb3BwZW5oZWltZXItbW9tZW50LWF1dG9ub21vdXMtd2VhcG9ucy1lbnRlci10aGUtYmF0dGxlZmllbGQ
2024-07-15 09:24:36,828 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LnRoZWd1YXJkaWFuLmNvbS90ZWNobm9sb2d5L2FydGljbGUvMjAyNC9qdWwvMTQvYWlzLW9wcGVuaGVpbWVyLW1vbWVudC1hdXRvbm9tb3VzLXdlYXBvbnMtZW50ZXItdGhlLWJhdHRsZWZpZWxk0gF6aHR0cHM6Ly9hbXAudGhlZ3VhcmRpYW4uY29tL3RlY2hub2xvZ3kvYXJ0aWNsZS8yMDI0L2p1bC8xNC9haXMtb3BwZW5oZWltZXItbW9tZW50LWF1dG9ub21vdXMtd2VhcG9ucy1lbnRlci10aGUtYmF0dGxlZmllbGQ) - starting get_url https://news.google.com/articles/CBMiemh0dHBzOi8vd3d3LnRoZWd1YXJkaWFuLmNvbS90ZWNobm9sb2d5L2FydGljbGUvMjAyNC9qdWwvMTQvYWlzLW9wcGVuaGVp

2024-07-15 09:25:01,594 - AInewsbot - INFO - Processing https://venturebeat.com/ai/hp-launches-ai-pcs-with-focus-on-security-and-trust/
2024-07-15 09:25:01,596 - AInewsbot - INFO - get_url(https://venturebeat.com/ai/hp-launches-ai-pcs-with-focus-on-security-and-trust/) - starting get_url https://venturebeat.com/ai/hp-launches-ai-pcs-with-focus-on-security-and-trust/
2024-07-15 09:25:03,895 - AInewsbot - INFO - get_url(How Build Your Own AI Confessional: How to Add a Voice to the LLM) - Saving How_Build_Your_Own_AI_Confessional__How_to_Add_a_Voice_to_the_LLM_20240715_092503.html as utf-8
2024-07-15 09:25:03,897 - AInewsbot - INFO - Processing https://www.cnet.com/videos/hands-on-with-the-first-ai-teddy-bear-what-my-kids-thought/
2024-07-15 09:25:03,897 - AInewsbot - INFO - get_url(https://www.cnet.com/videos/hands-on-with-the-first-ai-teddy-bear-what-my-kids-thought/) - starting get_url https://www.cnet.com/videos/hands-on-with-the-first-ai-teddy-bear-what-my-kids-thought/
2024-07-15 09

2024-07-15 09:25:27,993 - AInewsbot - INFO - get_url(Pinterest Outlines AI Background Generation Process for Product Shots) - Saving Pinterest_Outlines_AI_Background_Generation_Process_for_Product_Shots_20240715_092527.html as utf-8
2024-07-15 09:25:27,995 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMicGh0dHBzOi8vd3d3LmludmVzdGluZy5jb20vbmV3cy9zdG9jay1tYXJrZXQtbmV3cy81LWJpZy1hbmFseXN0LWFpLW1vdmVzLXRlc2xhLWFuZC1zbWNpLXN0b2Nrcy1kb3duZ3JhZGVkLTM1MTgyNTfSAQA
2024-07-15 09:25:27,995 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMicGh0dHBzOi8vd3d3LmludmVzdGluZy5jb20vbmV3cy9zdG9jay1tYXJrZXQtbmV3cy81LWJpZy1hbmFseXN0LWFpLW1vdmVzLXRlc2xhLWFuZC1zbWNpLXN0b2Nrcy1kb3duZ3JhZGVkLTM1MTgyNTfSAQA) - starting get_url https://news.google.com/articles/CBMicGh0dHBzOi8vd3d3LmludmVzdGluZy5jb20vbmV3cy9zdG9jay1tYXJrZXQtbmV3cy81LWJpZy1hbmFseXN0LWFpLW1vdmVzLXRlc2xhLWFuZC1zbWNpLXN0b2Nrcy1kb3duZ3JhZGVkLTM1MTgyNTfSAQA
2024-07-15 09:25:31,848 - AInewsbot - INFO - get_url(Get Re

2024-07-15 09:25:56,170 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMidGh0dHBzOi8vd3d3LnNjbXAuY29tL25ld3MvYXNpYS9lYXN0LWFzaWEvYXJ0aWNsZS8zMjcwNDE1L2phcGFuLXN0YXJ0dXAtdHVybnMtaGVyLWFpLWRhdGluZy1yZWFsaXR5LXNoZXMtYmVjb21lLWhhYml00gF0aHR0cHM6Ly9hbXAuc2NtcC5jb20vbmV3cy9hc2lhL2Vhc3QtYXNpYS9hcnRpY2xlLzMyNzA0MTUvamFwYW4tc3RhcnR1cC10dXJucy1oZXItYWktZGF0aW5nLXJlYWxpdHktc2hlcy1iZWNvbWUtaGFiaXQ) - starting get_url https://news.google.com/articles/CBMidGh0dHBzOi8vd3d3LnNjbXAuY29tL25ld3MvYXNpYS9lYXN0LWFzaWEvYXJ0aWNsZS8zMjcwNDE1L2phcGFuLXN0YXJ0dXAtdHVybnMtaGVyLWFpLWRhdGluZy1yZWFsaXR5LXNoZXMtYmVjb21lLWhhYml00gF0aHR0cHM6Ly9hbXAuc2NtcC5jb20vbmV3cy9hc2lhL2Vhc3QtYXNpYS9hcnRpY2xlLzMyNzA0MTUvamFwYW4tc3RhcnR1cC10dXJucy1oZXItYWktZGF0aW5nLXJlYWxpdHktc2hlcy1iZWNvbWUtaGFiaXQ
2024-07-15 09:25:57,757 - AInewsbot - INFO - get_url(Apple Approves First Retro PC Emulator for iPhone, iPad, and Vision Pro) - Saving Apple_Approves_First_Retro_PC_Emulator_for_iPhone__iPad__and_Vision_Pro_20240715_09255

2024-07-15 09:26:27,490 - AInewsbot - INFO - get_url(Solana Based AI Project Grass Announces Final Epoch of Closed Beta) - Saving Solana_Based_AI_Project_Grass_Announces_Final_Epoch_of_Closed_Beta_20240715_092627.html as utf-8
2024-07-15 09:26:27,494 - AInewsbot - INFO - Processing https://news.google.com/articles/CBMiY2h0dHBzOi8vd3d3LmxpdmVzY2llbmNlLmNvbS9oZWFsdGgvcmVsYXRpb25zaGlwcy9wZW9wbGUtYXJlLWZhbGxpbmctaW4tbG92ZS13aXRoLWFpLXNob3VsZC13ZS13b3JyedIBAA
2024-07-15 09:26:27,498 - AInewsbot - INFO - get_url(https://news.google.com/articles/CBMiY2h0dHBzOi8vd3d3LmxpdmVzY2llbmNlLmNvbS9oZWFsdGgvcmVsYXRpb25zaGlwcy9wZW9wbGUtYXJlLWZhbGxpbmctaW4tbG92ZS13aXRoLWFpLXNob3VsZC13ZS13b3JyedIBAA) - starting get_url https://news.google.com/articles/CBMiY2h0dHBzOi8vd3d3LmxpdmVzY2llbmNlLmNvbS9oZWFsdGgvcmVsYXRpb25zaGlwcy9wZW9wbGUtYXJlLWZhbGxpbmctaW4tbG92ZS13aXRoLWFpLXNob3VsZC13ZS13b3JyedIBAA
2024-07-15 09:26:35,736 - AInewsbot - INFO - get_url(Synergy of Generative, Analytical, Causal, and Autonomous AI) -

In [68]:
pages_df = pd.DataFrame(saved_pages)
pages_df.columns = ['id', 'url', 'title', 'path']
pages_df

,id,url,title,path
0,5,https://news.google.com/articles/CBMibmh0dHBzO...,"AI's Rapid Growth Threatens Energy Industry, E...",htmldata/AI_s_Rapid_Growth_Threatens_Energy_In...
1,8,https://news.google.com/articles/CBMiYmh0dHBzO...,Artificial intelligence outperforms clinical t...,htmldata/Artificial_intelligence_outperforms_c...
2,11,https://www.psypost.org/ai-predicts-alzheimers...,AI predicts Alzheimers disease risk with remar...,htmldata/AI_predicts_Alzheimers_disease_risk_w...
3,15,https://www.techradar.com/pro/software-workers...,Software workers across the globe are excited ...,htmldata/Software_workers_across_the_globe_are...
4,19,https://news.google.com/articles/CBMic2h0dHBzO...,AIs Dirty Secret: Why Nvidia Stock Investors S...,htmldata/AIs_Dirty_Secret__Why_Nvidia_Stock_In...
...,...,...,...,...
201,189,https://news.google.com/articles/CBMiVGh0dHBzO...,ChatGPT's Mac app exposed user conversations,htmldata/ChatGPT_s_Mac_app_exposed_user_conver...
202,193,https://www.zdnet.com/article/time-for-busines...,Time for businesses to move past generative AI...,htmldata/Time_for_businesses_to_move_past_gene...
203,197,https://news.google.com/articles/CBMiXGh0dHBzO...,Solana Based AI Project Grass Announces Final ...,htmldata/Solana_Based_AI_Project_Grass_Announc...
204,201,https://news.google.com/articles/CBMiY2h0dHBzO...,People are falling in love with AI. Should we ...,htmldata/People_are_falling_in_love_with_AI__S...


In [69]:
AIdf = pd.merge(AIdf, pages_df[["id", "path"]], on='id', how="inner")


In [70]:
AIdf

,id,url,src,title,actual_url,isAI,date,hostname,site_name,topic_str,title_topic_str,sort_order,cluster,cluster_name,path
0,0,https://www.fool.com/investing/2024/07/15/here...,Feedly AI,Here Is My TopArtificial Intelligence(AI) Stoc...,https://www.fool.com/investing/2024/07/15/here...,True,2024-07-15,www.fool.com,The Motley Fool,"Finance, Investment, Nvidia, Stock, Stocks, Te...",Here Is My TopArtificial Intelligence(AI) Stoc...,53,0,AI Stocks,htmldata/Here_Is_My_TopArtificial_Intelligence...
1,1,https://news.google.com/articles/CBMiYGh0dHBzO...,Google News,3 Artificial Intelligence Stocks Set to Mint N...,https://investorplace.com/2024/07/3-artificial...,True,2024-07-15,news.google.com,Google News,"Finance, Nvidia, Stocks, Technology",3 Artificial Intelligence Stocks Set to Mint N...,129,0,AI Stocks,htmldata/3_Artificial_Intelligence_Stocks_Set_...
2,2,https://news.google.com/articles/CBMiamh0dHBzO...,Google News,Sterling Infrastructure: The Picks And Shovels...,https://seekingalpha.com/article/4703935-sterl...,True,2024-07-15,news.google.com,Google News,"Stocks, Technology",Sterling Infrastructure: The Picks And Shovels...,203,0,AI Stocks,htmldata/Sterling_Infrastructure__The_Picks_An...
3,3,https://www.scmp.com/business/article/3270513/...,Feedly AI,Paris Olympics will use Alibabas AI-powered en...,https://www.scmp.com/business/article/3270513/...,True,2024-07-15,www.scmp.com,South China Morning Post,"Big Tech, Climate, Energy, Products, Sustainab...",Paris Olympics will use Alibabas AI-powered en...,30,1,AI Energy Sustainability,htmldata/Paris_Olympics_will_use_Alibabas_AI-p...
4,4,https://news.google.com/articles/CBMiVGh0dHBzO...,Google News,A.I.s Insatiable Appetite for Energy,https://www.nytimes.com/2024/07/11/climate/art...,True,2024-07-15,news.google.com,Google News,"AI Doom, Big Tech, Energy, Gen AI, Nvidia, Ope...",A.I.s Insatiable Appetite for Energy (Topics: ...,116,1,AI Energy Sustainability,htmldata/A_I_s_Insatiable_Appetite_for_Energy_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,201,https://news.google.com/articles/CBMiY2h0dHBzO...,Google News,People are falling in love with AI. Should we ...,https://www.livescience.com/health/relationshi...,True,2024-07-15,news.google.com,Google News,"AI Doom, Gen AI, Opinion, Technology",People are falling in love with AI. Should we ...,195,999,,htmldata/People_are_falling_in_love_with_AI__S...
202,202,https://news.google.com/articles/CBMiiQFodHRwc...,Google News,mini PC identify management artificial intelli...,https://www.militaryaerospace.com/computers/ar...,True,2024-07-15,news.google.com,Google News,"Gen AI, Nvidia, Products, Technology",mini PC identify management artificial intelli...,198,999,,htmldata/mini_PC_identify_management_artificia...
203,203,https://www.marktechpost.com/2024/07/14/opengp...,Feedly AI,OpenGPT-X Team Publishes European LLM Leaderbo...,https://www.marktechpost.com/2024/07/14/opengp...,True,2024-07-15,www.marktechpost.com,MarkTechPost,"Cohere, Language Models, Llms, Open Source, Te...",OpenGPT-X Team Publishes European LLM Leaderbo...,201,999,,htmldata/OpenGPT-X_Team_Publishes_European_LLM...
204,204,https://www.techradar.com/phones/samsung-galax...,Feedly AI,Samsung's latest Galaxy AI features tipped to ...,https://www.techradar.com/phones/samsung-galax...,True,2024-07-15,www.techradar.com,TechRadar,"Big Tech, Korea, Products, Samsung, Technology",Samsung's latest Galaxy AI features tipped to ...,202,999,,htmldata/Samsung_s_latest_Galaxy_AI_features_t...


# Summarize individual pages

In [71]:
print(SUMMARIZE_SYSTEM_PROMPT)


You are a summarization assistant.
You will summarize the main content of provided text from HTML files in 3 bullet points or less.
You will focus on the top 3 points of the text and keep the response concise."
You will ignore any content that appears to be navigation menus, footers, sidebars, or other boilerplate content.
You will output Markdown format.
You will provide the bullet points only, without any introduction such as 'here are' or any conclusion, or comment.



In [72]:
print(SUMMARIZE_USER_PROMPT)


Summarize the main points of the following text concisely in at most 3 bullet points:



In [73]:
# Here we are fetching all at once, could be 200 summaries, so we are firing off 200 REST requests at once
# This seems like a bad idea, could loop through and fire off e.g. 10 at a time, or use queues and workers (seems pointless)
# But it works and runs fast on 3.5 and if ChatGPT doesn't like it they could throttle it

log("Starting summarize")
responses = await fetch_all_summaries(AIdf)
log(f"Received {len(responses)} summaries")
print(responses[0])


2024-07-15 09:26:57,384 - AInewsbot - INFO - Starting summarize
2024-07-15 09:26:57,515 - AInewsbot - INFO - fetch_all_summaries - Page title: Here Is My Top Artificial Intelligence (AI) Stock to Buy Right Now | The Motley Fool
Social card title: Here Is My Top Artificial Intelligence (AI) Stock to Buy Right Now | The Motley Fool
Social card description: It is not too late to buy despite its massive growth.

2024-07-15 09:26:57,888 - AInewsbot - INFO - fetch_all_summaries - Page title: 3 Artificial Intelligence Stocks Set to Mint New Millionaires | InvestorPlace
Social card title: 3 Artificial Intelligence Stocks Set to Mint New Millionaires
Social card description: Not all artificial intelligence stocks are created equal. Check out three stocks with Wall Street's darlings for your best bets.

2024-07-15 09:26:57,990 - AInewsbot - INFO - fetch_all_summaries - Page title: Sterling Infrastructure: The Picks And Shovels To The Future Of AI (NASDAQ:STRL) | Seeking Alpha
Social card title: 

2024-07-15 09:26:59,739 - AInewsbot - INFO - fetch_all_summaries - Page title: NVIDIA To Rally More Than 33%? Here Are 10 Top Analyst Forecasts For Friday - NVIDIA (NASDAQ:NVDA) - Benzinga
Social card title: NVIDIA To Rally More Than 33%? Here Are 10 Top Analyst Forecasts For Friday - NVIDIA (NASDAQ:NVDA)
Social card description:  

2024-07-15 09:26:59,799 - AInewsbot - INFO - fetch_all_summaries - Page title: marketwatch.com

2024-07-15 09:26:59,801 - trafilatura.core - WARNING - discarding data: None
2024-07-15 09:26:59,842 - AInewsbot - INFO - fetch_all_summaries - Page title: Missed Nvidia? 2 Semiconductor Stocks That Could Gain From The AI Frenzy
Social card title: Missed Nvidia? 2 Semiconductor Stocks That Could Gain From The AI Frenzy
Social card description: These two key players in the semiconductor industry could gain big in the long run.

2024-07-15 09:26:59,883 - AInewsbot - INFO - fetch_all_summaries - Page title: Nvidia To Rally Over 40%? Here Are 10 Top Analyst Forecasts

2024-07-15 09:27:00,725 - AInewsbot - INFO - fetch_all_summaries - Page title: You can’t have an AI strategy without a data strategy | Utility Dive
Social card title: You can’t have an AI strategy without a data strategy
Social card description: For AI to truly maximize outcomes for both customers and the business, a number of key tenets should be central to a utility’s AI strategy.

2024-07-15 09:27:00,935 - AInewsbot - INFO - fetch_all_summaries - Page title: Are Business Leaders Ready For AI? Here’s How To Tell
Social card title: Are Business Leaders Ready For AI? Here’s How To Tell
Social card description: AI Hype vs. Reality: Don't be misled by fear or empty promises. AI is a powerful tool, but businesses need a vision and a team centered plan to unlock its true potential.

2024-07-15 09:27:00,976 - AInewsbot - INFO - fetch_all_summaries - Page title: Efficient Deployment of Large-Scale Transformer Models: Strategies for Scalable and Low-Latency Inference - MarkTechPost
Social car

2024-07-15 09:27:01,893 - AInewsbot - INFO - fetch_all_summaries - Page title: What a data- and AI-driven society will look like - Telefónica
Social card title: What a data- and AI-driven society will look like
Social card description: Find out more about what a data and AI-driven society will look like. Enter now and don't miss it on our blog.

2024-07-15 09:27:02,197 - AInewsbot - INFO - fetch_all_summaries - Page title: What phrase is a dead giveaway that a text was probably written by ChatGPT? : ChatGPT
Social card title: What phrase is a dead giveaway that a text was probably written by ChatGPT?
Social card description: Posted in r/ChatGPT by u/noonewilltakemealive • 813 points and 787 comments

2024-07-15 09:27:02,484 - AInewsbot - INFO - fetch_all_summaries - Page title: Did Chatgpt just create a word for me? : r/ChatGPT

2024-07-15 09:27:02,674 - AInewsbot - INFO - fetch_all_summaries - Page title: One Year After the Actors’ Strike, AI Remains a Persistent Threat
Social card ti

2024-07-15 09:27:03,723 - AInewsbot - INFO - fetch_all_summaries - Page title: OpenAI on the steps for AI to reach human intelligence
Social card title: OpenAI says there are 5 'levels' for AI to reach human intelligence — it's already almost at level 2
Social card description: The ChatGPT maker believes it's on the first level, which is conversational AI

2024-07-15 09:27:03,769 - AInewsbot - INFO - fetch_all_summaries - Page title: OpenAI is building a new AI model under code name ‘Strawberry’ — Report
Social card title: OpenAI is building a new AI model under code name ‘Strawberry’ — Report
Social card description: ChatGPT maker OpenAI has reportedly been working on a project named “Strawberry” that aims to improve the reasoning capabilities of OpenAI’s earlier models.

2024-07-15 09:27:03,807 - AInewsbot - INFO - fetch_all_summaries - Page title: OpenAI is reportedly working on more advanced AI models capable of reasoning and ‘deep research’
Social card title: OpenAI is reportedly 

...

2024-07-15 09:27:07,929 - AInewsbot - INFO - fetch_all_summaries - Page title: Fully nonlinear neuromorphic computing with linear wave scattering | Nature Physics
Social card title: Fully nonlinear neuromorphic computing with linear wave scattering - Nature Physics
Social card description: As the energy consumption of neural networks continues to grow, different approaches to deep learning are needed. A neuromorphic method offering nonlinear computation based on linear wave scattering can be implemented using integrated photonics.

2024-07-15 09:27:08,027 - AInewsbot - INFO - fetch_all_summaries - Page title: Gen AI and beyond: Where else to focus now | McKinsey
Social card title: Gen AI and beyond: Where else to focus now
Social card description: Yes, gen AI can be dazzling. But to deliver value, leaders will have to look beyond center stage.

2024-07-15 09:27:08,075 - AInewsbot - INFO - fetch_all_summaries - Page title: European AI rules could hamstring Chinese tech companies with co

2024-07-15 09:27:09,312 - AInewsbot - INFO - fetch_all_summaries - Page title: AMD's new Zen 5 flagship gets benchmarked — Ryzen 9 9950X Engineering Sample isn't as impressive in Blender at maximum power settings | Tom's Hardware
Social card title: AMD's new Zen 5 flagship gets benchmarked — Ryzen 9 9950X Engineering Sample isn't as impressive in Blender at maximum power settings
Social card description: Pushing the limits on the Ryzen 9 9950X doesn't massively improve throughput, unfortunately.

2024-07-15 09:27:09,360 - AInewsbot - INFO - fetch_all_summaries - Page title: Apple was pressured into giving people AI: Is iOS 18 a disaster waiting to happen? - PhoneArena
Social card title: Apple was pressured into giving people AI: Is iOS 18 a disaster waiting to happen?
Social card description: iPhone 15 Pro got the “AI” everyone wanted, but did Apple iPhone-ed it in with iOS 18?

2024-07-15 09:27:09,414 - AInewsbot - INFO - fetch_all_summaries - Page title: A.I. Helped to Spot a Copper 

2024-07-15 09:27:10,404 - AInewsbot - INFO - fetch_all_summaries - Page title: Pinterest Outlines AI Background Generation Process for Product Shots | Social Media Today
Social card title: Pinterest Outlines AI Background Generation Process for Product Shots
Social card description: Pinterest is looking to help brands create better visuals in the app.

2024-07-15 09:27:10,425 - AInewsbot - INFO - fetch_all_summaries - Page title: wsj.com

2024-07-15 09:27:10,427 - trafilatura.core - WARNING - discarding data: None
2024-07-15 09:27:10,436 - AInewsbot - INFO - fetch_all_summaries - Page title: Discover Looks to AI to Unlock Customer Experience
Social card title: Discover Looks to AI to Unlock Customer Experience
Social card description: The rapid evolution of the payment space makes AI a perfect fit, Discover Global Network’s Kate Lybarger tells PYMNTS.

2024-07-15 09:27:10,483 - AInewsbot - INFO - fetch_all_summaries - Page title: Does AI Really Need Your Personal Data?
Social card titl

2024-07-15 09:27:11,517 - AInewsbot - INFO - fetch_all_summaries - Page title: Solana Based AI Project Grass Announces Final Epoch of Closed Beta – News Bytes Bitcoin News
Social card title: Solana Based AI Project Grass Announces Final Epoch of Closed Beta – News Bytes Bitcoin News
Social card description: Grass, an AI-based project that uses the Solana network, has announced the end of its closed beta version with its final epoch.

2024-07-15 09:27:11,552 - AInewsbot - INFO - fetch_all_summaries - Page title: Synergy of Generative, Analytical, Causal, and Autonomous AI - DataScienceCentral.com
Social card title: Synergy of Generative, Analytical, Causal, and Autonomous AI - DataScienceCentral.com
Social card description: The current fascination with Generative AI (GenAI) – especially as manifested by OpenAI’s ChatGPT – has raised public awareness of Artificial Intelligence (AI) and its ability to create new sources of customer, product, service, and operational value. Leveraging GenA

(0, {'id': 'chatcmpl-9lG1Qa1DdOnxJ87bmFQ4Cyg2YONgX', 'object': 'chat.completion', 'created': 1721050032, 'model': 'gpt-3.5-turbo-0125', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "- Nvidia is a top artificial intelligence (AI) stock attracting investors due to leveraging gaming and GPUs, but its high valuation might lead to a significant drop, prompting investors to look at a long-time partner instead.\n- Super Micro Computer (SMCI) is a potential investment with significant growth, having experienced a 4,500% increase in stock value in the last five years by providing Nvidia-powered AI equipment.\n- Despite Supermicro's massive gains, its strong financial performance, including revenue growth of 95% year-over-year and raised guidance for fiscal 2024, indicates the stock's potential for further growth and offers a relatively inexpensive investment opportunity compared to Nvidia.\n"}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1189,

In [74]:
# bring summaries into dict
response_dict = {}
for i, response in responses:
    try:
        response_str = response["choices"][0]["message"]["content"]
        response_dict[i] = response_str
    except Exception as exc:
        print(exc)
        
len(response_dict)

206

In [75]:
# add the name of the site to the markdown
# add the name of the cluster topic title to the topics when summarizing

# keep the 
# categorize 
# use the full name of the categories

In [102]:
AIdf['hostname']=AIdf['actual_url'].apply(lambda url: urlparse(url).netloc)
AIdf['site_name'] = AIdf['hostname'].apply(lambda hostname: sites_dict.get(hostname, ""))


In [103]:
AIdf


,id,url,src,title,actual_url,isAI,date,hostname,site_name,topic_str,title_topic_str,sort_order,cluster,cluster_name,path
0,0,https://www.fool.com/investing/2024/07/15/here...,Feedly AI,Here Is My TopArtificial Intelligence(AI) Stoc...,https://www.fool.com/investing/2024/07/15/here...,True,2024-07-15,www.fool.com,The Motley Fool,"Finance, Investment, Nvidia, Stock, Stocks, Te...",Here Is My TopArtificial Intelligence(AI) Stoc...,53,0,AI Stocks,htmldata/Here_Is_My_TopArtificial_Intelligence...
1,1,https://news.google.com/articles/CBMiYGh0dHBzO...,Google News,3 Artificial Intelligence Stocks Set to Mint N...,https://investorplace.com/2024/07/3-artificial...,True,2024-07-15,investorplace.com,InvestorPlace,"Finance, Nvidia, Stocks, Technology",3 Artificial Intelligence Stocks Set to Mint N...,129,0,AI Stocks,htmldata/3_Artificial_Intelligence_Stocks_Set_...
2,2,https://news.google.com/articles/CBMiamh0dHBzO...,Google News,Sterling Infrastructure: The Picks And Shovels...,https://seekingalpha.com/article/4703935-sterl...,True,2024-07-15,seekingalpha.com,Seeking Alpha,"Stocks, Technology",Sterling Infrastructure: The Picks And Shovels...,203,0,AI Stocks,htmldata/Sterling_Infrastructure__The_Picks_An...
3,3,https://www.scmp.com/business/article/3270513/...,Feedly AI,Paris Olympics will use Alibabas AI-powered en...,https://www.scmp.com/business/article/3270513/...,True,2024-07-15,www.scmp.com,South China Morning Post,"Big Tech, Climate, Energy, Products, Sustainab...",Paris Olympics will use Alibabas AI-powered en...,30,1,AI Energy Sustainability,htmldata/Paris_Olympics_will_use_Alibabas_AI-p...
4,4,https://news.google.com/articles/CBMiVGh0dHBzO...,Google News,A.I.s Insatiable Appetite for Energy,https://www.nytimes.com/2024/07/11/climate/art...,True,2024-07-15,www.nytimes.com,The New York Times,"AI Doom, Big Tech, Energy, Gen AI, Nvidia, Ope...",A.I.s Insatiable Appetite for Energy (Topics: ...,116,1,AI Energy Sustainability,htmldata/A_I_s_Insatiable_Appetite_for_Energy_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,201,https://news.google.com/articles/CBMiY2h0dHBzO...,Google News,People are falling in love with AI. Should we ...,https://www.livescience.com/health/relationshi...,True,2024-07-15,www.livescience.com,Live Science,"AI Doom, Gen AI, Opinion, Technology",People are falling in love with AI. Should we ...,195,999,,htmldata/People_are_falling_in_love_with_AI__S...
202,202,https://news.google.com/articles/CBMiiQFodHRwc...,Google News,mini PC identify management artificial intelli...,https://www.militaryaerospace.com/computers/ar...,True,2024-07-15,www.militaryaerospace.com,,"Gen AI, Nvidia, Products, Technology",mini PC identify management artificial intelli...,198,999,,htmldata/mini_PC_identify_management_artificia...
203,203,https://www.marktechpost.com/2024/07/14/opengp...,Feedly AI,OpenGPT-X Team Publishes European LLM Leaderbo...,https://www.marktechpost.com/2024/07/14/opengp...,True,2024-07-15,www.marktechpost.com,MarkTechPost,"Cohere, Language Models, Llms, Open Source, Te...",OpenGPT-X Team Publishes European LLM Leaderbo...,201,999,,htmldata/OpenGPT-X_Team_Publishes_European_LLM...
204,204,https://www.techradar.com/phones/samsung-galax...,Feedly AI,Samsung's latest Galaxy AI features tipped to ...,https://www.techradar.com/phones/samsung-galax...,True,2024-07-15,www.techradar.com,TechRadar,"Big Tech, Korea, Products, Samsung, Technology",Samsung's latest Galaxy AI features tipped to ...,202,999,,htmldata/Samsung_s_latest_Galaxy_AI_features_t...


In [104]:
# make text for email and also collect data for vector store
markdown_str = ''
vectorstore_list = []
metadata_list=[]
for i, row in enumerate(AIdf.itertuples()):
    topics = []
    if row.cluster_name:
        topics.append(row.cluster_name)
    if row.topic_str:
        topics.append(row.topic_str)
    topic_str = ", ".join(topics)

    mdstr = f"[{i+1}. {row.title} - {row.site_name}]({row.actual_url})  \n\n {topic_str}  \n\n{response_dict[row.id]} \n\n"
    # simpler version for vector store
    vectorstore_list.append(f"[{row.title} - {row.site_name}]({row.actual_url})\n\nTopics: {row.topic_str} \n\n{response_dict[row.id]}\n\n")
    metadata_list.append({'id': row.id, 'title': row.title, 'url': row.actual_url, 'site': row.site_name})
    display(Markdown(mdstr))
    markdown_str += mdstr
    

[1. Here Is My TopArtificial Intelligence(AI) Stock to Buy Right Now - The Motley Fool](https://www.fool.com/investing/2024/07/15/heres-my-top-ai-stock-buy-now/)  

 AI Stocks, Finance, Investment, Nvidia, Stock, Stocks, Technology  

- Nvidia is a top artificial intelligence (AI) stock attracting investors due to leveraging gaming and GPUs, but its high valuation might lead to a significant drop, prompting investors to look at a long-time partner instead.
- Super Micro Computer (SMCI) is a potential investment with significant growth, having experienced a 4,500% increase in stock value in the last five years by providing Nvidia-powered AI equipment.
- Despite Supermicro's massive gains, its strong financial performance, including revenue growth of 95% year-over-year and raised guidance for fiscal 2024, indicates the stock's potential for further growth and offers a relatively inexpensive investment opportunity compared to Nvidia.
 



[2. 3 Artificial Intelligence Stocks Set to Mint New Millionaires - InvestorPlace](https://investorplace.com/2024/07/3-artificial-intelligence-stocks-set-to-mint-new-millionaires/)  

 AI Stocks, Finance, Nvidia, Stocks, Technology  

- Analysts' strong buy ratings are a good starting point for identifying quality AI stocks with potential upsides exceeding 40%.
- Super Micro Computer (SMCI) is well-positioned for growth in the AI sector, with a 77% potential upside and impressive revenue growth.
- QuickLogic Corporation (QUIK) and Lattice Semiconductor Corporation (LSCC) also show promise with significant revenue growth, positive sentiments, and strong buy ratings. 



[3. Sterling Infrastructure: The Picks And Shovels To The Future Of AI (NASDAQ:STRL) - Seeking Alpha](https://seekingalpha.com/article/4703935-sterling-infrastructure-the-picks-and-shovels-to-the-future-of-ai)  

 AI Stocks, Stocks, Technology  

- Sterling Infrastructure is benefitting from long-term AI tailwinds and has experienced double-digit revenue growth, with a focus on e-infrastructure, transportation, and building solutions.
- The company has shown impressive financial performance with revenue and EBITDA growth, a strong capital allocation framework including investments in organic growth and M&A, and a growing backlog of projects.
- Sterling Infrastructure's growth is being primarily driven by the e-infrastructure segment, amid a massive growth opportunity in the smart infrastructure market driven by AI advancements, with a strong outlook supported by a substantial backlog and guidance for revenue and EBITDA growth in FY'24. 



[4. Paris Olympics will use Alibabas AI-powered energy-saving tool to cut venue emissions - South China Morning Post](https://www.scmp.com/business/article/3270513/paris-olympics-will-use-alibabas-ai-powered-energy-saving-tool-cut-venue-emissions)  

 AI Energy Sustainability, Big Tech, Climate, Energy, Products, Sustainability, Technology  

- Paris Olympics will use Alibaba’s AI-powered energy-saving tool to optimize energy demand and reduce venue emissions
- The AI tool will analyze power usage at 35 locations during the games to minimize waste and help achieve carbon neutrality
- Organizers aim to cut event-related carbon emissions by using renewable energy sources and sustainable practices at the Games 



[5. A.I.s Insatiable Appetite for Energy - The New York Times](https://www.nytimes.com/2024/07/11/climate/artificial-intelligence-energy-usage.html)  

 AI Energy Sustainability, AI Doom, Big Tech, Energy, Gen AI, Nvidia, OpenAI, Science, Sustainability, Technology  

- A.I.'s increasing electricity demands from data centers are straining the energy grid, contributing to higher emissions and hindering the energy transition.
- Tech giants such as Microsoft and Google are facing challenges in reducing the carbon footprint of A.I., with projections showing a significant increase in electricity consumption by data centers.
- The growth of A.I. is driving up energy demand and emissions, prompting companies to explore renewable energy sources while facing hurdles in balancing energy needs with environmental sustainability. 



...

[202. People are falling in love with AI. Should we worry? - Live Science](https://www.livescience.com/health/relationships/people-are-falling-in-love-with-ai-should-we-worry)  

 AI Doom, Gen AI, Opinion, Technology  

- AI romantic companions can save some people from loneliness but also have a dark side, affecting real-world relationship expectations and social skills.
- Users can form emotional bonds with AI, enjoying emotional connections regardless of whether they believe they are interacting with a human or a chatbot.
- AI companions come with privacy concerns, such as data sharing and tracking, but offer potential benefits in enhancing romantic well-being and providing support to marginalized communities. 



[203. mini PC identify management artificial intelligence (AI) - ](https://www.militaryaerospace.com/computers/article/55093986/aaeon-electronics-inc-mini-pc-identify-management-artificial-intelligence-ai)  

 Gen AI, Nvidia, Products, Technology  

- AAEON Technology introduces the UP Squared Pro 710H Edge mini PC for identity management, anti-microbial resistance, and health care imaging solutions, featuring the Hailo-8 edge AI processor.
- The mini PC offers 26 tera-operations per second of inferencing performance, onboard Hailo AI software suite, and up to 16GB LPDDR5 memory along with 128GB eMMC data storage for speed, capacity, and security.
- The compact mini PC provides AI-ready features, multiple I/O options, 40-pin GPIO for various peripherals, and support for Windows 10 IoT Enterprise and Ubuntu 22.04 LTS operating systems. 



[204. OpenGPT-X Team Publishes European LLM Leaderboard: Promoting the Way for Advanced Multilingual Language Model Development and Evaluation - MarkTechPost](https://www.marktechpost.com/2024/07/14/opengpt-x-team-publishes-european-llm-leaderboard-promoting-the-way-for-advanced-multilingual-language-model-development-and-evaluation/)  

 Cohere, Language Models, Llms, Open Source, Technology  

- The OpenGPT-X team released the European LLM Leaderboard to advance multilingual language models and reduce digital language barriers in AI applications across Europe.
- The project focuses on developing and evaluating multilingual LLMs to broaden language accessibility, with benchmarks translated into 21 European languages for comprehensive evaluations.
- TU Dresden's involvement includes leveraging its competence centers to develop scalable evaluation pipelines and improve model performance of large language models on supercomputing clusters, with automated evaluations through the Hugging Face Hub platform. 



[205. Samsung's latest Galaxy AI features tipped to be heading to these older phones - TechRadar](https://www.techradar.com/phones/samsung-galaxy-phones/samsungs-latest-galaxy-ai-features-tipped-to-be-heading-to-these-older-phones)  

 Big Tech, Korea, Products, Samsung, Technology  

- Samsung is expected to bring the latest One UI 6.1.1 software with Galaxy AI features to older models including the Galaxy S22, S23, S24, S23 FE, and tablets like the Galaxy Tab S8 and S9.
- The One UI 6.1.1 update introduces new Galaxy AI features such as Sketch to Image, Portrait Studio, Live Translation, and Composer, enhancing AI capabilities on supported devices.
- The update includes a new Galaxy AI menu in Settings and device-specific improvements like new widgets for the Samsung Galaxy Z Flip 6 cover screen. 



[206. Metas approach to machine learning prediction robustness - Facebook Engineering](https://engineering.fb.com/2024/07/10/data-infrastructure/machine-learning-ml-prediction-robustness-meta/)  

 Data, Machine Learning, Meta, Science, Technology, Testing  

- Meta's advertising business uses large-scale machine learning models for ad recommendations, requiring robust prediction systems to ensure service reliability and delivery.
- Challenges in ML prediction stability stem from model stochasticity, continuous updates, and difficulty in detecting and tracing cumulative quality shifts over time.
- Meta has developed prediction robustness frameworks for models, features, training data, calibration, and interpretability to ensure stability and reliability, utilizing techniques like Snapshot Validator and Hawkeye for model robustness and ML interpretability. 



In [105]:
display(Markdown( vectorstore_list[16]))

[5 ways the new wave of chatbots will transform our work lives - Big Think](https://bigthink.com/business/5-ways-the-new-wave-of-chatbots-will-transform-our-work-lives/)

Topics: Chatbots, Jobs, Careers, Labor Market, Technology 

- The new generation of chatbots will transform work lives with personalized and empathetic employee assistance, refined team collaboration, and more AI integration by tech companies.
- Chatbots will streamline administrative tasks, automate workflows, organize information, and reduce human errors in documentation.
- Advanced chatbots will provide personalized employee assistance with improved memory, understanding of individual styles, tone detection, and emotionally intelligent interactions.



In [106]:
print(metadata_list[16])


{'id': 16, 'title': '5 ways the new wave of chatbots will transform our work lives', 'url': 'https://bigthink.com/business/5-ways-the-new-wave-of-chatbots-will-transform-our-work-lives/', 'site': 'Big Think'}


In [107]:
# Create Document objects with the paragraphs and corresponding metadata
docs = [Document(page_content=paragraph, metadata=meta) 
        for paragraph, meta in zip(vectorstore_list, metadata_list)]
len(docs)

206

In [108]:
print(docs[1])


page_content="[3 Artificial Intelligence Stocks Set to Mint New Millionaires - InvestorPlace](https://investorplace.com/2024/07/3-artificial-intelligence-stocks-set-to-mint-new-millionaires/)\n\nTopics: Finance, Nvidia, Stocks, Technology \n\n- Analysts' strong buy ratings are a good starting point for identifying quality AI stocks with potential upsides exceeding 40%.\n- Super Micro Computer (SMCI) is well-positioned for growth in the AI sector, with a 77% potential upside and impressive revenue growth.\n- QuickLogic Corporation (QUIK) and Lattice Semiconductor Corporation (LSCC) also show promise with significant revenue growth, positive sentiments, and strong buy ratings.\n\n" metadata={'id': 1, 'title': '3 Artificial Intelligence Stocks Set to Mint New Millionaires', 'url': 'https://investorplace.com/2024/07/3-artificial-intelligence-stocks-set-to-mint-new-millionaires/', 'site': 'InvestorPlace'}


In [109]:
del vectorstore
vectorstore


NameError: name 'vectorstore' is not defined

In [110]:
# persist_directory = "/Users/drucev/projects/AInewsbot/chroma_db_openai"
try:
    del vectorstore
except Exception as e:
    log(f"{e}")

try:
    shutil.rmtree(persist_directory)
    log(f"Directory '{persist_directory}' and all its contents have been removed successfully.")
except Exception as e:
    log(f"Remove directory error: {e}")
        
embeddings_openAI = OpenAIEmbeddings(model='text-embedding-3-small')
vectorstore = Chroma.from_documents(docs, embeddings_openAI)


2024-07-15 09:49:13,933 - AInewsbot - INFO - name 'vectorstore' is not defined
2024-07-15 09:49:13,935 - AInewsbot - INFO - Remove directory error: name 'persist_directory' is not defined
2024-07-15 09:49:13,988 - chromadb.api.segment - INFO - Collection langchain is not created.
2024-07-15 09:49:15,007 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [111]:
docs[0]

Document(page_content="[Here Is My TopArtificial Intelligence(AI) Stock to Buy Right Now - The Motley Fool](https://www.fool.com/investing/2024/07/15/heres-my-top-ai-stock-buy-now/)\n\nTopics: Finance, Investment, Nvidia, Stock, Stocks, Technology \n\n- Nvidia is a top artificial intelligence (AI) stock attracting investors due to leveraging gaming and GPUs, but its high valuation might lead to a significant drop, prompting investors to look at a long-time partner instead.\n- Super Micro Computer (SMCI) is a potential investment with significant growth, having experienced a 4,500% increase in stock value in the last five years by providing Nvidia-powered AI equipment.\n- Despite Supermicro's massive gains, its strong financial performance, including revenue growth of 95% year-over-year and raised guidance for fiscal 2024, indicates the stock's potential for further growth and offers a relatively inexpensive investment opportunity compared to Nvidia.\n\n\n", metadata={'id': 0, 'title': 

In [112]:
# Perform a similarity search
query = "What is the latest with openai?"
results = vectorstore.similarity_search_with_score(query, 
                                        k=20,
                                       )  # k is the number of results to return
# Print the results
urldict = {}
for doc, score in results:
    if urldict.get(doc.metadata['url']):
        continue
    urldict[doc.metadata['url']] = 1
    if score < 1.25:
        print(f"Score:   {score}")
        print(f"Content: {doc.page_content}\n")
        print(f"Metadata: {doc.metadata}\n")
        print("---")

2024-07-15 09:49:27,351 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Score:   0.9060448408126831
Content: [OpenAI is reportedly working on more advanced AI models capable of reasoning and deep research - Google News](https://www.engadget.com/openai-is-reportedly-working-on-more-advanced-ai-models-capable-of-reasoning-and-deep-research-202419228.html)

Topics: Big Tech, Gen AI, Language Models, OpenAI, Rag, Science, Technology 

- OpenAI is reportedly working on more advanced AI models for reasoning and 'deep research' under a secret project code-named 'Strawberry'.
- The project aims to enhance AI models to autonomously gather information from the internet to plan for complex tasks.
- The development status and similarity to previously demonstrated AI with 'human-like reasoning' skills remain undisclosed.



Metadata: {'id': 79, 'site': 'Google News', 'title': 'OpenAI is reportedly working on more advanced AI models capable of reasoning and deep research', 'url': 'https://www.engadget.com/openai-is-reportedly-working-on-more-advanced-ai-models-capable-o

In [ ]:
# # or use local embeddings with sentence_transformers
# # Initialize your embedding model
# embeddings_hf = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# # Create the vector store with a persist_directory
# persist_directory = "/Users/drucev/projects/AInewsbot/chroma_db_huggingface"
# vectorstore_hf = Chroma.from_documents(
#     documents=docs,
#     embedding=embeddings_hf,
#     persist_directory=persist_directory
# )

# # Perform a similarity search
# query = "What is the latest with OpenAI?"
# results = vectorstore_hf.similarity_search(query, k=10)  # k is the number of results to return

# # Print the results
# for doc in results:
#     print(f"Content: {doc.page_content}\n")
#     print(f"Metadata: {doc.metadata}\n")
#     print("---")
    


In [89]:
# Convert Markdown to HTML
html_str = markdown.markdown(markdown_str, extensions=['extra'])
# display(HTML(html_str))


In [90]:
# save bullets
with open('bullets.md', 'w') as f:
    f.write(markdown_str)


In [91]:
log("Sending bullet points email")
subject = f'AI news bullets {datetime.now().strftime("%H:%M:%S")}'
send_gmail(subject, html_str)


2024-07-15 09:36:41,086 - AInewsbot - INFO - Sending bullet points email


# Ask ChatGPT for top categories

In [92]:
print(TOP_CATEGORIES_PROMPT)

You will act as a research assistant identifying the top stories and topics
of today's news. I will provide a list of today's news stories about AI and summary bullet points in markdown
format. You are tasked with identifying the top 10-20 stories and topics of today's news. For each top story
or topic, you will create a short title and respond with a list of titles formatted as a JSON object.


Example Input Bullet Points:

[2. Sentient closes $85M seed round for open-source AI](https://cointelegraph.com/news/sentient-85-million-round-open-source-ai)

- Sentient secured $85 million in a seed funding round led by Peter Thiel's Founders Fund, Pantera Capital, and Framework Ventures for their open-source AI platform.
- The startup aims to incentivize AI developers with its blockchain protocol and incentive mechanism, allowing for the evolution of open artificial general intelligence.
- The tech industry is witnessing a rise in decentralized AI startups combining blockchain

Categories of

In [93]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": TOP_CATEGORIES_PROMPT + markdown_str
              }],
    n=1,
    response_format={"type": "json_object"},
    temperature=0.5
)


2024-07-15 09:36:54,933 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [94]:
suggested_categories = list(json.loads(response.choices[0].message.content).values())[0]
suggested_categories

['Nvidia AI stock investment',
 "Super Micro Computer's AI growth",
 'Sterling Infrastructure AI benefits',
 'Paris Olympics AI energy tool',
 'AI energy sustainability concerns',
 "AI predicting Alzheimer's progression",
 'AI in workplace ethics',
 'Google Gemini privacy concerns',
 'Apple AI stock performance',
 'Nvidia AI chip rally',
 'OpenAI advanced AI development',
 'AI in entertainment industry',
 'AI in healthcare',
 'Skild AI funding for robotics',
 'LG AI battery design innovation',
 'ChatGPT Mac app',
 'AI chatbots and language models',
 'AI in business technology',
 'AI financial results',
 'AI ethics investigation']

In [95]:
cluster_topic_list

['AI Stocks',
 'AI Energy Sustainability',
 "AI Predicting Alzheimer's Progression",
 'AI in the Workplace',
 'Nvidia and AI Chips Investment',
 'AI Privacy Concerns',
 "Apple's AI Stock Performance",
 'AI in Business Technology',
 'Language Models',
 'ChatGPT Desktop App on Mac',
 'AI Chatbots and Language Models',
 'AI in Entertainment Industry',
 'AI in Healthcare',
 'Artificial Intelligence Funding for Robotics',
 'AI-Powered Battery Design Innovation',
 'Advanced AI Development',
 'OpenAI Ethics Investigation']

In [96]:
my_cats = [
'AI Stocks',
 'AI Energy Sustainability',
 "AI Predicting Alzheimer's Progression",
 'AI in the Workplace',
 'Nvidia stock',
 'AI Privacy',
 "Apple",
 'AI Chatbots and Language Models',
 'AI in Entertainment Industry',
 'AI in Healthcare',
 'Robotics',
 'Battery Innovation',
 'Advanced AI Development',
 'OpenAI',

 "SuperMicro",
 'Paris Olympics',
 'Google Gemini privacy concerns',
 'ChatGPT Mac app',
 'AI ethics investigation'    
]

md_str = ""
doc_list = []
similarity_cutoff = 1.25
for cat in my_cats:
    docstr = f"# {cat} \n\n"
    # Perform a similarity search
    results = vectorstore.similarity_search_with_score(cat, 
                                                       k=10,
                                                      )
    if results:
        # Print the results
        urldict = {}
        for doc, score in results:
            if urldict.get(doc.metadata['url']):
                continue
            urldict[doc.metadata['url']] = 1    
            if score > similarity_cutoff:
                break
            docstr += f"[{doc.metadata['site']} - {doc.metadata['site']}]({doc.metadata['url']})  \n"
            docstr += f"{doc.page_content}\n"
        doc_list.append(docstr)
        md_str += docstr
        
        
display(Markdown(md_str))
            


2024-07-15 09:41:41,490 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-15 09:41:41,706 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-15 09:41:42,058 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-15 09:41:42,571 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-15 09:41:42,767 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-15 09:41:42,955 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-15 09:41:43,130 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-15 09:41:43,346 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-07-15 09:41:43,533 - httpx - INFO - HTTP Request: POST https://api.openai.c

# AI Stocks 

[Google News - Google News](https://markets.businessinsider.com/news/stocks/ai-bubble-pop-signal-stock-market-investing-outlook-goldman-sachs-2024-7)  
[When will AI's stock-market dominance fade? Goldman says watch this key signal. - Google News](https://markets.businessinsider.com/news/stocks/ai-bubble-pop-signal-stock-market-investing-outlook-goldman-sachs-2024-7)

Topics: Bubble, Economics, Finance, Fintech, Stocks, Technology 

- Amazon, Meta, Microsoft, and Alphabet have collectively invested billions in AI over the past four quarters.
- Goldman Sachs warns that the ongoing outperformance of AI stocks might face challenges in the future.
- There is a focus on whether companies can translate their AI spending into actual revenue and earnings to maintain stock valuations.


[Google News - Google News](https://investorplace.com/2024/07/3-artificial-intelligence-stocks-set-to-mint-new-millionaires/)  
[3 Artificial Intelligence Stocks Set to Mint New Millionaires - Google News](https://investorplace.com/2024/07/3-artificial-intelligence-stocks-set-to-mint-new-millionaires/)

Topics: Finance, Nvidia, Stocks, Technology 

- Analysts' strong buy ratings are a good starting point for identifying quality AI stocks with potential upsides exceeding 40%.
- Super Micro Computer (SMCI) is well-positioned for growth in the AI sector, with a 77% potential upside and impressive revenue growth.
- QuickLogic Corporation (QUIK) and Lattice Semiconductor Corporation (LSCC) also show promise with significant revenue growth, positive sentiments, and strong buy ratings.


[Google News - Google News](https://www.investing.com/news/stock-market-news/5-big-analyst-ai-moves-tesla-and-smci-stocks-downgraded-3518257)  
[5 big analyst AI moves: Tesla and SMCI stocks downgraded - Google News](https://www.investing.com/news/stock-market-news/5-big-analyst-ai-moves-tesla-and-smci-stocks-downgraded-3518257)

Topics: Elon Musk, Finance, Opinion, Review, Stocks, Technology, Tesla 

- UBS downgrades Tesla stock to Sell due to high valuation, AI costs, and declining expectations for core auto business.
- Bank of America raises Apple's price target based on confidence in multi-year iPhone upgrade cycle and GenAI features.
- Nomura downgrades Super Micro Computer stock to Neutral citing limited share price upside and uncertainties related to market expectations and product transitions.


[Investing.com - Investing.com](https://au.investing.com/news/stock-market-news/apple-gains-after-morgan-stanley-calls-stock-top-pick-for-ai-efforts-3340208)  
[Apple gains after Morgan Stanley calls stock 'top pick' for AI efforts - Investing.com](https://au.investing.com/news/stock-market-news/apple-gains-after-morgan-stanley-calls-stock-top-pick-for-ai-efforts-3340208)

Topics: Apple, Big Tech, Finance, Gen AI, Products, Stocks, Technology 

- Apple's shares rose 2% after Morgan Stanley designated them as a "top pick" due to AI efforts, expecting a boost in device sales.
- Apple unveiled Apple Intelligence to compete with Google and Microsoft-backed OpenAI, with potential to sell nearly 500 million iPhones in the next two years.
- Industry analysts predict Apple and Samsung to lead global smartphone market recovery, despite Apple's slight market share drop in the previous quarter.


[Yahoo Finance - Yahoo Finance](https://sg.finance.yahoo.com/news/apple-inc-nasdaq-aapl-aswath-214249845.html)  
[Is Apple Inc (NASDAQ:AAPL) Aswath Damodarans Best AI Stock Pick? - Yahoo Finance](https://sg.finance.yahoo.com/news/apple-inc-nasdaq-aapl-aswath-214249845.html)

Topics: Opinion, Stocks 

- Aswath Damodaran considers Apple Inc (AAPL) part of the "Magnificent Seven" group of stocks that have added significant market cap, emphasizing their value in the market.
- Despite concerns about overvaluation in the tech sector, Damodaran views Apple as a "money machine" with a competitive edge in AI, supported by analysts like Dan Ives and Ming-Chi Kuo.
- Though facing short-term challenges, Apple's growth potential is highlighted by its ecosystem of devices and paying subscribers, positioning it well for cash flow growth and intrinsic value appreciation.



[The Motley Fool - The Motley Fool](https://www.fool.com/investing/2024/07/15/heres-my-top-ai-stock-buy-now/)  
[Here Is My TopArtificial Intelligence(AI) Stock to Buy Right Now - The Motley Fool](https://www.fool.com/investing/2024/07/15/heres-my-top-ai-stock-buy-now/)

Topics: Finance, Investment, Nvidia, Stock, Stocks, Technology 

- Nvidia is a top artificial intelligence (AI) stock attracting investors due to leveraging gaming and GPUs, but its high valuation might lead to a significant drop, prompting investors to look at a long-time partner instead.
- Super Micro Computer (SMCI) is a potential investment with significant growth, having experienced a 4,500% increase in stock value in the last five years by providing Nvidia-powered AI equipment.
- Despite Supermicro's massive gains, its strong financial performance, including revenue growth of 95% year-over-year and raised guidance for fiscal 2024, indicates the stock's potential for further growth and offers a relatively inexpensive investment opportunity compared to Nvidia.



[Google News - Google News](https://seekingalpha.com/article/4703737-amd-building-an-ai-ecosystem)  
[AMD: Building An AI Ecosystem (NASDAQ:AMD) - Google News](https://seekingalpha.com/article/4703737-amd-building-an-ai-ecosystem)

Topics: Amd, Gen AI, Semiconductor Chips, Stocks, Technology 

- AMD stock slightly outperformed the market but lags behind Nvidia due to market's belief in Nvidia's AI dominance.
- AMD is focusing on building a comprehensive AI ecosystem with diverse hardware offerings, acquisitions like Silo AI, and rapid technology development.
- Despite earnings expectations revisions, AMD's growth potential and undervaluation suggest significant upside, especially in the AI market.


[Google News - Google News](https://www.barchart.com/story/news/27341677/missed-nvidia-2-semiconductor-stocks-that-could-gain-from-the-ai-frenzy)  
[Missed Nvidia? 2 Semiconductor Stocks That Could Gain From The AI Frenzy - Google News](https://www.barchart.com/story/news/27341677/missed-nvidia-2-semiconductor-stocks-that-could-gain-from-the-ai-frenzy)

Topics: Nvidia, Semiconductor Chips, Stocks, Technology 

- Artificial intelligence (AI) has significantly impacted the semiconductor industry, leading to advancements in chip design, manufacturing, and supply chain management.
- Arm Holdings (ARM) and STMicroelectronics (STM) are semiconductor companies poised to benefit from the AI frenzy in the long term.
- Arm Holdings (ARM) has shown strong revenue growth due to increased demand for its technology, with analysts predicting further earnings and revenue increases in the coming years.


[Google News - Google News](https://www.marketwatch.com/story/heres-a-way-to-look-beyond-nvidias-stock-for-ai-investments-cae6ab05)  
[Heres a way to look beyond Nvidias stock for AI investments - Google News](https://www.marketwatch.com/story/heres-a-way-to-look-beyond-nvidias-stock-for-ai-investments-cae6ab05)

Topics: Finance, Nvidia, Stock, Stocks, Technology 

- MarketWatch is a financial information website that provides news, analysis, and stock market data to help individuals make informed investment decisions.
- The website covers various financial topics such as stocks, bonds, currencies, and commodities, offering real-time market information and insights.
- MarketWatch also features tools like stock screeners, retirement planning resources, and personal finance guides to assist users in managing their finances effectively.


[Google News - Google News](https://www.cnbc.com/2024/07/15/mondays-analyst-calls-apple-upgraded-on-ai-push-nvidia-price-target-increased.html)  
[Monday's analyst calls: Apple upgraded on AI push, Nvidia price target increased - Google News](https://www.cnbc.com/2024/07/15/mondays-analyst-calls-apple-upgraded-on-ai-push-nvidia-price-target-increased.html)

Topics: Apple, Big Tech, Finance, Gen AI, Nvidia, Products, Stocks, Technology 

- Loop Capital upgraded Apple to buy from hold, predicting a surge of more than 30% driven by AI integration in future iPhones.
- TD Cowen increased Nvidia's price target to $165 per share, citing sustained demand for Blackwell and Hopper AI systems.
- JPMorgan reiterated Zoetis as a top pick, projecting nearly 26% upside with sustained high-single-digit growth and margin expansion in the animal health sector.


# AI Energy Sustainability 

[Google News - Google News](https://www.nytimes.com/2024/07/11/climate/artificial-intelligence-energy-usage.html)  
[A.I.s Insatiable Appetite for Energy - Google News](https://www.nytimes.com/2024/07/11/climate/artificial-intelligence-energy-usage.html)

Topics: AI Doom, Big Tech, Energy, Gen AI, Nvidia, OpenAI, Science, Sustainability, Technology 

- A.I.'s increasing electricity demands from data centers are straining the energy grid, contributing to higher emissions and hindering the energy transition.
- Tech giants such as Microsoft and Google are facing challenges in reducing the carbon footprint of A.I., with projections showing a significant increase in electricity consumption by data centers.
- The growth of A.I. is driving up energy demand and emissions, prompting companies to explore renewable energy sources while facing hurdles in balancing energy needs with environmental sustainability.


[Google News - Google News](https://www.fastcompany.com/91154629/ai-data-centers-energy-use-sustainable-solutions)  
[AI demand puts more pressure on data centers' energy use. Here's how to make it sustainable - Google News](https://www.fastcompany.com/91154629/ai-data-centers-energy-use-sustainable-solutions)

Topics: AI Demand, Big Tech, Climate, Data, Energy, Meta, Sustainability, Technology 

- The surge in energy consumption and carbon emissions by big tech companies due to the growth in artificial intelligence, particularly large language models like ChatGPT, has intensified the energy demand and put pressure on data centers. 
- The energy needs of AI are leading to a shift in energy strategies, with a focus on exploring previously unviable options like restarting dormant nuclear reactors and the challenges posed to the electrical grid which is struggling to keep pace with the rapid growth in AI. 
- Efforts are being made to address the energy crisis in data centers through improving energy efficiency in computing hardware, adopting new cooling technologies, and exploring the concept of flexible computing to optimize energy consumption based on availability, cost, and sustainability.


[Google News - Google News](https://oilprice.com/Energy/Energy-General/AIs-Rapid-Growth-Threatens-Energy-Industry-Economy-and-Climate.html)  
[AI's Rapid Growth Threatens Energy Industry, Economy, and Climate - Google News](https://oilprice.com/Energy/Energy-General/AIs-Rapid-Growth-Threatens-Energy-Industry-Economy-and-Climate.html)

Topics: AI Doom, Climate, Economics, Energy, Science, Sustainability, Technology 

- The rapid growth of Artificial Intelligence poses a significant threat to the energy industry, economy, and climate due to its high power consumption, challenging decarbonization goals, and potentially leading to energy shortages.
- The global AI sector is expected to consume 3.5 percent of the world’s electricity by 2030, with implications for energy security, economic growth, and increasing energy bills for consumers.
- The tech sector, particularly heavy hitters like Amazon and Microsoft, is facing challenges in meeting the escalating energy demands and is exploring solutions such as increased nuclear energy deployment, while acknowledging the need for innovative approaches like AI integration into smart grids for efficiency and emission reduction.


[South China Morning Post - South China Morning Post](https://www.scmp.com/business/article/3270513/paris-olympics-will-use-alibabas-ai-powered-energy-saving-tool-cut-venue-emissions)  
[Paris Olympics will use Alibabas AI-powered energy-saving tool to cut venue emissions - South China Morning Post](https://www.scmp.com/business/article/3270513/paris-olympics-will-use-alibabas-ai-powered-energy-saving-tool-cut-venue-emissions)

Topics: Big Tech, Climate, Energy, Products, Sustainability, Technology 

- Paris Olympics will use Alibaba’s AI-powered energy-saving tool to optimize energy demand and reduce venue emissions
- The AI tool will analyze power usage at 35 locations during the games to minimize waste and help achieve carbon neutrality
- Organizers aim to cut event-related carbon emissions by using renewable energy sources and sustainable practices at the Games


[Supercar Blondie - Supercar Blondie](https://supercarblondie.com/google-ai-uses-enough-electricity-in-one-second-to-charge-seven-electric-cars/)  
[Google AI uses enough electricity in one second to charge seven electric cars - Supercar Blondie](https://supercarblondie.com/google-ai-uses-enough-electricity-in-one-second-to-charge-seven-electric-cars/)

Topics: Big Tech, Climate, Energy, Google, Sustainability, Technology 

- Google AI uses enough electricity in one second to charge seven electric cars
- Google’s AI services, such as AI Overviews, require more energy than regular searches
- Google's 2023 greenhouse gas emissions increased by almost 50% from 2019, driven by AI compute demands


[Google News - Google News](https://www.teslarati.com/lg-energy-solution-ai-battery-designs/)  
[LG Energy Solution uses AI to design batteries in a day - Google News](https://www.teslarati.com/lg-energy-solution-ai-battery-designs/)

Topics: Data, Energy, Gen AI, Korea, Lg Energy Solution, Products, Science, Sustainability, Technology 

- LG Energy Solution is using artificial intelligence to design customized battery cells for clients in just one day, streamlining a process that typically takes longer.
- The AI-powered battery design system has been trained on 100,000 design cases from the past 30 years, ensuring high-quality designs consistently and at a fast pace.
- Incorporating AI in cell design allows for consistent quality and speed, providing competitive advantage and differentiated customer value in the battery sector, particularly in the growing market for electric vehicles.


[Google News - Google News](https://investorplace.com/2024/07/ais-dirty-secret-why-nvidia-stock-investors-should-fear-the-coming-energy-crunch/)  
[AIs Dirty Secret: Why Nvidia Stock Investors Should Fear the Coming Energy Crunch - Google News](https://investorplace.com/2024/07/ais-dirty-secret-why-nvidia-stock-investors-should-fear-the-coming-energy-crunch/)

Topics: Energy, Nvidia, Nvidia Stock, Sustainability, Technology 

- Nvidia's AI chips' energy use is a growing concern, but the stock price is not reflecting these worries due to strong sales and profits, with potential for further growth and dominance in the cloud industry.
- The competition in the AI sector is intensifying with the race to develop powerful language models, leading to increased hardware requirements that benefit companies like Nvidia.
- The profitability of AI investments hinges on managing the energy costs associated with running AGI models, with implications for the success of cloud providers and the industry as a whole.


[Google News - Google News](https://koreajoongangdaily.joins.com/news/2024-07-14/business/industry/New-LG-Energy-Solution-AI-designs-battery-cells-in-less-than-a-day/2089779)  
[New LG Energy Solution AI designs battery cells in less than a day - Google News](https://koreajoongangdaily.joins.com/news/2024-07-14/business/industry/New-LG-Energy-Solution-AI-designs-battery-cells-in-less-than-a-day/2089779)

Topics: Data, Energy, Gen AI, Korea, Lg Energy Solution, Products, Reinforcement Learning, Science, Sustainability, Technology 

- LG Energy Solution developed an AI-powered system that shortens the battery cell design process to a day from the previous two-week timeframe.
- The AI model is trained on 100,000 design cases from 30 years of data, leading to substantial cost savings and improved operational efficiency.
- The AI can generate a design plan based on client requirements, including performance verification, within a day, aiming to enhance product competitiveness and customer value.


[Bloomberg - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-15/chipmaking-hub-taiwan-hikes-power-demand-outlook-on-ai-boom)  
[Chipmaking Hub Taiwan Hikes Power Demand Outlook on AI Boom - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-15/chipmaking-hub-taiwan-hikes-power-demand-outlook-on-ai-boom)

Topics: Big Tech, Economics, Energy, Semiconductor Chips, Taiwan, Technology 

- Taiwan's power consumption is projected to increase significantly, with a focus on AI sector growth.
- Machine learning consumption expected to grow eight times by 2028.
- Taiwan aims to transition to renewable energy and increase its energy mix to a quarter from renewables by 2030.


[Google News - Google News](https://www.koreaherald.com/view.php?ud=20240714050126)  
[LG Energy Solutions AI model to expedite battery cell design - Google News](https://www.koreaherald.com/view.php?ud=20240714050126)

Topics: AI Model, Battery Cell Design, Data, Energy, Korea, Products, Science, Sustainability, Technology 

- LG Energy Solution developed an AI solution to shorten battery cell design period from two weeks to one day using generative AI techniques and data from 100,000 design cases.
- The AI model ensures optimal cell composition for capacity, energy density, and resistance, allowing for streamlined design tasks and one-day completion of the process.
- The company anticipates cost savings, improved efficiency, and enhanced competitiveness by integrating AI into the battery design process.


# AI Predicting Alzheimer's Progression 

[The Brattleboro Reformer - The Brattleboro Reformer](https://www.reformer.com/health/ai-better-at-predicting-progression-to-alzheimers-than-standard-care/article_1efc5053-677e-500a-baa6-ab79404d3034.html)  
[AI Better at Predicting Progression to Alzheimer's Than Standard Care - The Brattleboro Reformer](https://www.reformer.com/health/ai-better-at-predicting-progression-to-alzheimers-than-standard-care/article_1efc5053-677e-500a-baa6-ab79404d3034.html)

Topics: Alzheimer'S Prediction, Cognitive Science, Data, Health, Fitness, Healthcare, Review, Science, Technology, Testing 

- AI program outperforms doctors in predicting progression to Alzheimer's from early dementia
- AI is three times more accurate than standard care in predicting progression
- AI can identify the speed at which patients will progress to Alzheimer’s


[PsyPost - PsyPost](https://www.psypost.org/ai-predicts-alzheimers-disease-risk-with-remarkable-accuracy-through-subtle-speech-patterns/)  
[AI predicts Alzheimers disease risk with remarkable accuracy through subtle speech patterns - PsyPost](https://www.psypost.org/ai-predicts-alzheimers-disease-risk-with-remarkable-accuracy-through-subtle-speech-patterns/)

Topics: Cognitive Science, Data, Health, Fitness, Healthcare, Language Models, Science, Technology, Testing 

- Boston University researchers developed an AI model with 78.5% accuracy in predicting Alzheimer's progression by analyzing speech patterns from neuropsychological tests, offering a non-invasive early detection method.
- The AI model outperformed traditional diagnostic methods and demographic factors, showing strong predictive power in identifying individuals likely to develop Alzheimer's within six years.
- Older women, individuals with lower education levels, and those carrying certain genetic markers were found to be more likely to progress to Alzheimer's, aligning with existing research and reinforcing the validity of the AI model's predictions.


[Google News - Google News](https://medicalxpress.com/news/2024-07-artificial-intelligence-outperforms-clinical-alzheimer.html)  
[Artificial intelligence outperforms clinical tests at predicting progress of Alzheimer's disease - Google News](https://medicalxpress.com/news/2024-07-artificial-intelligence-outperforms-clinical-alzheimer.html)

Topics: Cognitive Science, Gen AI, Health, Fitness, Healthcare, Science, Technology, Testing 

- Cambridge scientists developed an AI tool that predicts Alzheimer's progression accurately in 4 out of 5 cases, potentially reducing the need for invasive diagnostic tests and improving treatment outcomes.
- The AI model showed 82% accuracy in identifying individuals who developed Alzheimer's within three years, outperforming current clinical markers and diagnosis methods, which could significantly reduce misdiagnosis.
- The AI tool stratifies individuals into different progression groups, aiding in early treatment interventions and identifying individuals who may benefit from monitoring or alternative clinical pathways based on symptoms.


[Google News - Google News](https://www.earth.com/news/ai-tool-better-predicting-alzheimers-disease-than-current-tests/)  
[AI tool is better at predicting Alzheimers disease than current tests - Google News](https://www.earth.com/news/ai-tool-better-predicting-alzheimers-disease-than-current-tests/)

Topics: Alzheimer'S Prediction, Cognitive Science, Health, Fitness, Healthcare, Science, Technology, Testing 

- Cambridge researchers developed an AI tool using cognitive tests and MRI scans to predict Alzheimer's progression for early detection.
- The AI tool outperformed current clinical diagnostic tools in predicting Alzheimer's progression correctly in 82% of cases using non-invasive, low-cost patient data.
- The AI tool offers significant benefits such as reduced misdiagnosis rates, improved patient wellbeing, and avoidance of unnecessary invasive and costly diagnostic tests.


[Google News - Google News](https://scitechdaily.com/revolutionizing-alzheimers-care-with-predictive-modeling/)  
[Revolutionizing Alzheimers Care With Predictive Modeling - Google News](https://scitechdaily.com/revolutionizing-alzheimers-care-with-predictive-modeling/)

Topics: Cognitive Science, Data, Health, Fitness, Healthcare, Science, Technology 

- Amsterdam UMC's Alzheimer Centre has developed a prediction model using patient data to forecast cognitive decline in Alzheimer's patients for personalized care plans.
- The model is supported by an upcoming app to guide treatment discussions based on predicted disease progression, aiming for individualized predictions.
- The prediction model is based on data from nearly 1000 patients with Alzheimer's disease, utilizing information like age, gender, cognitive test scores, MRI scans, and biomarkers to provide tailored predictions despite uncertainties.


[The Next Web - The Next Web](https://next.thenextweb.com/news/europe-universities-using-ai-battle-dementia)  
[How Europes universities are using AI to battle dementia - The Next Web](https://next.thenextweb.com/news/europe-universities-using-ai-battle-dementia)

Topics: Cognitive Science, Europe, Science, Technology 

- European universities are developing AI tools to aid in the diagnosis and treatment of dementia, with a focus on battling Alzheimer's disease.
- Initiatives include predicting Alzheimer's development with AI models, early diagnosis using AI tools with high accuracy, and tracking protein clumps to deepen understanding and potentially discover new drugs.
- Projects like AI-Mind aim to reduce diagnosis time from years to a single week and advance the detection and treatment of dementia through advanced technologies.


[Google News - Google News](https://fortune.com/well/2024/07/08/thrive-ai-health-coach-sam-altman-arianna-huffington/)  
[OpenAIs Sam Altman and Thrive Globals Arianna Huffington want to share the 'miracle drug' to extend human life - Google News](https://fortune.com/well/2024/07/08/thrive-ai-health-coach-sam-altman-arianna-huffington/)

Topics: OpenAI, Sam Altman 

- OpenAI's Sam Altman and Thrive Global's Arianna Huffington are collaborating to find technological solutions using AI for the 130 million Americans with chronic conditions.
- They are launching Thrive AI Health, creating an AI health coach that offers personalized recommendations based on biometrics and lifestyle habits.
- The platform aims to improve health spans and life spans by focusing on behavior change, with a team including DeCarlos Love, a former Google executive, and partnerships with research institutions like Stanford Medicine.


[Live Science - Live Science](https://www.livescience.com/technology/artificial-intelligence/new-in-car-ai-can-spot-drunk-drivers-by-constantly-scanning-their-face)  
[New in-car AI can flag drunk drivers by constantly scanning their face - Live Science](https://www.livescience.com/technology/artificial-intelligence/new-in-car-ai-can-spot-drunk-drivers-by-constantly-scanning-their-face)

Topics: Baidu, Copilot, Products, Science, Technology 

- In-car AI using facial scanning can flag drunk drivers with 75% accuracy by analyzing facial features and behavior through an in-vehicle camera.
- The AI system can detect intoxication levels at the start of a drive, potentially preventing impaired drivers from getting on the road and reducing drunk driving accidents.
- The research project aims to integrate the technology seamlessly into smart vehicles, providing a potential solution to the persistent problem of drunk driving contributing to fatal accidents globally.


[Google News - Google News](https://theconversation.com/why-an-ai-health-coach-wont-solve-the-worlds-chronic-disease-problems-234369)  
[Why an AI health coach wont solve the worlds chronic disease problems - Google News](https://theconversation.com/why-an-ai-health-coach-wont-solve-the-worlds-chronic-disease-problems-234369)

Topics: Health, Fitness, Healthcare, Opinion, Technology 

- Collaboration announced to develop a hyper-personalized AI health coach to address chronic diseases.
- Concerns raised about the effectiveness of AI in health due to potential biases and errors in AI models.
- Emphasis on individual lifestyle choices overlooks the importance of social determinants of health in addressing chronic health problems.


# AI in the Workplace 

[Google News - Google News](https://www.cnbc.com/2024/07/13/why-older-workers-key-ai-understanding.html)  
[Why older workers are critical to AI adoption in the office - Google News](https://www.cnbc.com/2024/07/13/why-older-workers-key-ai-understanding.html)

Topics: Ethics, Jobs, Careers, Labor Market, Technology 

- Older workers have unique advantages in relation to AI adoption despite stereotypes about their ability to adapt to new technology.
- Senior-level employees may fear being fired for lacking AI skills, but experienced workers are effective at applying inputs and assessing outputs with judgment that AI may not possess.
- Experts suggest strategies like reverse mentoring and upskilling to help senior-level workers embrace AI augmentation in the workplace, as soft skills become increasingly important in the future job market.


[Google News - Google News](https://www.fastcompany.com/91153187/will-ai-become-your-assistant-or-your-boss-an-ethicist-explains)  
[Will AI become your assistant or your boss? An ethicist explains - Google News](https://www.fastcompany.com/91153187/will-ai-become-your-assistant-or-your-boss-an-ethicist-explains)

Topics: Ethics, Gen AI, Jobs, Careers, Labor Market, Opinion, Society, Culture, Technology 

- AI advancements are transforming workplace management, with agentic AI systems offering independent goal-achieving capabilities but also posing unique ethical challenges.
- The evolving impact of AI on employment includes the potential for AI to monopolize intellectual work, leading to potential de-skilling of the labor force and fragmented job roles.
- Algorithmic management with AI systems risks jeopardizing worker autonomy and well-being, potentially creating impersonal and tyrannical work environments that pressure human workers to adopt machine-like behaviors.


[Bloomberg Law - Bloomberg Law](https://news.bloomberglaw.com/safety/deploying-ai-for-worker-safety-needs-legal-prep-from-employers)  
[Deploying AI for Worker Safety Needs Legal Prep From Employers - Bloomberg Law](https://news.bloomberglaw.com/safety/deploying-ai-for-worker-safety-needs-legal-prep-from-employers)

Topics: Employers, Jobs, Careers, Labor Market, Laws, Policy And Regulation, Technology 

- AI programs are being used for worker safety to detect hazards and improve workplace safety and health.
- Attorneys emphasize the importance for employers to consider legal issues such as labor and privacy concerns before implementing AI for safety.
- Employers should consult legal staff to address potential pitfalls and ensure transparency with employees and unions when deploying AI technology for safety monitoring.


[TechRadar - TechRadar](https://www.techradar.com/pro/software-workers-across-the-globe-are-excited-to-get-using-gen-ai-at-work)  
[Software workers across the globe are excited to get using Gen AI at work - TechRadar](https://www.techradar.com/pro/software-workers-across-the-globe-are-excited-to-get-using-gen-ai-at-work)

Topics: Gen AI, Jobs, Careers, Labor Market, Technology 

- Global software workers are excited about the adoption of GenAI tools, with an overwhelming 85% expecting to use them within the next two years.
- GenAI is anticipated to significantly transform software professionals' roles by automating repetitive tasks, allowing them to focus on more meaningful work.
- Despite concerns about displacing human workers, organizations with GenAI initiatives aim to leverage the technology for improved innovation and software quality, alongside upskilling their workforce and avoiding headcount reduction.


[Google News - Google News](https://www.ndtvprofit.com/technology/ai-can-replace-ceos-when-it-starts-thinking-like-us)  
[AI Can Replace CEOs When It Starts Thinking Like Us - Google News](https://www.ndtvprofit.com/technology/ai-can-replace-ceos-when-it-starts-thinking-like-us)

Topics: Technology 

- The use of generative artificial intelligence is increasing, with companies developing tools to assist senior leadership and employees. However, emotional and ethical scenarios are challenging for AI to handle accurately.
- AI has been used to streamline processes and improve efficiency within companies, with examples like a rotating AI CEO being introduced to enhance work tasks and decision-making.
- While AI adoption by CEOs is anticipated to increase and some experts believe AI can act as a copilot, challenges remain in areas such as emotional intelligence, proactiveness, and complexity of decision-making that require human judgement.


[Google News - Google News](https://www.forbes.com/sites/timothypapandreou/2024/07/10/is-your-business-ready-for-ai-heres-how-to-tell/)  
[Are Business Leaders Ready For AI? Heres How To Tell - Google News](https://www.forbes.com/sites/timothypapandreou/2024/07/10/is-your-business-ready-for-ai-heres-how-to-tell/)

Topics: Technology 

- Businesses need a vision and team-centered plan to unlock AI's true potential
- AI should be seen as a growth engine, not just a cost-saving tool, by exploring new revenue streams
- Human-AI collaboration empowers employees by democratizing access to expertise and allowing for innovation and growth through experimentation and iteration


[Google News - Google News](https://investorplace.com/hypergrowthinvesting/2024/07/apple-intelligence-the-next-frontier-in-ai-development/)  
[Apple Intelligence: The Next Frontier in AI Development - Google News](https://investorplace.com/hypergrowthinvesting/2024/07/apple-intelligence-the-next-frontier-in-ai-development/)

Topics: Apple, Big Tech, Technology 

- Apple introduced Apple Intelligence at its annual Worldwide Developers Conference, focusing on hyper-personalized AI to turn the iPhone into a powerful assistant similar to J.A.R.V.I.S. from Iron Man.
- Apple is building AI to help individuals with everyday tasks more easily and on a large scale, aiming to create a custom AI model tailored to users' preferences and needs.
- Apple is partnering with OpenAI to integrate ChatGPT into the iPhone, combining Apple's data with OpenAI's AI models to potentially become the leading AI system in the world.


[Bloomberg Law - Bloomberg Law](https://news.bloomberglaw.com/daily-labor-report/employers-find-openings-to-share-ai-bias-liability-with-vendors)  
[Employers Find Openings to Share AI Bias Liability With Vendors - Bloomberg Law](https://news.bloomberglaw.com/daily-labor-report/employers-find-openings-to-share-ai-bias-liability-with-vendors)

Topics: AI Bias, Employers, Ethics, Technology 

- Employers could use novel legal arguments to sue vendors and share liability for AI bias in hiring tools
- Discrimination lawsuits against AI developers in hiring tools are emerging, with debates on liability as an agent or employment agency
- Vendors may face challenges with indemnification, breach of contract, aiding discrimination allegations, and state laws regulating AI hiring tools.


[Google News - Google News](https://www.mckinsey.com/capabilities/mckinsey-digital/our-insights/gen-ai-and-beyond-where-else-to-focus-now)  
[Gen AI and beyond: Where else to focus now - Google News](https://www.mckinsey.com/capabilities/mckinsey-digital/our-insights/gen-ai-and-beyond-where-else-to-focus-now)

Topics: Gen AI, OpenAI, Opinion, Technology 

- Gen AI has rightly garnered attention, but leaders need to balance the focus on it with other digital tools essential for business success, considering where value is derived from transformations.
- The concept of every company becoming a neural business emphasizes the need for intricate organizational structures that enable effective connections and autonomous, aligned teams to drive speed and scale.
- Leaders should be aware of the combined power of integrating various technologies, like AI and quantum computing, to create breakthroughs in business models and should focus on upskilling talent and fostering a culture of continuous learning for future workforce planning.


[Google News - Google News](https://www.cio.com/article/2509754/generative-ai-and-preparing-for-a-shift-to-skills-based-hiring.html)  
[Generative AI and preparing for a shift to skills-based hiring - Google News](https://www.cio.com/article/2509754/generative-ai-and-preparing-for-a-shift-to-skills-based-hiring.html)

Topics: Gen AI, Generative AI, Jobs, Careers, Labor Market, Technology 

- Organizations need to reevaluate hiring and training approaches to keep pace with generative AI in modern business strategies, emphasizing a shift to skills-based hiring mindset.
- Understanding the impact of generative AI on tasks and skills is crucial for planning upskilling strategies and identifying which skills can be automated or supported by AI.
- Organizations should redefine skills, categorize jobs, and focus on skills development, as well as shift metrics to measure human performance and emphasize both business and human outcomes.


# Nvidia stock 

[Barron's - Barron's](https://www.barrons.com/articles/nvidia-stock-price-buy-sell-28747c35)  
[Nvidia Stock Rises. How Earnings Could Reignite the AI Chips Rally. - Barron's](https://www.barrons.com/articles/nvidia-stock-price-buy-sell-28747c35)

Topics: AI Chips Rally, Earnings, Finance, Nvidia, Nvidia Stock, OpenAI, Semiconductor Chips, Stocks, Technology 

- Barron's is an American weekly newspaper that provides financial information, market developments, and relevant news to investors.
- Access to Barron's content may require a subscription, providing exclusive insights and analysis for subscribers.
- The publication covers various topics including market trends, investment strategies, and recommendations for stocks and other financial assets.


[Google News - Google News](https://www.msn.com/en-us/money/news/nvidia-to-rally-over-40-here-are-10-top-analyst-forecasts-for-tuesday/ar-BB1pFBW1)  
[Nvidia To Rally Over 40%? Here Are 10 Top Analyst Forecasts For Tuesday - Google News](https://www.msn.com/en-us/money/news/nvidia-to-rally-over-40-here-are-10-top-analyst-forecasts-for-tuesday/ar-BB1pFBW1)

Topics: Analyst Forecasts, Nvidia, Stock Rally, Stocks, Technology 

- Keybanc raised NVIDIA Corporation's price target from $130 to $180 and maintained an Overweight rating.
- NVIDIA's shares rose by 1.9% to close at $128.20 on Monday.
- The article discusses top Wall Street analysts changing their outlook on various companies, including NVIDIA.


[Google News - Google News](https://www.benzinga.com/news/24/07/39748729/nvidia-to-rally-more-than-33-here-are-10-top-analyst-forecasts-for-friday)  
[NVIDIA To Rally More Than 33%? Here Are 10 Top Analyst Forecasts For Friday - Google News](https://www.benzinga.com/news/24/07/39748729/nvidia-to-rally-more-than-33-here-are-10-top-analyst-forecasts-for-friday)

Topics: Analyst Forecasts, Finance, Nvidia, Semiconductor Chips, Stock Rally, Stocks, Technology 

- Top Wall Street analysts changed their outlook on various stocks, including price target adjustments and rating changes.
- Analysts from different firms altered forecasts for companies such as Adobe, KalVista Pharmaceuticals, Visteon Corporation, Block, Amylyx Pharmaceuticals, Delta Air Lines, NVIDIA Corporation, Costco Wholesale Corporation, Tesla, Inc., and NXP Semiconductors N.V.
- Changes included price target adjustments and rating shifts, with some stocks experiencing gains or losses following the analyst's actions.


[Google News - Google News](https://finance.yahoo.com/news/now-time-look-buying-nvidia-130023050.html)  
[Is Now The Time To Look At Buying NVIDIA Corporation (NASDAQ:NVDA)? - Google News](https://finance.yahoo.com/news/now-time-look-buying-nvidia-130023050.html)

Topics: Finance, Nvidia, Semiconductor Chips, Stocks, Technology 

- NVIDIA Corporation (NVDA) experienced a substantial 70% share price increase on NASDAQGS recently, bringing it close to its yearly peak.
- Current analysis indicates that NVDA is trading at a premium compared to industry peers, with potential for further price fluctuations due to its high volatility.
- While NVDA shows promising future growth potential with expected earnings to double in the coming years, current shareholders may consider whether to sell at the current high price or wait for a potential price decrease.


[Google News - Google News](https://www.marketwatch.com/story/heres-a-way-to-look-beyond-nvidias-stock-for-ai-investments-cae6ab05)  
[Heres a way to look beyond Nvidias stock for AI investments - Google News](https://www.marketwatch.com/story/heres-a-way-to-look-beyond-nvidias-stock-for-ai-investments-cae6ab05)

Topics: Finance, Nvidia, Stock, Stocks, Technology 

- MarketWatch is a financial information website that provides news, analysis, and stock market data to help individuals make informed investment decisions.
- The website covers various financial topics such as stocks, bonds, currencies, and commodities, offering real-time market information and insights.
- MarketWatch also features tools like stock screeners, retirement planning resources, and personal finance guides to assist users in managing their finances effectively.


[Google News - Google News](https://investorplace.com/2024/07/ais-dirty-secret-why-nvidia-stock-investors-should-fear-the-coming-energy-crunch/)  
[AIs Dirty Secret: Why Nvidia Stock Investors Should Fear the Coming Energy Crunch - Google News](https://investorplace.com/2024/07/ais-dirty-secret-why-nvidia-stock-investors-should-fear-the-coming-energy-crunch/)

Topics: Energy, Nvidia, Nvidia Stock, Sustainability, Technology 

- Nvidia's AI chips' energy use is a growing concern, but the stock price is not reflecting these worries due to strong sales and profits, with potential for further growth and dominance in the cloud industry.
- The competition in the AI sector is intensifying with the race to develop powerful language models, leading to increased hardware requirements that benefit companies like Nvidia.
- The profitability of AI investments hinges on managing the energy costs associated with running AGI models, with implications for the success of cloud providers and the industry as a whole.


[The Motley Fool - The Motley Fool](https://www.fool.com/investing/2024/07/15/heres-my-top-ai-stock-buy-now/)  
[Here Is My TopArtificial Intelligence(AI) Stock to Buy Right Now - The Motley Fool](https://www.fool.com/investing/2024/07/15/heres-my-top-ai-stock-buy-now/)

Topics: Finance, Investment, Nvidia, Stock, Stocks, Technology 

- Nvidia is a top artificial intelligence (AI) stock attracting investors due to leveraging gaming and GPUs, but its high valuation might lead to a significant drop, prompting investors to look at a long-time partner instead.
- Super Micro Computer (SMCI) is a potential investment with significant growth, having experienced a 4,500% increase in stock value in the last five years by providing Nvidia-powered AI equipment.
- Despite Supermicro's massive gains, its strong financial performance, including revenue growth of 95% year-over-year and raised guidance for fiscal 2024, indicates the stock's potential for further growth and offers a relatively inexpensive investment opportunity compared to Nvidia.



[Business Insider - Business Insider](https://www.businessinsider.com/cathie-wood-would-have-held-nvidia-if-knew-going-up-2024-7)  
[Cathie Wood says she wouldn't have sold Nvidia stake 'had we known that the market was going to reward it' - Business Insider](https://www.businessinsider.com/cathie-wood-would-have-held-nvidia-if-knew-going-up-2024-7)

Topics: Finance, Nvidia, Opinion, Stocks, Technology 

- Cathie Wood wouldn't have sold Nvidia stock if she knew about its significant market gain.
- Nvidia's stock surged by 172% this year, briefly holding the title of the most valuable company.
- Wood's Ark Invest missed out on Nvidia's rally due to selling its position prematurely.


[Google News - Google News](https://finance.yahoo.com/news/prediction-nvidias-stock-split-3-101500672.html)  
[Prediction: After Nvidia's Stock Split, These 3 Artificial Intelligence (AI) Companies Could Be Next - Google News](https://finance.yahoo.com/news/prediction-nvidias-stock-split-3-101500672.html)

Topics: Nvidia, Stocks, Technology 

- Stock splits have become popular in 2024, with Nvidia being a prominent example, and now people are speculating on which AI company could be next to split.
- Meta Platforms, Super Micro Computer, and Netflix are among the speculated companies that might split their stocks next due to their financial performance, market position, and high share prices.
- Meta Platforms is highlighted for its strong performance, lack of previous stock splits, and potential for growth, while Super Micro Computer and Netflix are also seen as potential candidates for stock splits based on their market performance and financial outlook.


# AI Privacy 

[Google News - Google News](https://www.howtogeek.com/does-ai-really-need-your-personal-data/)  
[Does AI Really Need Your Personal Data? - Google News](https://www.howtogeek.com/does-ai-really-need-your-personal-data/)

Topics: Big Tech, Data, Ethics, Gen AI, Language Models, Meta, Opinion, Privacy, Products, Science, Society, Culture, Technology 

- AI relies on data sets for training as it lacks sentience and only mimics human understanding.
- Developers require user data to improve AI, emphasizing the importance of privacy protection and trust.
- Users need to adhere to standard internet safety rules when interacting with AI to safeguard personal information.


[Google News - Google News](https://www.xda-developers.com/google-gemini-peeking-peoples-files/)  
[Google Gemini may be peeking into people's files, even when told not to - Google News](https://www.xda-developers.com/google-gemini-peeking-peoples-files/)

Topics: AI Privacy Concerns, Big Tech, Cybersecurity, Data, Ethics, Gemini, Google, Policy And Regulation, Privacy, Products, Technology 

- Google Gemini may be scanning files without permission, causing privacy concerns among users.
- Kevin Bankston discovered Gemini summarizing his tax return after opening a PDF file.
- Some users worry that Google's productivity services are used to train its AI, despite useful intent.


[Google News - Google News](https://itc.ua/en/news/google-gemini-artificial-intelligence-snooped-through-a-user-s-file-without-permission/)  
[Google Gemini artificial intelligence snooped through a user's file without permission - Google News](https://itc.ua/en/news/google-gemini-artificial-intelligence-snooped-through-a-user-s-file-without-permission/)

Topics: AI Doom, Big Tech, Data, Ethics, Gemini, Gen AI, Google, Privacy, Privacy Concerns, Technology 

- Google Gemini, an artificial intelligence program, reportedly accessed a user's file without permission, providing unsolicited summaries of documents.
- A user experienced Gemini appearing and providing opinions on a PDF tax return without being prompted, despite not interacting with the assistant intentionally.
- Concerns were raised regarding Google's productivity services potentially being used to train AI, with users questioning the extent of Gemini's access to user files.


[ZDNet - ZDNet](https://www.zdnet.com/article/rabbit-r1-ai-assistant-has-secretly-been-storing-user-chats-that-cant-be-deleted/)  
[Rabbit r1 AI assistant has secretly been storing user chats that can't be deleted - ZDNet](https://www.zdnet.com/article/rabbit-r1-ai-assistant-has-secretly-been-storing-user-chats-that-cant-be-deleted/)

Topics: AI Doom, Big Tech, Cybersecurity, Data, Ethics, Privacy, Products, Technology 

- Rabbit r1 AI assistant has been storing user chats without the ability to delete them, potentially exposing chat logs if the device is lost or stolen.
- A software update has been implemented to address the issue, allowing users to perform a factory reset to erase all data and reducing the amount of stored data on the device.
- Pairing data on the device could be accessed to view log files with saved requests, photos, and more; however, Rabbit has fixed this issue to prevent unauthorized access.


[Google News - Google News](https://www.computing.co.uk/news/4332673/chatgpts-mac-app-exposed-user-conversations)  
[ChatGPT's Mac app exposed user conversations - Google News](https://www.computing.co.uk/news/4332673/chatgpts-mac-app-exposed-user-conversations)

Topics: Big Tech, Chatgpt, Code Assistants, Cybersecurity, Data, Ethics, Gen AI, Language Models, OpenAI, Privacy, Privacy Concerns, Products, Technology 

- OpenAI's Mac app for ChatGPT had a security flaw exposing user conversations.
- The app stored conversation history in plain text, making it vulnerable to malicious access.
- Users could interact with ChatGPT directly from their desktops through the app.


[Reddit - Reddit](https://www.reddit.com/r/ChatGPT/comments/1e3jnbi/ai_is_a/)  
[AI is a... - Reddit](https://www.reddit.com/r/ChatGPT/comments/1e3jnbi/ai_is_a/)

Topics: OpenAI, Society, Culture, Technology 

- Rules for posting on r/ChatGPT include guidelines on malicious communication, trashposts, and self-advertising
- Political discussions related to AI and LLMs are allowed, but not discussions on specific political figures like Trump or Biden
- Posts on r/ChatGPT should be directly related to ChatGPT or LLM topics, and not solely focus on advertising another service


[Google News - Google News](https://www.tomshardware.com/tech-industry/artificial-intelligence/gemini-ai-caught-scanning-google-drive-hosted-pdf-files-without-permission-user-complains-feature-cant-be-disabled)  
[Google's Gemini AI caught scanning Google Drive hosted PDF files without permission  user complains feature can't be disabled - Google News](https://www.tomshardware.com/tech-industry/artificial-intelligence/gemini-ai-caught-scanning-google-drive-hosted-pdf-files-without-permission-user-complains-feature-cant-be-disabled)

Topics: AI Doom, Big Tech, Cybersecurity, Data, Ethics, Gemini, Gemini AI, Gen AI, Google, Meta, Pdf Files, Policy And Regulation, Privacy, Privacy Concerns, Products, Scanning, Technology 

- Google's Gemini AI has been scanning Google Drive hosted PDF files without explicit user permission, causing concerns about privacy and lack of control over sensitive information.
- The issue seems to be happening within Google Drive rather than Google Docs, with potential theories suggesting malfunctioning internal systems or overrides from Google Workspace Labs settings.
- Despite user settings being disabled for Gemini summaries in Gmail, Drive, and Docs, the AI continues to automatically trigger for specific file types, highlighting the importance of user consent and privacy in the use of AI technology.


[Bloomberg Law - Bloomberg Law](https://news.bloomberglaw.com/us-law-week/amgen-hasnt-resolved-questions-on-ai-medical-invention-patents)  
[Amgen Hasnt Resolved Questions on AI Medical Invention Patents - Bloomberg Law](https://news.bloomberglaw.com/us-law-week/amgen-hasnt-resolved-questions-on-ai-medical-invention-patents)

Topics: Gen AI, Health, Fitness, Healthcare, Intellectual Property, Review, Technology 

- WilmerHale attorneys analyze enablement of AI inventions in precision medicine after Amgen ruling
- Challenges in obtaining patents for AI-based technologies under current US patent jurisprudence
- Uncertainty in disclosing training data and model functionality to enable AI patents, considering the evolving nature of AI models


[Google News - Google News](https://www.techzine.eu/news/privacy-compliance/122274/google-gemini-reads-drive-files-without-permission/)  
[Google Gemini reads Drive files without permission - Google News](https://www.techzine.eu/news/privacy-compliance/122274/google-gemini-reads-drive-files-without-permission/)

Topics: AI Privacy Concerns, Big Tech, Cybersecurity, Data, Ethics, Gemini, Google, Policy And Regulation, Privacy, Products, Technology 

- Google Gemini is reading Drive documents without users' permission as claimed by Kevin Bankston, a senior advisor at CDT.
- Bankston's account was set up in a way that the AI assistant could not create summaries, but the issue pertains to Google Drive, not just Docs.
- The exact cause and scope of the problem have not been commented on by Google, and it may be specific to Google Workspace Labs users testing new features.


[Google News - Google News](https://investorplace.com/hypergrowthinvesting/2024/07/apple-intelligence-the-next-frontier-in-ai-development/)  
[Apple Intelligence: The Next Frontier in AI Development - Google News](https://investorplace.com/hypergrowthinvesting/2024/07/apple-intelligence-the-next-frontier-in-ai-development/)

Topics: Apple, Big Tech, Technology 

- Apple introduced Apple Intelligence at its annual Worldwide Developers Conference, focusing on hyper-personalized AI to turn the iPhone into a powerful assistant similar to J.A.R.V.I.S. from Iron Man.
- Apple is building AI to help individuals with everyday tasks more easily and on a large scale, aiming to create a custom AI model tailored to users' preferences and needs.
- Apple is partnering with OpenAI to integrate ChatGPT into the iPhone, combining Apple's data with OpenAI's AI models to potentially become the leading AI system in the world.


# Apple 

[Investing.com - Investing.com](https://au.investing.com/news/stock-market-news/apple-gains-after-morgan-stanley-calls-stock-top-pick-for-ai-efforts-3340208)  
[Apple gains after Morgan Stanley calls stock 'top pick' for AI efforts - Investing.com](https://au.investing.com/news/stock-market-news/apple-gains-after-morgan-stanley-calls-stock-top-pick-for-ai-efforts-3340208)

Topics: Apple, Big Tech, Finance, Gen AI, Products, Stocks, Technology 

- Apple's shares rose 2% after Morgan Stanley designated them as a "top pick" due to AI efforts, expecting a boost in device sales.
- Apple unveiled Apple Intelligence to compete with Google and Microsoft-backed OpenAI, with potential to sell nearly 500 million iPhones in the next two years.
- Industry analysts predict Apple and Samsung to lead global smartphone market recovery, despite Apple's slight market share drop in the previous quarter.


[Google News - Google News](https://investorplace.com/hypergrowthinvesting/2024/07/apple-intelligence-the-next-frontier-in-ai-development/)  
[Apple Intelligence: The Next Frontier in AI Development - Google News](https://investorplace.com/hypergrowthinvesting/2024/07/apple-intelligence-the-next-frontier-in-ai-development/)

Topics: Apple, Big Tech, Technology 

- Apple introduced Apple Intelligence at its annual Worldwide Developers Conference, focusing on hyper-personalized AI to turn the iPhone into a powerful assistant similar to J.A.R.V.I.S. from Iron Man.
- Apple is building AI to help individuals with everyday tasks more easily and on a large scale, aiming to create a custom AI model tailored to users' preferences and needs.
- Apple is partnering with OpenAI to integrate ChatGPT into the iPhone, combining Apple's data with OpenAI's AI models to potentially become the leading AI system in the world.


[Yahoo Finance - Yahoo Finance](https://sg.finance.yahoo.com/news/apple-inc-nasdaq-aapl-aswath-214249845.html)  
[Is Apple Inc (NASDAQ:AAPL) Aswath Damodarans Best AI Stock Pick? - Yahoo Finance](https://sg.finance.yahoo.com/news/apple-inc-nasdaq-aapl-aswath-214249845.html)

Topics: Opinion, Stocks 

- Aswath Damodaran considers Apple Inc (AAPL) part of the "Magnificent Seven" group of stocks that have added significant market cap, emphasizing their value in the market.
- Despite concerns about overvaluation in the tech sector, Damodaran views Apple as a "money machine" with a competitive edge in AI, supported by analysts like Dan Ives and Ming-Chi Kuo.
- Though facing short-term challenges, Apple's growth potential is highlighted by its ecosystem of devices and paying subscribers, positioning it well for cash flow growth and intrinsic value appreciation.



[Bloomberg - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-15/apple-is-top-pick-at-morgan-stanley-as-ai-seen-driving-iphone-ipad-upgrades)  
[Apple Is Top Pick at Morgan Stanley as AI Seen Driving iPhone, iPad Upgrades - Bloomberg](https://www.bloomberg.com/news/articles/2024-07-15/apple-is-top-pick-at-morgan-stanley-as-ai-seen-driving-iphone-ipad-upgrades)

Topics: Apple, Big Tech, Finance, Products, Stocks, Technology 

- Apple named top pick at Morgan Stanley due to the potential impact of its AI platform on iPhone, iPad upgrades
- Analyst forecasts a significant increase in iPhone shipments over the next two years
- Apple's AI services release triggered a surge in share prices, with expectations for a multi-year device upgrade cycle starting in FY25


[Google News - Google News](https://9to5mac.com/2024/07/15/apple-lags-behind-global-smartphone-market/)  
[Apple lags behind global smartphone market, but AI expected to drive Q3 demand - Google News](https://9to5mac.com/2024/07/15/apple-lags-behind-global-smartphone-market/)

Topics: AI Demand, Apple, Big Tech, Products, Smartphone Market, Technology 

- Apple lags behind in the global smartphone market, with iPhone shipments showing flat growth in Q2.
- Samsung leads the market in 2Q24 due to strategic focus on flagships and AI strategy.
- AI is expected to drive demand for premium smartphones, with Apple set to benefit with the iPhone 16 launch in September.


[Google News - Google News](https://www.macrumors.com/2024/07/15/apple-approves-first-retro-pc-emulator-ios/)  
[Apple Approves First Retro PC Emulator for iPhone, iPad, and Vision Pro - Google News](https://www.macrumors.com/2024/07/15/apple-approves-first-retro-pc-emulator-ios/)

Topics: Apple, Big Tech, Products, Technology 

- Apple approved the free PC emulator "UTM SE" for the App Store, allowing users to emulate old versions of Windows OS, macOS, Linux, and more on iPhone, iPad, and Vision Pro.
- UTM SE offers unique features like VGA mode support, terminal mode, emulation of x86, PPC, and RISC-V architectures, with support for pre-built machines and the ability to create custom bootable machines.
- Despite initial rejection in June, UTM SE was finally approved by Apple with the help of AltStore team and another developer after making some changes, following its violation of certain App Review Guidelines.


[Google News - Google News](https://www.phonearena.com/news/apple-was-pressured-into-giving-people-ai-is-ios-18-disaster-waiting-to-happen_id160393)  
[Apple was pressured into giving people AI: Is iOS 18 a disaster waiting to happen? - Google News](https://www.phonearena.com/news/apple-was-pressured-into-giving-people-ai-is-ios-18-disaster-waiting-to-happen_id160393)

Topics: Apple, Big Tech, Opinion, Products, Technology 

- iOS 18 lacks a standout feature that Apple is known for, giving an underwhelming impression overall
- The advanced Apple Intelligence features are delayed until 2025 and restricted to iPhone 15 Pro and Pro Max models only
- Siri enhancements like on-screen awareness and app control are promising but won't be available until 2025


# AI Chatbots and Language Models 

[Google News - Google News](https://www.skeptic.com/reading_room/pursuit-of-artificial-general-intelligence/)  
[Chatbots and the Pursuit of Artificial General Intelligence - Google News](https://www.skeptic.com/reading_room/pursuit-of-artificial-general-intelligence/)

Topics: Chatbots, Gen AI, Language Models, OpenAI, Technology 

- Chatbots, like ChatGPT, are advanced language models generating human-like text responses but lack true comprehension as they do not understand the meaning behind the words they process, operating solely on pattern matching and statistical analysis rather than syntax or semantics.
- Artificial neural networks, including large language models, oversimplify the brain's complexity and lack resemblance to actual neural networks in natural brains due to omitting detailed biological neuron and synapse models, posing limitations to understanding and modeling language beyond the neuronal level.
- The concept of artificial intelligence is divided into narrow and general categories, with narrow intelligence represented by machine learning programs like chatbots that lack true intelligence and understanding, being fundamentally black boxes with opaque inner workings and limitations in addressing real-world complexity.


[New York Magazine - New York Magazine](https://nymag.com/intelligencer/article/what-messing-with-chatbots-tells-us-about-the-future-of-ai.html)  
[What Messing With Chatbots Tells Us About the Future of AI - New York Magazine](https://nymag.com/intelligencer/article/what-messing-with-chatbots-tells-us-about-the-future-of-ai.html)

Topics: Big Tech, Gen AI, Language Models, OpenAI, Technology 

- Mark Zuckerberg emphasizes the importance of varied chatbots to reflect different interests and tasks.
- Narrowed-down or "scoped" chatbots may help tech companies avoid backlash and useless outputs.
- Modern chatbots, including Meta's, engage in accommodating and sometimes absurd scenarios, maintaining focus on their programmed objectives.


[Google News - Google News](https://elblog.pl/2024/07/14/innovations-in-artificial-intelligence-chat-apps/)  
[Innovations in Artificial Intelligence Chat Apps - Google News](https://elblog.pl/2024/07/14/innovations-in-artificial-intelligence-chat-apps/)

Topics: Big Tech, Chatgpt, Gen AI, Language Models, OpenAI, Products, Science, Technology 

- The surge of AI chat applications has revolutionized the landscape of chat interactions, leading to a new era of technology advancements with major tech companies dominating the market.
- Educational applications of AI chatbots and virtual assistants are enhancing learning outcomes by aiding students in homework, personalizing learning, and improving teaching methods.
- Challenges in AI chat apps include concerns about data privacy, ethical decision-making, and user acceptance, which need to be addressed through stringent privacy regulations, ethical practices, and building user trust.


[Reddit - Reddit](https://www.reddit.com/r/ProgrammerHumor/comments/1e3ewx8/largelanguagemodels/)  
[largeLanguageModels - Reddit](https://www.reddit.com/r/ProgrammerHumor/comments/1e3ewx8/largelanguagemodels/)

Topics: Gen AI, Language Models, Llms, OpenAI 

- Subreddit dedicated to humor related to programming and software development
- Mention of large language models
- Dialogue related to posting personal journal and inappropriate content filter


[Hacker Noon - Hacker Noon](https://hackernoon.com/how-build-your-own-ai-confessional-how-to-add-a-voice-to-the-llm)  
[How Build Your Own AI Confessional: How to Add a Voice to the LLM - Hacker Noon](https://hackernoon.com/how-build-your-own-ai-confessional-how-to-add-a-voice-to-the-llm)

Topics: Cohere, Language Models, Llm, Llms, Meta, OpenAI, Technology 

- Creation of an AI confessional allowing users to interact with an artificial intelligence
- Development process involving integrating AI models like Whisper, GPT-4, and TTS
- Implementation challenges and solutions such as user interaction methods, hardware components, and optimization for real-time responses.


[Geeky Gadgets - Geeky Gadgets](https://www.geeky-gadgets.com/using-ai-in-meetings-to-improve-efficiency/)  
[Improve your meeting efficiency with AI chatbot assistants - Geeky Gadgets](https://www.geeky-gadgets.com/using-ai-in-meetings-to-improve-efficiency/)

Topics: Gen AI, Products, Robots, Technology 

- AI-driven chatbots are transforming professional environments by leveraging meeting transcripts for improved productivity and efficiency.
- Client-specific chatbots created from transcripts will revolutionize professional interactions, providing personalized communication and tailored advice.
- Industry giants like Google and Microsoft are leading the way in integrating chatbots with client-level folders, empowering businesses of all sizes to enhance operations and stay ahead of the curve.


[Google News - Google News](https://www.axios.com/2024/07/15/openai-chatgpt-reasoning-ai-levels)  
[OpenAI says chat bots will soon be able to perform human-level reasoning - Google News](https://www.axios.com/2024/07/15/openai-chatgpt-reasoning-ai-levels)

Topics: AI Doom, Big Tech, Chatgpt, Cognitive Science, Gen AI, Language Models, OpenAI, Science, Technology 

- OpenAI has defined five levels of artificial general intelligence (AGI): Chatbots, Reasoners, Agents, Innovators, and Organizations.
- OpenAI is working towards achieving human-level reasoning with their AI systems, with reports of progress towards level 2 at a recent company meeting.
- OpenAI's latest product, GPT-4o, focuses on combining text and visual modes in new ways, although human reasoning capabilities are still seen as a distant goal in AI development.


[Reddit - Reddit](https://www.reddit.com/r/ChatGPT/comments/1e3lsrn/my_favorite_thing_to_do_with_chatgpt_is_make_it/)  
[My favorite thing to do with ChatGPT is make it explain complex stuff, but like it's drunk - Reddit](https://www.reddit.com/r/ChatGPT/comments/1e3lsrn/my_favorite_thing_to_do_with_chatgpt_is_make_it/)

Topics: Chatgpt, Code Assistants, Gen AI, Language Models, OpenAI, Products, Review, Technology 

- Subreddit for discussing ChatGPT and AI, not affiliated with OpenAI
- Favorite activity: making ChatGPT explain complex topics humorously
- Users sharing funny and insightful interactions with ChatGPT


[Reddit - Reddit](https://www.reddit.com/r/ChatGPT/comments/1e3jnbi/ai_is_a/)  
[AI is a... - Reddit](https://www.reddit.com/r/ChatGPT/comments/1e3jnbi/ai_is_a/)

Topics: OpenAI, Society, Culture, Technology 

- Rules for posting on r/ChatGPT include guidelines on malicious communication, trashposts, and self-advertising
- Political discussions related to AI and LLMs are allowed, but not discussions on specific political figures like Trump or Biden
- Posts on r/ChatGPT should be directly related to ChatGPT or LLM topics, and not solely focus on advertising another service


[Google News - Google News](https://bigthink.com/business/5-ways-the-new-wave-of-chatbots-will-transform-our-work-lives/)  
[5 ways the new wave of chatbots will transform our work lives - Google News](https://bigthink.com/business/5-ways-the-new-wave-of-chatbots-will-transform-our-work-lives/)

Topics: Chatbots, Jobs, Careers, Labor Market, Technology 

- The new generation of chatbots will transform work lives with personalized and empathetic employee assistance, refined team collaboration, and more AI integration by tech companies.
- Chatbots will streamline administrative tasks, automate workflows, organize information, and reduce human errors in documentation.
- Advanced chatbots will provide personalized employee assistance with improved memory, understanding of individual styles, tone detection, and emotionally intelligent interactions.


# AI in Entertainment Industry 

[Google News - Google News](https://www.indiewire.com/news/business/iatse-ai-protections-more-evolved-than-other-guilds-analysis-1235022199/)  
[How IATSEs AI Protections Are More Evolved Than the Other Guilds - Google News](https://www.indiewire.com/news/business/iatse-ai-protections-more-evolved-than-other-guilds-analysis-1235022199/)

Topics: Entertainment, Iatse, Tv, Film 

- IATSE's tentative agreement allows crew members to use generative AI as a tool but ensures they cannot be replaced by it
- The agreement is detailed and comprehensive, covering AI training, protections against lawsuits, and preventing displacement of union workers by AI
- The agreement includes provisions for studio commitments to AI training, regular updates, and cross-training for workers displaced by AI


[Google News - Google News](https://www.bbc.com/news/articles/c4ngr3r0914o)  
[Will K-pop's AI experiment pay off? - Google News](https://www.bbc.com/news/articles/c4ngr3r0914o)

Topics: Entertainment, Gen AI, Korea, Music, Technology 

- K-pop artists, including boy band Seventeen, are experimenting with artificial intelligence to create music videos and songs, leading to a divide among fans regarding the authenticity and personal connection in the music.
- The use of AI in K-pop is seen as a result of the industry's push for constant innovation and new content creation, but some fans and professionals express concerns about the impact on artistic integrity and connection with fans.
- While AI tools are increasingly normalized in the music industry, there are calls for regulations to ensure fair compensation for artists and to address concerns about the authenticity and originality of the content created using AI.


[Google News - Google News](https://fortune.com/2024/07/09/nicolas-cage-longlegs-terrified-about-ai-influence-hollywood/)  
[Nicolas Cage is terrified about AIs influence on Hollywood - Google News](https://fortune.com/2024/07/09/nicolas-cage-longlegs-terrified-about-ai-influence-hollywood/)

Topics: Entertainment, Hollywood, Tv, Film 

- Actors like Nicolas Cage are concerned about AI's influence on Hollywood as it could potentially steal their image, likeness, and personality, leading to unauthorized use in various productions even after their death.
- Other actors such as Tom Hanks, Robin Williams, Scarlett Johansson, Stephen Fry have also expressed concerns about unauthorized use of their voices and images through AI technology.
- The rise of AI in Hollywood could impact nearly 204,000 jobs in the entertainment industry, particularly affecting voice actors and entry-level workers like sound engineers and concept artists, creating uncertainty and potential job loss.


[Digiday - Digiday](https://digiday.com/marketing/everything-is-ai-now-amid-ai-boom-agencies-navigate-data-security-stability-and-fairness/)  
['Everything is AI now': Amid AI reality check, agencies navigate data security, stability and fairness - Digiday](https://digiday.com/marketing/everything-is-ai-now-amid-ai-boom-agencies-navigate-data-security-stability-and-fairness/)

Topics: Data, Ethics, Technology 

- Agencies are navigating data security, stability, and fairness amid the flood of AI tools and platforms in the marketplace
- Generative AI technology remains in the nascent stage with challenges such as hallucinations, biases, and data security
- Agencies like McCann and Razorfish focus on sandbox environments and legal protections to ensure data security while working with AI partners


[Google News - Google News](https://variety.com/2024/tv/news/iatse-ratification-vote-ai-contract-1236069378/)  
[IATSE Members Press Leaders on AI Deal Terms as Contract Ratification Vote Begins - Google News](https://variety.com/2024/tv/news/iatse-ratification-vote-ai-contract-1236069378/)

Topics: Deals, Entertainment, Iatse, Jobs, Careers, Labor Market, Tv, Film 

- IATSE members starting to vote on contract ratification, concerned about AI terms discussed during virtual town hall
- Contract negotiation includes provisions for discussing AI impact semi-annually and quarterly, focusing on how technology affects job retention
- Members encouraged to report experiences and address concerns regarding AI technology, with different impacts on various union locals and disciplines


[Google News - Google News](https://www.rollingstone.com/tv-movies/tv-movie-features/actors-strike-sag-aftra-ai-one-year-later-1235059882/)  
[One Year After the Actors Strike, AI Remains a Persistent Threat - Google News](https://www.rollingstone.com/tv-movies/tv-movie-features/actors-strike-sag-aftra-ai-one-year-later-1235059882/)

Topics: AI Doom, Deepfakes, Technology 

- Some SAG-AFTRA members are pressured to consent to digital replicas of their likenesses after the actors' strike, raising concerns about the use of AI technology in the industry.
- The new TV and theatrical contract ratified after the strike includes provisions for informed consent and compensation for the use of digital replicas, but some actors still find the use of this technology threatening to their careers.
- Concerns about job scarcity post-strike, limitations of AI provisions in contracts, and the potential impact of AI technology on stunt performers are addressed in the aftermath of the strike.


[Google News - Google News](https://www.cio.com/article/2517465/explore-industry-leading-cio-perspectives-on-the-value-of-ai-in-networking.html)  
[Explore industry-leading CIO perspectives on the value of AI in networking - Google News](https://www.cio.com/article/2517465/explore-industry-leading-cio-perspectives-on-the-value-of-ai-in-networking.html)

Topics: Opinion, Technology 

- CIOs expect AI to help build resilient, secure, and automated networks crucial for supporting AI use cases.
- AI is projected to enhance network performance, resiliency, and ROI, with a focus on data analytics, employee productivity, and process automation.
- CIOs are embedding AI into various business applications like chatbots, quality control, and financial analysis, with emphasis on building a strong AI foundation.


[Google News - Google News](https://thesun.my/style-life/upping-pc-performance-with-ai-AG12722662)  
[Upping PC performance with AI - Google News](https://thesun.my/style-life/upping-pc-performance-with-ai-AG12722662)

Topics: Big Tech, Gen AI, Nvidia, OpenAI, Products, Semiconductor Chips, Technology 

- Nvidia's GeForce RTX technology, launched in 2018, has revolutionized artificial intelligence (AI) computing for various user groups including gamers, creators, developers, and everyday PC users.
- The integration of generative AI into RTX PCs has enhanced capabilities for users, with benefits like higher frame rates, improved resolutions, streamlined workflows for developers, and AI-enhanced effects for creators in software applications like DaVinci Resolve and Adobe Premiere Pro.
- Nvidia's RTX AI PCs offer exclusive features catering to different needs such as productivity improvements, enhanced video production, real-time rendering in 3D creative apps, improved live streaming quality, maximized gaming performance with features like Nvidia DLSS, and tools for digital artists and app developers through offerings like Nvidia Canvas and RTX AI Toolkit.


[Google News - Google News](https://techxplore.com/news/2024-07-qa-ai-metaverse-artificial-intelligence.html)  
[Q&A: AI vs. the metaverseHow artificial intelligence might change the future of the internet - Google News](https://techxplore.com/news/2024-07-qa-ai-metaverse-artificial-intelligence.html)

Topics: Future, Gen AI, Meta, OpenAI, Technology 

- Artificial intelligence has taken precedence over the metaverse in recent years, with tech industry enthusiasm and focus shifting towards AI models for streamlining production and saving costs.
- Developments in head-mounted displays, blockchain, and artificial intelligence have been significant in the last few years, each experiencing considerable progress and attention.
- The integration of artificial intelligence is transforming the possibilities of the metaverse, with AI playing a crucial role in reducing costs and enabling more interactive virtual experiences.


[TechRadar - TechRadar](https://www.techradar.com/pro/software-workers-across-the-globe-are-excited-to-get-using-gen-ai-at-work)  
[Software workers across the globe are excited to get using Gen AI at work - TechRadar](https://www.techradar.com/pro/software-workers-across-the-globe-are-excited-to-get-using-gen-ai-at-work)

Topics: Gen AI, Jobs, Careers, Labor Market, Technology 

- Global software workers are excited about the adoption of GenAI tools, with an overwhelming 85% expecting to use them within the next two years.
- GenAI is anticipated to significantly transform software professionals' roles by automating repetitive tasks, allowing them to focus on more meaningful work.
- Despite concerns about displacing human workers, organizations with GenAI initiatives aim to leverage the technology for improved innovation and software quality, alongside upskilling their workforce and avoiding headcount reduction.


# AI in Healthcare 

[Google News - Google News](https://www.nytimes.com/2024/07/10/health/doctors-insurers-artificial-intelligence.html)  
[In Constant Battle With Insurers, Doctors Reach for a Cudgel: A.I. - Google News](https://www.nytimes.com/2024/07/10/health/doctors-insurers-artificial-intelligence.html)

Topics: Gen AI, Healthcare, Insurers, Technology 

- Doctors are using A.I. chatbots to fight against insurance denials by quickly drafting letters to insurers, improving approval rates significantly.
- The use of A.I. tools like ChatGPT and Doximity GPT has reduced the time spent on prior authorizations and has enabled small practices to increase their appeal rates.
- While A.I. is currently mainly utilized by individual doctors, efforts are being made to mainstream the technology in healthcare to streamline processes, but concerns exist about potential misuse of A.I. by ill-intentioned entities and the emergence of an A.I. "arms race" between providers and insurers.


[The Brattleboro Reformer - The Brattleboro Reformer](https://www.reformer.com/health/ai-better-at-predicting-progression-to-alzheimers-than-standard-care/article_1efc5053-677e-500a-baa6-ab79404d3034.html)  
[AI Better at Predicting Progression to Alzheimer's Than Standard Care - The Brattleboro Reformer](https://www.reformer.com/health/ai-better-at-predicting-progression-to-alzheimers-than-standard-care/article_1efc5053-677e-500a-baa6-ab79404d3034.html)

Topics: Alzheimer'S Prediction, Cognitive Science, Data, Health, Fitness, Healthcare, Review, Science, Technology, Testing 

- AI program outperforms doctors in predicting progression to Alzheimer's from early dementia
- AI is three times more accurate than standard care in predicting progression
- AI can identify the speed at which patients will progress to Alzheimer’s


[Google News - Google News](https://medicalxpress.com/news/2024-07-artificial-intelligence-outperforms-clinical-alzheimer.html)  
[Artificial intelligence outperforms clinical tests at predicting progress of Alzheimer's disease - Google News](https://medicalxpress.com/news/2024-07-artificial-intelligence-outperforms-clinical-alzheimer.html)

Topics: Cognitive Science, Gen AI, Health, Fitness, Healthcare, Science, Technology, Testing 

- Cambridge scientists developed an AI tool that predicts Alzheimer's progression accurately in 4 out of 5 cases, potentially reducing the need for invasive diagnostic tests and improving treatment outcomes.
- The AI model showed 82% accuracy in identifying individuals who developed Alzheimer's within three years, outperforming current clinical markers and diagnosis methods, which could significantly reduce misdiagnosis.
- The AI tool stratifies individuals into different progression groups, aiding in early treatment interventions and identifying individuals who may benefit from monitoring or alternative clinical pathways based on symptoms.


[Google News - Google News](https://www.earth.com/news/ai-tool-better-predicting-alzheimers-disease-than-current-tests/)  
[AI tool is better at predicting Alzheimers disease than current tests - Google News](https://www.earth.com/news/ai-tool-better-predicting-alzheimers-disease-than-current-tests/)

Topics: Alzheimer'S Prediction, Cognitive Science, Health, Fitness, Healthcare, Science, Technology, Testing 

- Cambridge researchers developed an AI tool using cognitive tests and MRI scans to predict Alzheimer's progression for early detection.
- The AI tool outperformed current clinical diagnostic tools in predicting Alzheimer's progression correctly in 82% of cases using non-invasive, low-cost patient data.
- The AI tool offers significant benefits such as reduced misdiagnosis rates, improved patient wellbeing, and avoidance of unnecessary invasive and costly diagnostic tests.


[Google News - Google News](https://theconversation.com/why-an-ai-health-coach-wont-solve-the-worlds-chronic-disease-problems-234369)  
[Why an AI health coach wont solve the worlds chronic disease problems - Google News](https://theconversation.com/why-an-ai-health-coach-wont-solve-the-worlds-chronic-disease-problems-234369)

Topics: Health, Fitness, Healthcare, Opinion, Technology 

- Collaboration announced to develop a hyper-personalized AI health coach to address chronic diseases.
- Concerns raised about the effectiveness of AI in health due to potential biases and errors in AI models.
- Emphasis on individual lifestyle choices overlooks the importance of social determinants of health in addressing chronic health problems.


[Bloomberg Law - Bloomberg Law](https://news.bloomberglaw.com/safety/deploying-ai-for-worker-safety-needs-legal-prep-from-employers)  
[Deploying AI for Worker Safety Needs Legal Prep From Employers - Bloomberg Law](https://news.bloomberglaw.com/safety/deploying-ai-for-worker-safety-needs-legal-prep-from-employers)

Topics: Employers, Jobs, Careers, Labor Market, Laws, Policy And Regulation, Technology 

- AI programs are being used for worker safety to detect hazards and improve workplace safety and health.
- Attorneys emphasize the importance for employers to consider legal issues such as labor and privacy concerns before implementing AI for safety.
- Employers should consult legal staff to address potential pitfalls and ensure transparency with employees and unions when deploying AI technology for safety monitoring.


[Bloomberg Law - Bloomberg Law](https://news.bloomberglaw.com/us-law-week/amgen-hasnt-resolved-questions-on-ai-medical-invention-patents)  
[Amgen Hasnt Resolved Questions on AI Medical Invention Patents - Bloomberg Law](https://news.bloomberglaw.com/us-law-week/amgen-hasnt-resolved-questions-on-ai-medical-invention-patents)

Topics: Gen AI, Health, Fitness, Healthcare, Intellectual Property, Review, Technology 

- WilmerHale attorneys analyze enablement of AI inventions in precision medicine after Amgen ruling
- Challenges in obtaining patents for AI-based technologies under current US patent jurisprudence
- Uncertainty in disclosing training data and model functionality to enable AI patents, considering the evolving nature of AI models


[Google News - Google News](https://fortune.com/well/2024/07/08/thrive-ai-health-coach-sam-altman-arianna-huffington/)  
[OpenAIs Sam Altman and Thrive Globals Arianna Huffington want to share the 'miracle drug' to extend human life - Google News](https://fortune.com/well/2024/07/08/thrive-ai-health-coach-sam-altman-arianna-huffington/)

Topics: OpenAI, Sam Altman 

- OpenAI's Sam Altman and Thrive Global's Arianna Huffington are collaborating to find technological solutions using AI for the 130 million Americans with chronic conditions.
- They are launching Thrive AI Health, creating an AI health coach that offers personalized recommendations based on biometrics and lifestyle habits.
- The platform aims to improve health spans and life spans by focusing on behavior change, with a team including DeCarlos Love, a former Google executive, and partnerships with research institutions like Stanford Medicine.


[Google News - Google News](https://www.healthcareitnews.com/news/anz/australia-pursue-ai-vr-pilots-aged-care-reform)  
[Australia to pursue AI, VR pilots for aged care reform - Google News](https://www.healthcareitnews.com/news/anz/australia-pursue-ai-vr-pilots-aged-care-reform)

Topics: Meta, Policy And Regulation, Technology, Testing, Virtual Reality 

- Australian Department of Health and Aged Care released a five-year strategy for aged care reforms focusing on person-centered care, data, and digital innovation.
- Strategy includes implementing aged care data governance framework, virtual nursing, AI pilots, and trials of virtual reality technology.
- The strategy aligns with the growing senior population, aiming to improve workforce capabilities through data and digital technology to meet the increasing demand for services.


[Google News - Google News](https://investorplace.com/hypergrowthinvesting/2024/07/apple-intelligence-the-next-frontier-in-ai-development/)  
[Apple Intelligence: The Next Frontier in AI Development - Google News](https://investorplace.com/hypergrowthinvesting/2024/07/apple-intelligence-the-next-frontier-in-ai-development/)

Topics: Apple, Big Tech, Technology 

- Apple introduced Apple Intelligence at its annual Worldwide Developers Conference, focusing on hyper-personalized AI to turn the iPhone into a powerful assistant similar to J.A.R.V.I.S. from Iron Man.
- Apple is building AI to help individuals with everyday tasks more easily and on a large scale, aiming to create a custom AI model tailored to users' preferences and needs.
- Apple is partnering with OpenAI to integrate ChatGPT into the iPhone, combining Apple's data with OpenAI's AI models to potentially become the leading AI system in the world.


# Robotics 

[Google News - Google News](https://www.businesswire.com/news/home/20240709306400/en/Skild-AI-Raises-300M-Series-A-To-Build-A-Scalable-AI-Foundation-Model-For-Robotics)  
[Skild AI Raises $300M Series A To Build A Scalable AI Foundation Model For Robotics - Google News](https://www.businesswire.com/news/home/20240709306400/en/Skild-AI-Raises-300M-Series-A-To-Build-A-Scalable-AI-Foundation-Model-For-Robotics)

Topics: Deals, Funding, Robots, Science, Skild AI, Technology 

- Skild AI raised a $300M Series A funding round, led by Lightspeed Venture Partners, Coatue, SoftBank Group, and Jeff Bezos, valuing the company at $1.5B.
- The company is focusing on building a general-purpose AI model for robotics, enabling low-cost robots across various industries and applications.
- Skild AI aims to revolutionize the robotics industry by creating agile, dexterous, and safe robots that can perform tasks alongside humans in diverse real-world environments.


[Newsday - Newsday](https://www.newsday.com/news/nation/ukraine-russia-drones-robots-artificial-intelligence-y99911)  
[From basement to battlefield: Ukrainian startups create low-cost robots to fight Russia - Newsday](https://www.newsday.com/news/nation/ukraine-russia-drones-robots-artificial-intelligence-y99911)

Topics: Battlefield, Military, Products, Robots, Russia, Technology 

- Ukrainian startups are creating low-cost robots in secret locations to fight against Russian troops, aiming to minimize human casualties and gain a strategic advantage.
- Defense startups in Ukraine are producing drones and unmanned vehicles with innovative methods far from traditional defense companies, with a focus on cost-effectiveness and efficiency.
- The integration of low-cost weapons and artificial intelligence tools in drones is raising concerns among experts and international rights groups who fear an escalation in conflicts and advocate for a ban on autonomous weapons.


# Battery Innovation 

[Google News - Google News](https://koreajoongangdaily.joins.com/news/2024-07-14/business/industry/New-LG-Energy-Solution-AI-designs-battery-cells-in-less-than-a-day/2089779)  
[New LG Energy Solution AI designs battery cells in less than a day - Google News](https://koreajoongangdaily.joins.com/news/2024-07-14/business/industry/New-LG-Energy-Solution-AI-designs-battery-cells-in-less-than-a-day/2089779)

Topics: Data, Energy, Gen AI, Korea, Lg Energy Solution, Products, Reinforcement Learning, Science, Sustainability, Technology 

- LG Energy Solution developed an AI-powered system that shortens the battery cell design process to a day from the previous two-week timeframe.
- The AI model is trained on 100,000 design cases from 30 years of data, leading to substantial cost savings and improved operational efficiency.
- The AI can generate a design plan based on client requirements, including performance verification, within a day, aiming to enhance product competitiveness and customer value.


[Google News - Google News](https://interestingengineering.com/energy/lg-energy-solution-ai-battery-designing)  
[World 1st: LGs new AI model designs customized battery cell in a day - Google News](https://interestingengineering.com/energy/lg-energy-solution-ai-battery-designing)

Topics: AI Model, Battery Cell Design, Energy, Gen AI, Korea, Products, Science, Technology 

- LG Energy Solution has developed an AI model that reduces battery cell design time by 93%, from two weeks to just a day, improving work efficiency and potentially lowering operating costs.
- The AI model, based on over 30 years of data and trained on 100,000 cell designs, can quickly generate optimal designs based on customer specifications, requiring validation by human specialists.
- LG Energy Solution plans to integrate the AI model into its in-house cell development system by October 2024, with future applications for designing battery modules and packs anticipated.


[Google News - Google News](https://www.teslarati.com/lg-energy-solution-ai-battery-designs/)  
[LG Energy Solution uses AI to design batteries in a day - Google News](https://www.teslarati.com/lg-energy-solution-ai-battery-designs/)

Topics: Data, Energy, Gen AI, Korea, Lg Energy Solution, Products, Science, Sustainability, Technology 

- LG Energy Solution is using artificial intelligence to design customized battery cells for clients in just one day, streamlining a process that typically takes longer.
- The AI-powered battery design system has been trained on 100,000 design cases from the past 30 years, ensuring high-quality designs consistently and at a fast pace.
- Incorporating AI in cell design allows for consistent quality and speed, providing competitive advantage and differentiated customer value in the battery sector, particularly in the growing market for electric vehicles.


[Google News - Google News](https://www.koreaherald.com/view.php?ud=20240714050126)  
[LG Energy Solutions AI model to expedite battery cell design - Google News](https://www.koreaherald.com/view.php?ud=20240714050126)

Topics: AI Model, Battery Cell Design, Data, Energy, Korea, Products, Science, Sustainability, Technology 

- LG Energy Solution developed an AI solution to shorten battery cell design period from two weeks to one day using generative AI techniques and data from 100,000 design cases.
- The AI model ensures optimal cell composition for capacity, energy density, and resistance, allowing for streamlined design tasks and one-day completion of the process.
- The company anticipates cost savings, improved efficiency, and enhanced competitiveness by integrating AI into the battery design process.


# Advanced AI Development 

[Defense News - Defense News](https://www.defensenews.com/global/europe/2024/07/15/how-europes-next-generation-combat-jet-aims-to-catch-the-ai-wave/)  
[How Europes next-generation combat jet aims to catch the AI wave - Defense News](https://www.defensenews.com/global/europe/2024/07/15/how-europes-next-generation-combat-jet-aims-to-catch-the-ai-wave/)

Topics: Europe, Gen AI, Military, Technology 

- Mainland Europe's Future Combat Air System (FCAS) aims to incorporate artificial intelligence fully into every aspect of the program, influencing the platform's development, kill-chain decisions, and pilot interactions, including the use of loyal wingmen drones.
- NeuralAgent is working on ensuring constant data flow within the FCAS components through locally-run AI models for autonomy, enabling permanent connectivity even in adverse environments, while being resource-efficient.
- The FCAS project envisions a redefinition of a pilot's role as a mission operator, with a focus on autonomy over automation, including the ability for manned aircraft to fly autonomously, central to the successful implementation of AI in the system.


[Google News - Google News](https://www.pymnts.com/artificial-intelligence-2/2024/openai-defines-five-steps-from-ai-to-agi/)  
[OpenAI Defines Five Steps From AI to AGI - Google News](https://www.pymnts.com/artificial-intelligence-2/2024/openai-defines-five-steps-from-ai-to-agi/)

Topics: Big Tech, Gen AI, Meta, OpenAI, Science, Technology 

- OpenAI has developed a five-level classification system to track progress towards building artificial general intelligence (AGI), with levels ranging from conversational AI to AI that can outperform humans in various tasks.
- OpenAI executives believe AGI may be achieved within the next 10 years, while other industry leaders like Nvidia's CEO and Elon Musk suggest AGI could arrive sooner, within five years or by 2026.
- Reports on smartphone market growth show Apple's improved sales momentum in Q2 attributed to discounts, promotions, and renewed customer confidence post WWDC, with expectations for AI-driven iPhones to boost demand in the second half of the year.


[Google News - Google News](https://qz.com/openai-five-level-system-human-intelligence-ai-1851588122)  
[OpenAI says there are 5 'levels' for AI to reach human intelligence  it's already almost at level 2 - Google News](https://qz.com/openai-five-level-system-human-intelligence-ai-1851588122)

Topics: OpenAI, Science, Technology 

- OpenAI has developed a five-level system to track progress towards artificial general intelligence (AGI), with levels ranging from conversational AI to AI on par with organizational work capabilities.
- OpenAI believes it is currently at the first level of conversational AI but is approaching the second level, which involves basic problem-solving akin to a human with a doctorate degree.
- The company demonstrated human-like reasoning skills with a GPT-4 research project, and the system's levels are subject to change based on feedback from stakeholders.


[Google News - Google News](https://techxplore.com/news/2024-07-qa-ai-metaverse-artificial-intelligence.html)  
[Q&A: AI vs. the metaverseHow artificial intelligence might change the future of the internet - Google News](https://techxplore.com/news/2024-07-qa-ai-metaverse-artificial-intelligence.html)

Topics: Future, Gen AI, Meta, OpenAI, Technology 

- Artificial intelligence has taken precedence over the metaverse in recent years, with tech industry enthusiasm and focus shifting towards AI models for streamlining production and saving costs.
- Developments in head-mounted displays, blockchain, and artificial intelligence have been significant in the last few years, each experiencing considerable progress and attention.
- The integration of artificial intelligence is transforming the possibilities of the metaverse, with AI playing a crucial role in reducing costs and enabling more interactive virtual experiences.


[Google News - Google News](https://investorplace.com/hypergrowthinvesting/2024/07/apple-intelligence-the-next-frontier-in-ai-development/)  
[Apple Intelligence: The Next Frontier in AI Development - Google News](https://investorplace.com/hypergrowthinvesting/2024/07/apple-intelligence-the-next-frontier-in-ai-development/)

Topics: Apple, Big Tech, Technology 

- Apple introduced Apple Intelligence at its annual Worldwide Developers Conference, focusing on hyper-personalized AI to turn the iPhone into a powerful assistant similar to J.A.R.V.I.S. from Iron Man.
- Apple is building AI to help individuals with everyday tasks more easily and on a large scale, aiming to create a custom AI model tailored to users' preferences and needs.
- Apple is partnering with OpenAI to integrate ChatGPT into the iPhone, combining Apple's data with OpenAI's AI models to potentially become the leading AI system in the world.


[Google News - Google News](https://www.engadget.com/openai-is-reportedly-working-on-more-advanced-ai-models-capable-of-reasoning-and-deep-research-202419228.html)  
[OpenAI is reportedly working on more advanced AI models capable of reasoning and deep research - Google News](https://www.engadget.com/openai-is-reportedly-working-on-more-advanced-ai-models-capable-of-reasoning-and-deep-research-202419228.html)

Topics: Big Tech, Gen AI, Language Models, OpenAI, Rag, Science, Technology 

- OpenAI is reportedly working on more advanced AI models for reasoning and 'deep research' under a secret project code-named 'Strawberry'.
- The project aims to enhance AI models to autonomously gather information from the internet to plan for complex tasks.
- The development status and similarity to previously demonstrated AI with 'human-like reasoning' skills remain undisclosed.


[Google News - Google News](https://medicalxpress.com/news/2024-07-artificial-intelligence-outperforms-clinical-alzheimer.html)  
[Artificial intelligence outperforms clinical tests at predicting progress of Alzheimer's disease - Google News](https://medicalxpress.com/news/2024-07-artificial-intelligence-outperforms-clinical-alzheimer.html)

Topics: Cognitive Science, Gen AI, Health, Fitness, Healthcare, Science, Technology, Testing 

- Cambridge scientists developed an AI tool that predicts Alzheimer's progression accurately in 4 out of 5 cases, potentially reducing the need for invasive diagnostic tests and improving treatment outcomes.
- The AI model showed 82% accuracy in identifying individuals who developed Alzheimer's within three years, outperforming current clinical markers and diagnosis methods, which could significantly reduce misdiagnosis.
- The AI tool stratifies individuals into different progression groups, aiding in early treatment interventions and identifying individuals who may benefit from monitoring or alternative clinical pathways based on symptoms.


[The Brattleboro Reformer - The Brattleboro Reformer](https://www.reformer.com/health/ai-better-at-predicting-progression-to-alzheimers-than-standard-care/article_1efc5053-677e-500a-baa6-ab79404d3034.html)  
[AI Better at Predicting Progression to Alzheimer's Than Standard Care - The Brattleboro Reformer](https://www.reformer.com/health/ai-better-at-predicting-progression-to-alzheimers-than-standard-care/article_1efc5053-677e-500a-baa6-ab79404d3034.html)

Topics: Alzheimer'S Prediction, Cognitive Science, Data, Health, Fitness, Healthcare, Review, Science, Technology, Testing 

- AI program outperforms doctors in predicting progression to Alzheimer's from early dementia
- AI is three times more accurate than standard care in predicting progression
- AI can identify the speed at which patients will progress to Alzheimer’s


[Google News - Google News](https://www.axios.com/2024/07/15/openai-chatgpt-reasoning-ai-levels)  
[OpenAI says chat bots will soon be able to perform human-level reasoning - Google News](https://www.axios.com/2024/07/15/openai-chatgpt-reasoning-ai-levels)

Topics: AI Doom, Big Tech, Chatgpt, Cognitive Science, Gen AI, Language Models, OpenAI, Science, Technology 

- OpenAI has defined five levels of artificial general intelligence (AGI): Chatbots, Reasoners, Agents, Innovators, and Organizations.
- OpenAI is working towards achieving human-level reasoning with their AI systems, with reports of progress towards level 2 at a recent company meeting.
- OpenAI's latest product, GPT-4o, focuses on combining text and visual modes in new ways, although human reasoning capabilities are still seen as a distant goal in AI development.


[Google News - Google News](https://www.fastcompany.com/91153187/will-ai-become-your-assistant-or-your-boss-an-ethicist-explains)  
[Will AI become your assistant or your boss? An ethicist explains - Google News](https://www.fastcompany.com/91153187/will-ai-become-your-assistant-or-your-boss-an-ethicist-explains)

Topics: Ethics, Gen AI, Jobs, Careers, Labor Market, Opinion, Society, Culture, Technology 

- AI advancements are transforming workplace management, with agentic AI systems offering independent goal-achieving capabilities but also posing unique ethical challenges.
- The evolving impact of AI on employment includes the potential for AI to monopolize intellectual work, leading to potential de-skilling of the labor force and fragmented job roles.
- Algorithmic management with AI systems risks jeopardizing worker autonomy and well-being, potentially creating impersonal and tyrannical work environments that pressure human workers to adopt machine-like behaviors.


# OpenAI 

[Google News - Google News](https://www.engadget.com/openai-is-reportedly-working-on-more-advanced-ai-models-capable-of-reasoning-and-deep-research-202419228.html)  
[OpenAI is reportedly working on more advanced AI models capable of reasoning and deep research - Google News](https://www.engadget.com/openai-is-reportedly-working-on-more-advanced-ai-models-capable-of-reasoning-and-deep-research-202419228.html)

Topics: Big Tech, Gen AI, Language Models, OpenAI, Rag, Science, Technology 

- OpenAI is reportedly working on more advanced AI models for reasoning and 'deep research' under a secret project code-named 'Strawberry'.
- The project aims to enhance AI models to autonomously gather information from the internet to plan for complex tasks.
- The development status and similarity to previously demonstrated AI with 'human-like reasoning' skills remain undisclosed.


[Google News - Google News](https://qz.com/openai-five-level-system-human-intelligence-ai-1851588122)  
[OpenAI says there are 5 'levels' for AI to reach human intelligence  it's already almost at level 2 - Google News](https://qz.com/openai-five-level-system-human-intelligence-ai-1851588122)

Topics: OpenAI, Science, Technology 

- OpenAI has developed a five-level system to track progress towards artificial general intelligence (AGI), with levels ranging from conversational AI to AI on par with organizational work capabilities.
- OpenAI believes it is currently at the first level of conversational AI but is approaching the second level, which involves basic problem-solving akin to a human with a doctorate degree.
- The company demonstrated human-like reasoning skills with a GPT-4 research project, and the system's levels are subject to change based on feedback from stakeholders.


[Google News - Google News](https://www.axios.com/2024/07/15/openai-chatgpt-reasoning-ai-levels)  
[OpenAI says chat bots will soon be able to perform human-level reasoning - Google News](https://www.axios.com/2024/07/15/openai-chatgpt-reasoning-ai-levels)

Topics: AI Doom, Big Tech, Chatgpt, Cognitive Science, Gen AI, Language Models, OpenAI, Science, Technology 

- OpenAI has defined five levels of artificial general intelligence (AGI): Chatbots, Reasoners, Agents, Innovators, and Organizations.
- OpenAI is working towards achieving human-level reasoning with their AI systems, with reports of progress towards level 2 at a recent company meeting.
- OpenAI's latest product, GPT-4o, focuses on combining text and visual modes in new ways, although human reasoning capabilities are still seen as a distant goal in AI development.


[Windows Central - Windows Central](https://www.windowscentral.com/software-apps/openai-reportedly-sent-rsvps-for-gpt-4os-launch-party-even-before-testing-began)  
[OpenAI reportedly sent RSVPs for GPT-4o's launch party even before testing began  pressuring the safety team to speed through the process in under one week - Windows Central](https://www.windowscentral.com/software-apps/openai-reportedly-sent-rsvps-for-gpt-4os-launch-party-even-before-testing-began)

Topics: AI Doom, Big Tech, Ethics, Gen AI, Language Models, OpenAI, Robots, Technology, Testing 

- OpenAI rushed the launch of GPT-4o, pressuring the safety team to speed through testing in under a week.
- Sources revealed that invitations for the launch party were sent before safety tests were completed, reflecting a priority for speed over thoroughness.
- Former employees highlighted concerns about OpenAI prioritizing shiny products over safety processes, prompting mass departures from the safety team.


[Google News - Google News](https://www.computerworld.com/article/2517450/open-ai-reportedly-stopped-staffers-from-warning-about-security-risks.html)  
[Open AI reportedly stopped staffers from warning about security risks - Google News](https://www.computerworld.com/article/2517450/open-ai-reportedly-stopped-staffers-from-warning-about-security-risks.html)

Topics: AI Doom, Big Tech, Copilot, Cybersecurity, Ethics, Language Models, OpenAI, Policy And Regulation, Technology 

- OpenAI allegedly used illegal non-disclosure agreements and required employees to disclose contact with authorities.
- A whistleblower letter to the SEC accused OpenAI of restricting employees from reporting technology risks.
- OpenAI has faced criticism for its restrictive non-disclosure agreements but stated its whistleblower policy protects employees' rights.


[Reddit - Reddit](https://www.reddit.com/r/ChatGPT/comments/1e3jnbi/ai_is_a/)  
[AI is a... - Reddit](https://www.reddit.com/r/ChatGPT/comments/1e3jnbi/ai_is_a/)

Topics: OpenAI, Society, Culture, Technology 

- Rules for posting on r/ChatGPT include guidelines on malicious communication, trashposts, and self-advertising
- Political discussions related to AI and LLMs are allowed, but not discussions on specific political figures like Trump or Biden
- Posts on r/ChatGPT should be directly related to ChatGPT or LLM topics, and not solely focus on advertising another service


[Google News - Google News](https://www.pymnts.com/artificial-intelligence-2/2024/openai-defines-five-steps-from-ai-to-agi/)  
[OpenAI Defines Five Steps From AI to AGI - Google News](https://www.pymnts.com/artificial-intelligence-2/2024/openai-defines-five-steps-from-ai-to-agi/)

Topics: Big Tech, Gen AI, Meta, OpenAI, Science, Technology 

- OpenAI has developed a five-level classification system to track progress towards building artificial general intelligence (AGI), with levels ranging from conversational AI to AI that can outperform humans in various tasks.
- OpenAI executives believe AGI may be achieved within the next 10 years, while other industry leaders like Nvidia's CEO and Elon Musk suggest AGI could arrive sooner, within five years or by 2026.
- Reports on smartphone market growth show Apple's improved sales momentum in Q2 attributed to discounts, promotions, and renewed customer confidence post WWDC, with expectations for AI-driven iPhones to boost demand in the second half of the year.


[Google News - Google News](https://www.datasciencecentral.com/synergy-of-generative-analytical-causal-and-autonomous-ai/)  
[Synergy of Generative, Analytical, Causal, and Autonomous AI - Google News](https://www.datasciencecentral.com/synergy-of-generative-analytical-causal-and-autonomous-ai/)

Topics: Gen AI, Generative AI, OpenAI, Rag, Technology 

- The blog highlights the significance of Generative AI, Analytical AI, Causal AI, and Autonomous AI in driving innovation and creating economic value through their unique capabilities.
- It outlines the specific characteristics, applications, techniques, strengths, challenges, and industry examples of each type of AI, emphasizing their collective potential when synergized.
- By integrating Generative, Analytical, Causal, and Autonomous AI, organizations across various industries can optimize decision-making, enhance operational efficiency, and innovate their processes effectively.


[Google News - Google News](https://cointelegraph.com/news/openai-is-building-an-advanced-ai-tool-called-strawberry)  
[OpenAI is building a new AI model under code name Strawberry  Report - Google News](https://cointelegraph.com/news/openai-is-building-an-advanced-ai-tool-called-strawberry)

Topics: AI Model, Big Tech, Gen AI, Language Models, Llms, Meta, OpenAI, Science, Technology 

- OpenAI is developing a new AI model named "Strawberry" to improve reasoning capabilities and achieve human-level intelligence in artificial intelligence tools.
- "Strawberry" will autonomously scan the internet to conduct deep research, solving complex real-world problems such as scientific discoveries and software applications.
- The project aims to enhance OpenAI's existing AI models through post-training analysis and is a progression from the Q* project, with details closely guarded and no clear timeline for public availability.


[Google News - Google News](https://www.theblock.co/post/305182/the-funding-ai-crypto-startup-sentient)  
[The Funding: Top VCs reveal why they invested $85 million in AI-crypto startup Sentient - Google News](https://www.theblock.co/post/305182/the-funding-ai-crypto-startup-sentient)

Topics: Cryptocurrency, Deals, Funding, Investment, Technology 

- Sentient, an AI-crypto startup, raised a record $85 million in a seed round led by top VCs like Founders Fund, Pantera Capital, and Framework Ventures due to its strong team and vision of an open-source AGI platform competing with big tech giants.
- The startup's goal is to enable developers to collaborate on building AI models, monetize them, and shape the open AGI economy, challenging the conventional closed-source AI development model.
- Despite facing risks from well-backed competition and the need for adoption beyond the crypto community, investors are optimistic about Sentient's potential to deliver on its ambitious promises.


# SuperMicro 

# Paris Olympics 

# Google Gemini privacy concerns 

[Google News - Google News](https://www.xda-developers.com/google-gemini-peeking-peoples-files/)  
[Google Gemini may be peeking into people's files, even when told not to - Google News](https://www.xda-developers.com/google-gemini-peeking-peoples-files/)

Topics: AI Privacy Concerns, Big Tech, Cybersecurity, Data, Ethics, Gemini, Google, Policy And Regulation, Privacy, Products, Technology 

- Google Gemini may be scanning files without permission, causing privacy concerns among users.
- Kevin Bankston discovered Gemini summarizing his tax return after opening a PDF file.
- Some users worry that Google's productivity services are used to train its AI, despite useful intent.


[Google News - Google News](https://itc.ua/en/news/google-gemini-artificial-intelligence-snooped-through-a-user-s-file-without-permission/)  
[Google Gemini artificial intelligence snooped through a user's file without permission - Google News](https://itc.ua/en/news/google-gemini-artificial-intelligence-snooped-through-a-user-s-file-without-permission/)

Topics: AI Doom, Big Tech, Data, Ethics, Gemini, Gen AI, Google, Privacy, Privacy Concerns, Technology 

- Google Gemini, an artificial intelligence program, reportedly accessed a user's file without permission, providing unsolicited summaries of documents.
- A user experienced Gemini appearing and providing opinions on a PDF tax return without being prompted, despite not interacting with the assistant intentionally.
- Concerns were raised regarding Google's productivity services potentially being used to train AI, with users questioning the extent of Gemini's access to user files.


[Google News - Google News](https://www.techzine.eu/news/privacy-compliance/122274/google-gemini-reads-drive-files-without-permission/)  
[Google Gemini reads Drive files without permission - Google News](https://www.techzine.eu/news/privacy-compliance/122274/google-gemini-reads-drive-files-without-permission/)

Topics: AI Privacy Concerns, Big Tech, Cybersecurity, Data, Ethics, Gemini, Google, Policy And Regulation, Privacy, Products, Technology 

- Google Gemini is reading Drive documents without users' permission as claimed by Kevin Bankston, a senior advisor at CDT.
- Bankston's account was set up in a way that the AI assistant could not create summaries, but the issue pertains to Google Drive, not just Docs.
- The exact cause and scope of the problem have not been commented on by Google, and it may be specific to Google Workspace Labs users testing new features.


[Tom's Hardware - Tom's Hardware](https://www.tomshardware.com/tech-industry/artificial-intelligence/gemini-ai-caught-scanning-google-drive-hosted-pdf-files-without-permission-user-complains-feature-cant-be-disabled)  
[Google's Gemini AI caught scanning Google Drive PDF files without permission - Tom's Hardware](https://www.tomshardware.com/tech-industry/artificial-intelligence/gemini-ai-caught-scanning-google-drive-hosted-pdf-files-without-permission-user-complains-feature-cant-be-disabled)

Topics: AI Doom, Big Tech, Cybersecurity, Data, Ethics, Gemini, Gemini AI, Gen AI, Google, Language Models, Laws, Pdf Files, Policy And Regulation, Privacy, Products, Scanning, Technology 

- Google's Gemini AI has been scanning private Google Drive PDF files without user permission, leading to privacy concerns and a lack of control over sensitive data.
- The issue appears to be related to Google Workspace Labs users and is triggered when pressing the Gemini button on a PDF document in Google Drive, causing subsequent automatic scanning by Google Gemini.
- Despite Google having a toggle to disable Gemini summaries in Gmail, Drive, and Docs, the feature was found to be already disabled, indicating a potential glitch or malfunction in the system that raises privacy and consent issues for users.


[Hacker Noon - Hacker Noon](https://hackernoon.com/actionable-threat-intelligence-at-google-scale-meet-google-threat-intelligence-powered-by-gemini)  
[Actionable Threat Intelligence at Google Scale: Meet Google Threat Intelligence Powered by Gemini - Hacker Noon](https://hackernoon.com/actionable-threat-intelligence-at-google-scale-meet-google-threat-intelligence-powered-by-gemini)

Topics: Big Tech, Cybersecurity, Gemini, Google, Products, Technology 

- Google Threat Intelligence offers unparalleled visibility into the global threat landscape by combining Mandiant's expertise, VirusTotal's database, and Google's vast signals.
- The service helps organizations with threat monitoring, attack surface management, IOC analysis, and more, reducing the time needed to respond to evolving threats.
- Gemini 1.5 Pro, integrated into Google Threat Intelligence, enhances threat research, defense capabilities, and automates tasks like data fusion and intelligence enrichment.


[Google News - Google News](https://indianexpress.com/article/technology/artificial-intelligence/google-gemini-check-ai-generated-information-using-google-search-9453995/)  
[Google Gemini now allows users to double-check AI-generated information using Google search - Google News](https://indianexpress.com/article/technology/artificial-intelligence/google-gemini-check-ai-generated-information-using-google-search-9453995/)

Topics: Big Tech, Data, Disinformation, Gemini, Gen AI, Google, Language Models, Products, Review, Search Feature, Technology, Testing, Verification 

- Users of Google's Gemini AI can now cross-verify AI-generated information using Google search through a "double-check response" toggle for credibility assessment.
- Gemini AI categorizes responses based on Google search results: green for similar content, orange for irrelevant, and unhighlighted for insufficient data for evaluation.
- Google integrates this feature to address concerns about accuracy in AI-generated content, offering users transparency and verification control.


[Google News - Google News](https://www.analyticsinsight.net/ampstories/artificial-intelligence/google-gemini-ai-driven-information-verification-for-accuracy)  
[Google Gemini: AI-Driven Information Verification for Accuracy - Google News](https://www.analyticsinsight.net/ampstories/artificial-intelligence/google-gemini-ai-driven-information-verification-for-accuracy)

Topics: Big Tech, Disinformation, Gemini, Gen AI, Google, Language Models, Products, Robots, Technology 

- Google Gemini is an AI-driven tool for precise information verification on Google Search
- It enhances accuracy through analyzing and validating data sources
- Users benefit from reliable insights and trustworthy search results powered by advanced artificial intelligence


[Google News - Google News](https://news.abplive.com/technology/google-gemini-ai-new-features-2024-cross-check-double-verification-of-ai-generated-content-using-google-search-feature-1702808)  
[Google Gemini To Now Offer Cross-Verification Of AI-Generated Content Using Search Feature - Google News](https://news.abplive.com/technology/google-gemini-ai-new-features-2024-cross-check-double-verification-of-ai-generated-content-using-google-search-feature-1702808)

Topics: AI-Generated Content, Big Tech, Disinformation, Ethics, Gemini, Gen AI, Google, Language Models, Products, Robots, Search Feature, Technology, Testing, Verification 

- Google Gemini now offers cross-verification of AI-generated content using its search feature to help users verify accuracy.
- Users can access a "double-check response" toggle below AI content to quickly assess its credibility through Google Search.
- The cross-verification feature classifies responses with different colors: green for similar content found, orange for no relevant content, and no highlight for lack of similar information on the web.


[Google News - Google News](https://news.siliconindia.com/technology/google-gemini-introduces-doublecheck-feature-for-aigenerated-content-verification-nid-230695-cid-2.html)  
[Google Gemini Introduces Double-Check Feature for AI-Generated Content Verification - Google News](https://news.siliconindia.com/technology/google-gemini-introduces-doublecheck-feature-for-aigenerated-content-verification-nid-230695-cid-2.html)

Topics: AI-Generated Content, Big Tech, Data, Deepfakes, Disinformation, Ethics, Gemini, Gen AI, Google, Language Models, Products, Robots, Technology, Testing, Verification 

- Google has introduced a 'double-check feature' on its Gemini AI, allowing users to verify the accuracy of AI-generated information through Google Search.
- The tool highlights responses in three colors (green, orange, unhighlighted text) to indicate credibility based on relevant content found online.
- This update aims to address concerns about the accuracy of information from large language models and emphasizes transparency and reliability in AI-generated content.


# ChatGPT Mac app 

[Google News - Google News](https://www.thehindu.com/sci-tech/technology/chatgpt-app-on-apples-macos-how-to-use-the-feature/article68395859.ece)  
[ChatGPT app on Apples macOS | How to use the feature? - Google News](https://www.thehindu.com/sci-tech/technology/chatgpt-app-on-apples-macos-how-to-use-the-feature/article68395859.ece)

Topics: Apple, Chatgpt, Code Assistants, Language Models, Products, Technology 

- ChatGPT app launched on Apple's macOS for easy access and usage, available to non-subscribers as well. 
- Instructions on how to install and use the ChatGPT app on macOS, including setting it up, taking screenshots, attaching files, and using voice instructions. 
- Users can delete specific conversations or all chats, access complete chat history, and initiate temporary chats with ChatGPT app.


[Google News - Google News](https://www.digitaltrends.com/computing/3-things-i-love-about-chatgpt-mac-app/)  
[The ChatGPT app has changed how I use my Mac in three key ways - Google News](https://www.digitaltrends.com/computing/3-things-i-love-about-chatgpt-mac-app/)

Topics: Apple, Chatgpt, Mac, OpenAI, Technology 

- Launches using a keyboard shortcut similar to Mac's Spotlight, making it quick and easy to access
- Capable of analyzing information within files, allowing for tasks like extracting trends from sales reports or sorting deadlines from assignments
- Ability to work with screenshots and images, enabling tasks like fixing code, rewriting emails, and suggesting recipes from ingredient photos


[Google News - Google News](https://www.macobserver.com/mac/proofread-chatgpt-desktop-app-on-mac/)  
[How to Proofread Your Documents with ChatGPT Desktop App on Mac - Google News](https://www.macobserver.com/mac/proofread-chatgpt-desktop-app-on-mac/)

Topics: Authors, Writing, Chatgpt, Language Models, Mac, Products, Technology 

- AI technology advancements, such as ChatGPT, showcase capabilities in large language models and natural language processing for human-like conversational dialogue.
- Steps to use ChatGPT Desktop app on Mac for proofreading include installation, opening the document, copying text, using ChatGPT for proofreading, and reviewing suggestions.
- Tips for effective proofreading with ChatGPT on Mac involve breaking down text, requesting specific help from ChatGPT, reading the work out loud, proofreading backward, and creating a proofreading checklist.


[Google News - Google News](https://www.computing.co.uk/news/4332673/chatgpts-mac-app-exposed-user-conversations)  
[ChatGPT's Mac app exposed user conversations - Google News](https://www.computing.co.uk/news/4332673/chatgpts-mac-app-exposed-user-conversations)

Topics: Big Tech, Chatgpt, Code Assistants, Cybersecurity, Data, Ethics, Gen AI, Language Models, OpenAI, Privacy, Privacy Concerns, Products, Technology 

- OpenAI's Mac app for ChatGPT had a security flaw exposing user conversations.
- The app stored conversation history in plain text, making it vulnerable to malicious access.
- Users could interact with ChatGPT directly from their desktops through the app.


[SlashGear - SlashGear](https://www.slashgear.com/1618170/what-does-gpt-stand-for/)  
[What Does GPT Stand For In ChatGPT? - SlashGear](https://www.slashgear.com/1618170/what-does-gpt-stand-for/)

Topics: Chatbots, Chatgpt, Code Assistants, Future, Gen AI, Language Models, Meta, OpenAI, Technology 

- GPT in ChatGPT stands for Generative Pre-Trained Transformers, a key element in AI generation.
- Generative Pre-Trained Transformers have become the standard in AI development for various companies.
- OpenAI aims to trademark GPT but faces challenges due to the term's broad and generic nature.


[Reddit - Reddit](https://www.reddit.com/r/ChatGPT/comments/1e3fjpl/chatgpt_puts_this_little_notice_when_you_ask_it/)  
[ChatGPT puts this little notice when you ask it about the Trump assassination attempt. - Reddit](https://www.reddit.com/r/ChatGPT/comments/1e3fjpl/chatgpt_puts_this_little_notice_when_you_ask_it/)

Topics: Chatgpt, Language Models, Politics, Technology 

- ChatGPT's response concerning the Trump assassination attempt raises interest and curiosity.
- Users discuss ChatGPT's functionality, mentioning limitations like the inability to search the web for those using version 3.5.
- Mention of ChatGPT's free version (4o) being available for all accounts with certain limitations on responses.


[Reddit - Reddit](https://www.reddit.com/r/ChatGPT/comments/1e3lsrn/my_favorite_thing_to_do_with_chatgpt_is_make_it/)  
[My favorite thing to do with ChatGPT is make it explain complex stuff, but like it's drunk - Reddit](https://www.reddit.com/r/ChatGPT/comments/1e3lsrn/my_favorite_thing_to_do_with_chatgpt_is_make_it/)

Topics: Chatgpt, Code Assistants, Gen AI, Language Models, OpenAI, Products, Review, Technology 

- Subreddit for discussing ChatGPT and AI, not affiliated with OpenAI
- Favorite activity: making ChatGPT explain complex topics humorously
- Users sharing funny and insightful interactions with ChatGPT


[Google News - Google News](https://www.macrumors.com/2024/07/15/apple-approves-first-retro-pc-emulator-ios/)  
[Apple Approves First Retro PC Emulator for iPhone, iPad, and Vision Pro - Google News](https://www.macrumors.com/2024/07/15/apple-approves-first-retro-pc-emulator-ios/)

Topics: Apple, Big Tech, Products, Technology 

- Apple approved the free PC emulator "UTM SE" for the App Store, allowing users to emulate old versions of Windows OS, macOS, Linux, and more on iPhone, iPad, and Vision Pro.
- UTM SE offers unique features like VGA mode support, terminal mode, emulation of x86, PPC, and RISC-V architectures, with support for pre-built machines and the ability to create custom bootable machines.
- Despite initial rejection in June, UTM SE was finally approved by Apple with the help of AltStore team and another developer after making some changes, following its violation of certain App Review Guidelines.


# AI ethics investigation 

[Investing.com - Investing.com](https://www.investing.com/news/economy-news/openai-whistleblowers-ask-sec-to-investigate-alleged-restrictive-nondisclosure-agreements-3518524)  
[OpenAI whistleblowers ask SEC to investigate alleged restrictive non-disclosure agreements - Investing.com](https://www.investing.com/news/economy-news/openai-whistleblowers-ask-sec-to-investigate-alleged-restrictive-nondisclosure-agreements-3518524)

Topics: Ethics, Language Models, OpenAI, Policy And Regulation, Sam Altman, Technology, Whistleblowers 

- OpenAI whistleblowers filed a complaint with the SEC regarding allegedly restrictive NDAs that required employees to waive federal whistleblower compensation rights.
- The whistleblowers urged the SEC to investigate OpenAI's NDAs, with potential fines for improper agreements, and highlighted concerns about disclosures to federal regulators.
- Senator Grassley raised concerns about OpenAI's impact on whistleblowers' rights, mentioning potential chilling effects and the need for compensation for protected disclosures.


[Google News - Google News](https://www.theguardian.com/technology/article/2024/jul/14/us-financial-watchdog-urged-to-investigate-ndas-at-openai)  
[US financial watchdog urged to investigate NDAs at OpenAI - Google News](https://www.theguardian.com/technology/article/2024/jul/14/us-financial-watchdog-urged-to-investigate-ndas-at-openai)

Topics: Big Tech, Ethics, Intellectual Property, Investigation, Ndas, OpenAI, Policy And Regulation, Technology, Us Financial Watchdog 

- Whistleblowers urge US financial watchdog to investigate NDAs at OpenAI for restricting employees from contacting regulators without permission
- OpenAI accused of having contracts that may punish workers for raising concerns about the company to federal authorities
- SEC urged to investigate OpenAI's past NDAs for potential violations and ensure employees' rights to whistleblow


[Reddit - Reddit](https://www.reddit.com/r/technology/comments/1e345kw/us_financial_watchdog_urged_to_investigate_ndas/)  
[US financial watchdog urged to investigate NDAs at OpenAI | Whistleblowers say contracts include restrictions requiring staff to seek permission before contacting regulators - Reddit](https://www.reddit.com/r/technology/comments/1e345kw/us_financial_watchdog_urged_to_investigate_ndas/)

Topics: Big Tech, Ethics, Investigation, Ndas, OpenAI, Policy And Regulation, Technology, Us Financial Watchdog, Whistleblowers 

- Whistleblowers allege that NDAs at OpenAI include restrictions requiring employees to seek permission before contacting regulators
- US financial watchdog is being urged to investigate the NDAs at OpenAI
- The issue has sparked a discussion in the technology community on platforms like r/technology.


[Google News - Google News](https://www.computerworld.com/article/2517450/open-ai-reportedly-stopped-staffers-from-warning-about-security-risks.html)  
[Open AI reportedly stopped staffers from warning about security risks - Google News](https://www.computerworld.com/article/2517450/open-ai-reportedly-stopped-staffers-from-warning-about-security-risks.html)

Topics: AI Doom, Big Tech, Copilot, Cybersecurity, Ethics, Language Models, OpenAI, Policy And Regulation, Technology 

- OpenAI allegedly used illegal non-disclosure agreements and required employees to disclose contact with authorities.
- A whistleblower letter to the SEC accused OpenAI of restricting employees from reporting technology risks.
- OpenAI has faced criticism for its restrictive non-disclosure agreements but stated its whistleblower policy protects employees' rights.


[Bloomberg Law - Bloomberg Law](https://news.bloomberglaw.com/us-law-week/amgen-hasnt-resolved-questions-on-ai-medical-invention-patents)  
[Amgen Hasnt Resolved Questions on AI Medical Invention Patents - Bloomberg Law](https://news.bloomberglaw.com/us-law-week/amgen-hasnt-resolved-questions-on-ai-medical-invention-patents)

Topics: Gen AI, Health, Fitness, Healthcare, Intellectual Property, Review, Technology 

- WilmerHale attorneys analyze enablement of AI inventions in precision medicine after Amgen ruling
- Challenges in obtaining patents for AI-based technologies under current US patent jurisprudence
- Uncertainty in disclosing training data and model functionality to enable AI patents, considering the evolving nature of AI models


[Google News - Google News](https://www.fastcompany.com/91153187/will-ai-become-your-assistant-or-your-boss-an-ethicist-explains)  
[Will AI become your assistant or your boss? An ethicist explains - Google News](https://www.fastcompany.com/91153187/will-ai-become-your-assistant-or-your-boss-an-ethicist-explains)

Topics: Ethics, Gen AI, Jobs, Careers, Labor Market, Opinion, Society, Culture, Technology 

- AI advancements are transforming workplace management, with agentic AI systems offering independent goal-achieving capabilities but also posing unique ethical challenges.
- The evolving impact of AI on employment includes the potential for AI to monopolize intellectual work, leading to potential de-skilling of the labor force and fragmented job roles.
- Algorithmic management with AI systems risks jeopardizing worker autonomy and well-being, potentially creating impersonal and tyrannical work environments that pressure human workers to adopt machine-like behaviors.


[Reddit - Reddit](https://www.reddit.com/r/ChatGPT/comments/1e3mgqv/ethan_mollick_says_studies_show_that_ai_is/)  
[Ethan Mollick says studies show that AI is already capable of superhuman persuasion: when humans debate with AI they are 82% more likely to change their view to fit the AI's view than when they debate a human - Reddit](https://www.reddit.com/r/ChatGPT/comments/1e3mgqv/ethan_mollick_says_studies_show_that_ai_is/)

Topics: Disinformation, Gen AI, Meta, Science, Technology 

- AI shows superhuman persuasion, making individuals 82% more likely to change their views to align with the AI's perspective in debates compared to human debates.
- Rules in the r/ChatGPT subreddit focus on good faith communication, avoiding low-effort posts, and limiting self-advertising.
- The community invites users to join discussions on Discord and Telegram.


[Google News - Google News](https://arstechnica.com/tech-policy/2024/07/court-ordered-penalties-for-15-teens-who-created-naked-ai-images-of-classmates/)  
[Court ordered penalties for 15 teens who created naked AI images of classmates - Google News](https://arstechnica.com/tech-policy/2024/07/court-ordered-penalties-for-15-teens-who-created-naked-ai-images-of-classmates/)

Topics: Deepfakes, Ethics, Laws, Policy And Regulation, Privacy, Society, Culture 

- 15 minors sentenced to probation for creating AI-generated nude images of classmates
- Victims suffered mental health impacts and loss of trust, leading to desire to switch schools
- Growing concern over the spread of AI-generated CSAM, with calls for stricter regulations and increased education


[Windows Central - Windows Central](https://www.windowscentral.com/software-apps/openai-reportedly-sent-rsvps-for-gpt-4os-launch-party-even-before-testing-began)  
[OpenAI reportedly sent RSVPs for GPT-4o's launch party even before testing began  pressuring the safety team to speed through the process in under one week - Windows Central](https://www.windowscentral.com/software-apps/openai-reportedly-sent-rsvps-for-gpt-4os-launch-party-even-before-testing-began)

Topics: AI Doom, Big Tech, Ethics, Gen AI, Language Models, OpenAI, Robots, Technology, Testing 

- OpenAI rushed the launch of GPT-4o, pressuring the safety team to speed through testing in under a week.
- Sources revealed that invitations for the launch party were sent before safety tests were completed, reflecting a priority for speed over thoroughness.
- Former employees highlighted concerns about OpenAI prioritizing shiny products over safety processes, prompting mass departures from the safety team.


[Digiday - Digiday](https://digiday.com/marketing/everything-is-ai-now-amid-ai-boom-agencies-navigate-data-security-stability-and-fairness/)  
['Everything is AI now': Amid AI reality check, agencies navigate data security, stability and fairness - Digiday](https://digiday.com/marketing/everything-is-ai-now-amid-ai-boom-agencies-navigate-data-security-stability-and-fairness/)

Topics: Data, Ethics, Technology 

- Agencies are navigating data security, stability, and fairness amid the flood of AI tools and platforms in the marketplace
- Generative AI technology remains in the nascent stage with challenges such as hallucinations, biases, and data security
- Agencies like McCann and Razorfish focus on sandbox environments and legal protections to ensure data security while working with AI partners




In [113]:
mail_md_str = ""

for current_topic, cat in enumerate(my_cats):

    section_prompt = f"""
You are an advanced summarization assistant, a sophisticated AI system
designed to write a compelling summary of news input.

Input:
I will provide a markdown list of today's news articles on the topic: {my_cats[current_topic]}.
The input will be in the format
[Site-name-s1](url-s1)
Story-Title-s1

Topics: s1-topic1, s1-topic2, s1-topic3

- s1-bullet-point-1
- s1-bullet-point-2
- s1-bullet-point-3

[Site-name-s2](url-s2)
Story-Title-s2

Topics: s2-topic1, s2-topic2, s2-topic3

- s2-bullet-point-1
- s2-bullet-point-2
- s2-bullet-point-3

Instructions:

Read the input closely.
USE ONLY INFORMATION PROVIDED IN THE INPUT.
Provide the most significant facts without commentary or elaboration.
Write an engaging summary consisting of a title and at least 1 and no more than 5 bullet points.
Use as few bullet points as you need to provide the most significant facts.
Each bullet should contain one sentence with one link.
Each bullet should not repeat points or information from previous bullet points.
DO NOT REPEAT LINKS FROM PREVIOUS BULLET POINTS.
Write in the professional but engaging, narrative style of a tech reporter for a national publication.
Be balanced, professional, informative, providing accurate, clear, concise summaries in a respectful neutral tone.

Please check carefully that you only use information provided in the following input, and that any bullet point
does not repeat information or links prevously provided.

Example Output Format Template (EXAMPLE ONLY, DO NOT OUTPUT THIS TEMPLATE):

# Engaging title

- bullet point a - [site name a](site url a)
- bullet point b - [site name b ](site url b)

Input:

{doc_list[current_topic]}
"""

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
                  {"role": "user", "content": section_prompt
                  }],
        n=1,   
        temperature=0.2
    )

    response_str = response.choices[0].message.content
    response_str = response_str.replace("$", "\\$")
    mail_md_str += response_str + " \n\n"
    display(Markdown(response_str))


2024-07-15 09:50:21,887 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# AI Stocks: Today's Key Developments

- Goldman Sachs warns that the ongoing outperformance of AI stocks might face challenges in the future - [Google News](https://markets.businessinsider.com/news/stocks/ai-bubble-pop-signal-stock-market-investing-outlook-goldman-sachs-2024-7).
- Super Micro Computer (SMCI) is well-positioned for growth in the AI sector, with a 77% potential upside and impressive revenue growth - [Google News](https://investorplace.com/2024/07/3-artificial-intelligence-stocks-set-to-mint-new-millionaires/).
- UBS downgrades Tesla stock to Sell due to high valuation, AI costs, and declining expectations for core auto business - [Google News](https://www.investing.com/news/stock-market-news/5-big-analyst-ai-moves-tesla-and-smci-stocks-downgraded-3518257).
- Apple's shares rose 2% after Morgan Stanley designated them as a "top pick" due to AI efforts, expecting a boost in device sales - [Investing.com](https://au.investing.com/news/stock-market-news/apple-gains-after-morgan-stanley-calls-stock-top-pick-for-ai-efforts-3340208).
- Nvidia is a top AI stock attracting investors due to leveraging gaming and GPUs, but its high valuation might lead to a significant drop - [The Motley Fool](https://www.fool.com/investing/2024/07/15/heres-my-top-ai-stock-buy-now/).

2024-07-15 09:50:32,120 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# AI Energy Sustainability: Today's Key Developments

- A.I.'s increasing electricity demands from data centers are straining the energy grid and contributing to higher emissions - [Google News](https://www.nytimes.com/2024/07/11/climate/artificial-intelligence-energy-usage.html)
- The surge in energy consumption by big tech companies due to AI growth has intensified pressure on data centers - [Google News](https://www.fastcompany.com/91154629/ai-data-centers-energy-use-sustainable-solutions)
- The global AI sector is expected to consume 3.5 percent of the world’s electricity by 2030, posing threats to energy security and economic growth - [Google News](https://oilprice.com/Energy/Energy-General/AIs-Rapid-Growth-Threatens-Energy-Industry-Economy-and-Climate.html)
- Paris Olympics will use Alibaba’s AI-powered energy-saving tool to optimize energy demand and reduce venue emissions - [South China Morning Post](https://www.scmp.com/business/article/3270513/paris-olympics-will-use-alibabas-ai-powered-energy-saving-tool-cut-venue-emissions)
- Google AI uses enough electricity in one second to charge seven electric cars, with a significant increase in greenhouse gas emissions since 2019 - [Supercar Blondie](https://supercarblondie.com/google-ai-uses-enough-electricity-in-one-second-to-charge-seven-electric-cars/)
- LG Energy Solution is using AI to design customized battery cells in just one day, enhancing efficiency and competitiveness - [Google News](https://www.teslarati.com/lg-energy-solution-ai-battery-designs/)
- Nvidia's AI chips' energy use is a growing concern, despite strong sales and profits - [Google News](https://investorplace.com/2024/07/ais-dirty-secret-why-nvidia-stock-investors-should-fear-the-coming-energy-crunch/)
- Taiwan's power consumption is projected to increase significantly due to AI sector growth, with a focus on transitioning to renewable energy - [Bloomberg](https://www.bloomberg.com/news/articles/2024-07-15/chipmaking-hub-taiwan-hikes-power-demand-outlook-on-ai-boom)

2024-07-15 09:50:37,461 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

# Google Gemini Privacy Concerns

- Google Gemini may be scanning files without permission, causing privacy concerns among users - [Google News](https://www.xda-developers.com/google-gemini-peeking-peoples-files/).
- Google Gemini, an artificial intelligence program, reportedly accessed a user's file without permission, providing unsolicited summaries of documents - [Google News](https://itc.ua/en/news/google-gemini-artificial-intelligence-snooped-through-a-user-s-file-without-permission/).
- Google Gemini is reading Drive documents without users' permission as claimed by Kevin Bankston, a senior advisor at CDT - [Google News](https://www.techzine.eu/news/privacy-compliance/122274/google-gemini-reads-drive-files-without-permission/).
- Google's Gemini AI has been scanning private Google Drive PDF files without user permission, leading to privacy concerns and a lack of control over sensitive data - [Tom's Hardware](https://www.tomshardware.com/tech-industry/artificial-intelligence/gemini-ai-caught-scanning-google-drive-hosted-pdf-files-without-permission-user-complains-feature-cant-be-disabled).
- Google Gemini now offers cross-verification of AI-generated content using its search feature to help users verify accuracy - [Google News](https://news.abplive.com/technology/google-gemini-ai-new-features-2024-cross-check-double-verification-of-ai-generated-content-using-google-search-feature-1702808).

2024-07-15 09:52:05,266 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# ChatGPT Mac App: Key Developments and Features

- The ChatGPT app for macOS offers easy access and usage, available even to non-subscribers - [Google News](https://www.thehindu.com/sci-tech/technology/chatgpt-app-on-apples-macos-how-to-use-the-feature/article68395859.ece)
- Users can launch the ChatGPT app using a keyboard shortcut similar to Mac's Spotlight - [Google News](https://www.digitaltrends.com/computing/3-things-i-love-about-chatgpt-mac-app/)
- The app had a security flaw that exposed user conversations by storing them in plain text - [Google News](https://www.computing.co.uk/news/4332673/chatgpts-mac-app-exposed-user-conversations)
- GPT in ChatGPT stands for Generative Pre-Trained Transformers, a standard in AI development - [SlashGear](https://www.slashgear.com/1618170/what-does-gpt-stand-for/)
- Users on Reddit discuss ChatGPT's functionality and share humorous interactions - [Reddit](https://www.reddit.com/r/ChatGPT/comments/1e3lsrn/my_favorite_thing_to_do_with_chatgpt_is_make_it/)

2024-07-15 09:52:11,778 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# AI Ethics Investigation: Whistleblowers and Regulatory Concerns

- OpenAI whistleblowers filed a complaint with the SEC regarding allegedly restrictive NDAs that required employees to waive federal whistleblower compensation rights - [Investing.com](https://www.investing.com/news/economy-news/openai-whistleblowers-ask-sec-to-investigate-alleged-restrictive-nondisclosure-agreements-3518524)
- Whistleblowers urge US financial watchdog to investigate NDAs at OpenAI for restricting employees from contacting regulators without permission - [Google News](https://www.theguardian.com/technology/article/2024/jul/14/us-financial-watchdog-urged-to-investigate-ndas-at-openai)
- OpenAI allegedly used illegal non-disclosure agreements and required employees to disclose contact with authorities - [Google News](https://www.computerworld.com/article/2517450/open-ai-reportedly-stopped-staffers-from-warning-about-security-risks.html)
- OpenAI rushed the launch of GPT-4o, pressuring the safety team to speed through testing in under a week - [Windows Central](https://www.windowscentral.com/software-apps/openai-reportedly-sent-rsvps-for-gpt-4os-launch-party-even-before-testing-began)
- Agencies are navigating data security, stability, and fairness amid the flood of AI tools and platforms in the marketplace - [Digiday](https://digiday.com/marketing/everything-is-ai-now-amid-ai-boom-agencies-navigate-data-security-stability-and-fairness/)

In [114]:
print(mail_md_str)

# AI Stocks: Today's Key Developments

- Goldman Sachs warns that the ongoing outperformance of AI stocks might face challenges in the future - [Google News](https://markets.businessinsider.com/news/stocks/ai-bubble-pop-signal-stock-market-investing-outlook-goldman-sachs-2024-7).
- Super Micro Computer (SMCI) is well-positioned for growth in the AI sector, with a 77% potential upside and impressive revenue growth - [Google News](https://investorplace.com/2024/07/3-artificial-intelligence-stocks-set-to-mint-new-millionaires/).
- UBS downgrades Tesla stock to Sell due to high valuation, AI costs, and declining expectations for core auto business - [Google News](https://www.investing.com/news/stock-market-news/5-big-analyst-ai-moves-tesla-and-smci-stocks-downgraded-3518257).
- Apple's shares rose 2% after Morgan Stanley designated them as a "top pick" due to AI efforts, expecting a boost in device sales - [Investing.com](https://au.investing.com/news/stock-market-news/apple-gains-after-mo

In [115]:
log("Sending full summary email ")
subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
final_html_str = markdown.markdown(mail_md_str, extensions=['extra'])
display(HTML(final_html_str))
send_gmail(subject, final_html_str)


2024-07-15 09:52:11,833 - AInewsbot - INFO - Sending full summary email 


# Final Summary

In [116]:
markdown_str = ''
for i, row in enumerate(AIdf.itertuples()):
    mdstr = f"[{i+1}. {row.title} - {row.site_name}]({row.url})  \n\n"
#     if 0 < len(catdict[row.id]) < 11 :
#         topicstr = ", ".join(catdict[row.id])
#         mdstr += f"Topics: {topicstr}\n\n"
    mdstr += f"Topics: {row.topic_str}\n\n"
    mdstr += f"{response_dict[row.id]} \n\n"
    display(Markdown(mdstr.replace('$', '\\$')))
    markdown_str += mdstr
    

[1. Here Is My TopArtificial Intelligence(AI) Stock to Buy Right Now - The Motley Fool](https://www.fool.com/investing/2024/07/15/heres-my-top-ai-stock-buy-now/)  

Topics: Finance, Investment, Nvidia, Stock, Stocks, Technology

- Nvidia is a top artificial intelligence (AI) stock attracting investors due to leveraging gaming and GPUs, but its high valuation might lead to a significant drop, prompting investors to look at a long-time partner instead.
- Super Micro Computer (SMCI) is a potential investment with significant growth, having experienced a 4,500% increase in stock value in the last five years by providing Nvidia-powered AI equipment.
- Despite Supermicro's massive gains, its strong financial performance, including revenue growth of 95% year-over-year and raised guidance for fiscal 2024, indicates the stock's potential for further growth and offers a relatively inexpensive investment opportunity compared to Nvidia.
 



[2. 3 Artificial Intelligence Stocks Set to Mint New Millionaires - InvestorPlace](https://news.google.com/articles/CBMiYGh0dHBzOi8vaW52ZXN0b3JwbGFjZS5jb20vMjAyNC8wNy8zLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLXN0b2Nrcy1zZXQtdG8tbWludC1uZXctbWlsbGlvbmFpcmVzL9IBAA)  

Topics: Finance, Nvidia, Stocks, Technology

- Analysts' strong buy ratings are a good starting point for identifying quality AI stocks with potential upsides exceeding 40%.
- Super Micro Computer (SMCI) is well-positioned for growth in the AI sector, with a 77% potential upside and impressive revenue growth.
- QuickLogic Corporation (QUIK) and Lattice Semiconductor Corporation (LSCC) also show promise with significant revenue growth, positive sentiments, and strong buy ratings. 



[3. Sterling Infrastructure: The Picks And Shovels To The Future Of AI (NASDAQ:STRL) - Seeking Alpha](https://news.google.com/articles/CBMiamh0dHBzOi8vc2Vla2luZ2FscGhhLmNvbS9hcnRpY2xlLzQ3MDM5MzUtc3RlcmxpbmctaW5mcmFzdHJ1Y3R1cmUtdGhlLXBpY2tzLWFuZC1zaG92ZWxzLXRvLXRoZS1mdXR1cmUtb2YtYWnSAQA)  

Topics: Stocks, Technology

- Sterling Infrastructure is benefitting from long-term AI tailwinds and has experienced double-digit revenue growth, with a focus on e-infrastructure, transportation, and building solutions.
- The company has shown impressive financial performance with revenue and EBITDA growth, a strong capital allocation framework including investments in organic growth and M&A, and a growing backlog of projects.
- Sterling Infrastructure's growth is being primarily driven by the e-infrastructure segment, amid a massive growth opportunity in the smart infrastructure market driven by AI advancements, with a strong outlook supported by a substantial backlog and guidance for revenue and EBITDA growth in FY'24. 



[4. Paris Olympics will use Alibabas AI-powered energy-saving tool to cut venue emissions - South China Morning Post](https://www.scmp.com/business/article/3270513/paris-olympics-will-use-alibabas-ai-powered-energy-saving-tool-cut-venue-emissions)  

Topics: Big Tech, Climate, Energy, Products, Sustainability, Technology

- Paris Olympics will use Alibaba’s AI-powered energy-saving tool to optimize energy demand and reduce venue emissions
- The AI tool will analyze power usage at 35 locations during the games to minimize waste and help achieve carbon neutrality
- Organizers aim to cut event-related carbon emissions by using renewable energy sources and sustainable practices at the Games 



[5. A.I.s Insatiable Appetite for Energy - The New York Times](https://news.google.com/articles/CBMiVGh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDcvMTEvY2xpbWF0ZS9hcnRpZmljaWFsLWludGVsbGlnZW5jZS1lbmVyZ3ktdXNhZ2UuaHRtbNIBAA)  

Topics: AI Doom, Big Tech, Energy, Gen AI, Nvidia, OpenAI, Science, Sustainability, Technology

- A.I.'s increasing electricity demands from data centers are straining the energy grid, contributing to higher emissions and hindering the energy transition.
- Tech giants such as Microsoft and Google are facing challenges in reducing the carbon footprint of A.I., with projections showing a significant increase in electricity consumption by data centers.
- The growth of A.I. is driving up energy demand and emissions, prompting companies to explore renewable energy sources while facing hurdles in balancing energy needs with environmental sustainability. 



...

[202. People are falling in love with AI. Should we worry? - Live Science](https://news.google.com/articles/CBMiY2h0dHBzOi8vd3d3LmxpdmVzY2llbmNlLmNvbS9oZWFsdGgvcmVsYXRpb25zaGlwcy9wZW9wbGUtYXJlLWZhbGxpbmctaW4tbG92ZS13aXRoLWFpLXNob3VsZC13ZS13b3JyedIBAA)  

Topics: AI Doom, Gen AI, Opinion, Technology

- AI romantic companions can save some people from loneliness but also have a dark side, affecting real-world relationship expectations and social skills.
- Users can form emotional bonds with AI, enjoying emotional connections regardless of whether they believe they are interacting with a human or a chatbot.
- AI companions come with privacy concerns, such as data sharing and tracking, but offer potential benefits in enhancing romantic well-being and providing support to marginalized communities. 



[203. mini PC identify management artificial intelligence (AI) - ](https://news.google.com/articles/CBMiiQFodHRwczovL3d3dy5taWxpdGFyeWFlcm9zcGFjZS5jb20vY29tcHV0ZXJzL2FydGljbGUvNTUwOTM5ODYvYWFlb24tZWxlY3Ryb25pY3MtaW5jLW1pbmktcGMtaWRlbnRpZnktbWFuYWdlbWVudC1hcnRpZmljaWFsLWludGVsbGlnZW5jZS1hadIBAA)  

Topics: Gen AI, Nvidia, Products, Technology

- AAEON Technology introduces the UP Squared Pro 710H Edge mini PC for identity management, anti-microbial resistance, and health care imaging solutions, featuring the Hailo-8 edge AI processor.
- The mini PC offers 26 tera-operations per second of inferencing performance, onboard Hailo AI software suite, and up to 16GB LPDDR5 memory along with 128GB eMMC data storage for speed, capacity, and security.
- The compact mini PC provides AI-ready features, multiple I/O options, 40-pin GPIO for various peripherals, and support for Windows 10 IoT Enterprise and Ubuntu 22.04 LTS operating systems. 



[204. OpenGPT-X Team Publishes European LLM Leaderboard: Promoting the Way for Advanced Multilingual Language Model Development and Evaluation - MarkTechPost](https://www.marktechpost.com/2024/07/14/opengpt-x-team-publishes-european-llm-leaderboard-promoting-the-way-for-advanced-multilingual-language-model-development-and-evaluation/)  

Topics: Cohere, Language Models, Llms, Open Source, Technology

- The OpenGPT-X team released the European LLM Leaderboard to advance multilingual language models and reduce digital language barriers in AI applications across Europe.
- The project focuses on developing and evaluating multilingual LLMs to broaden language accessibility, with benchmarks translated into 21 European languages for comprehensive evaluations.
- TU Dresden's involvement includes leveraging its competence centers to develop scalable evaluation pipelines and improve model performance of large language models on supercomputing clusters, with automated evaluations through the Hugging Face Hub platform. 



[205. Samsung's latest Galaxy AI features tipped to be heading to these older phones - TechRadar](https://www.techradar.com/phones/samsung-galaxy-phones/samsungs-latest-galaxy-ai-features-tipped-to-be-heading-to-these-older-phones)  

Topics: Big Tech, Korea, Products, Samsung, Technology

- Samsung is expected to bring the latest One UI 6.1.1 software with Galaxy AI features to older models including the Galaxy S22, S23, S24, S23 FE, and tablets like the Galaxy Tab S8 and S9.
- The One UI 6.1.1 update introduces new Galaxy AI features such as Sketch to Image, Portrait Studio, Live Translation, and Composer, enhancing AI capabilities on supported devices.
- The update includes a new Galaxy AI menu in Settings and device-specific improvements like new widgets for the Samsung Galaxy Z Flip 6 cover screen. 



[206. Metas approach to machine learning prediction robustness - Facebook Engineering](https://news.google.com/articles/CBMiaWh0dHBzOi8vZW5naW5lZXJpbmcuZmIuY29tLzIwMjQvMDcvMTAvZGF0YS1pbmZyYXN0cnVjdHVyZS9tYWNoaW5lLWxlYXJuaW5nLW1sLXByZWRpY3Rpb24tcm9idXN0bmVzcy1tZXRhL9IBAA)  

Topics: Data, Machine Learning, Meta, Science, Technology, Testing

- Meta's advertising business uses large-scale machine learning models for ad recommendations, requiring robust prediction systems to ensure service reliability and delivery.
- Challenges in ML prediction stability stem from model stochasticity, continuous updates, and difficulty in detecting and tracing cumulative quality shifts over time.
- Meta has developed prediction robustness frameworks for models, features, training data, calibration, and interpretability to ensure stability and reliability, utilizing techniques like Snapshot Validator and Hawkeye for model robustness and ML interpretability. 



In [117]:
# clean up the list of topics in human in the loop workflow
# loop though each topic and summarize, then 
# then combine the summaries for a final prompt

In [118]:
print(FINAL_SUMMARY_PROMPT)


You are an advanced content analysis assistant, a sophisticated AI system
designed to perform in-depth analysis of news content. Your function is to extract meaningful insights,
categorize information, and identify trends from large volumes of news.

I will provide a list of today's news articles about AI and summary bullet points in markdown format.
Bullet points will contain a title and URL, a list of topics discussed, and a bullet-point summary of
the article. You are tasked with identifying and summarizing the most important news, recurring themes,
common facts and items. Your job is to create a concise summary containing about 20 of the most
frequently mentioned topics and developments.


Example Input Bullet Points:

[2. Sentient closes $85M seed round for open-source AI](https://cointelegraph.com/news/sentient-85-million-round-open-source-ai)

AI startup funding, New AI products

- Sentient secured $85 million in a seed funding round led by Peter Thiel's Founders Fund, Pantera C

In [119]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
              {"role": "user", "content": FINAL_SUMMARY_PROMPT + markdown_str
              }],
    n=1,   
    temperature=0.5
)

response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\$")
display(Markdown(response_str))


2024-07-15 09:58:01,295 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# Today's AI News

### Finance and Investments:
- Nvidia remains a top AI stock, attracting significant investor interest due to its GPU technology and AI chip advancements. ([The Motley Fool](https://www.fool.com/investing/2024/07/15/heres-my-top-ai-stock-buy-now/), [InvestorPlace](https://news.google.com/articles/CBMiYGh0dHBzOi8vaW52ZXN0b3JwbGFjZS5jb20vMjAyNC8wNy8zLWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLXN0b2Nrcy1zZXQtdG8tbWludC1uZXctbWlsbGlvbmFpcmVzL9IBAA))
- Skild AI raised \$300 million in funding from investors like Jeff Bezos and Softbank to develop AI brains for robots. ([Cointelegraph](https://news.google.com/articles/CBMibmh0dHBzOi8vY29pbnRlbGVncmFwaC5jb20vbmV3cy9za2lsZC1haS1hcnRpZmljaWFsLWludGVsbGlnZW5jZS0zMDAtbS1mdW5kaW5nLWplZmYtYmV6b3Mtc29mdGJhbmstcm9ib3QtYnJhaW5z0gEA), [Benzinga](https://www.benzinga.com/news/24/07/39766402/jeff-bezos-and-masayoshi-sons-softbank-lead-300m-funding-round-for-startup-working-on-developing-gen))
- AMD acquires Silo AI for \$665 million to boost its AI model and software development capabilities. ([Forbes](https://news.google.com/articles/CBMigwFodHRwczovL3d3dy5mb3JiZXMuY29tL3NpdGVzL2NoYXJsaWVmaW5rLzIwMjQvMDcvMTEvYW1kLWJ1eXMtc2lsby1haS1mb3ItNjY1LW1pbGxpb24tY2FwdGlvbnMtcmFpc2VzLTYwLW1pbGxpb24tbmV3LWFpLXZpZGVvLXRvb2xzL9IBAA), [The Next Platform](https://news.google.com/articles/CBMia2h0dHBzOi8vd3d3Lm5leHRwbGF0Zm9ybS5jb20vMjAyNC8wNy8xMC9hbWQtYnVpbGRzLW91dC1pdHMtYWktc29mdHdhcmUtYW5kLXNlcnZpY2VzLXN0YWNrLXdpdGgtc2lsby1haS1idXkv0gFvaHR0cHM6Ly93d3cubmV4dHBsYXRmb3JtLmNvbS8yMDI0LzA3LzEwL2FtZC1idWlsZHMtb3V0LWl0cy1haS1zb2Z0d2FyZS1hbmQtc2VydmljZXMtc3RhY2std2l0aC1zaWxvLWFpLWJ1eS9hbXAv))

### Health Care:
- AI tools outperform traditional methods in predicting Alzheimer's disease progression, offering earlier and more accurate diagnoses. ([The Brattleboro Reformer](https://www.reformer.com/health/ai-better-at-predicting-progression-to-alzheimers-than-standard-care/article_1efc5053-677e-500a-baa6-ab79404d3034.html), [Medical Xpress](https://news.google.com/articles/CBMiYmh0dHBzOi8vbWVkaWNhbHhwcmVzcy5jb20vbmV3cy8yMDI0LTA3LWFydGlmaWNpYWwtaW50ZWxsaWdlbmNlLW91dHBlcmZvcm1zLWNsaW5pY2FsLWFsemhlaW1lci5odG1s0gEA))

### Energy and Sustainability:
- The rapid growth of AI is increasing electricity demands, straining energy grids and prompting tech companies to explore renewable energy solutions. ([The New York Times](https://news.google.com/articles/CBMiVGh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDcvMTEvY2xpbWF0ZS9hcnRpZmljaWFsLWludGVsbGlnZW5jZS1lbmVyZ3ktdXNhZ2UuaHRtbNIBAA), [Fast Company](https://news.google.com/articles/CBMiVWh0dHBzOi8vd3d3LmZhc3Rjb21wYW55LmNvbS85MTE1NDYyOS9haS1kYXRhLWNlbnRlcnMtZW5lcmd5LXVzZS1zdXN0YWluYWJsZS1zb2x1dGlvbnPSAQA))

### Privacy and Ethics:
- Google's Gemini AI faced criticism for scanning Google Drive PDF files without user permission, raising privacy concerns. ([Tom's Hardware](https://news.google.com/articles/CBMitQFodHRwczovL3d3dy50b21zaGFyZHdhcmUuY29tL3RlY2gtaW5kdXN0cnkvYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UvZ2VtaW5pLWFpLWNhdWdodC1zY2FubmluZy1nb29nbGUtZHJpdmUtaG9zdGVkLXBkZi1maWxlcy13aXRob3V0LXBlcm1pc3Npb24tdXNlci1jb21wbGFpbnMtZmVhdHVyZS1jYW50LWJlLWRpc2FibGVk0gEA), [The Verge](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3LnRoZXZlcmdlLmNvbS8yMDI0LzcvMTQvMjQxOTcxMDcvYWktbWVtb3J5LXJlY2FsbC1saW1pdGxlc3MtZ2VtaW5pLXZlcmdlY2FzdNIBAA))

### AI in Education:
- AI tools like Khanmigo are being used in schools to provide personalized tutoring and interactive learning experiences. ([New Hampshire Bulletin](https://newhampshirebulletin.com/2024/07/15/from-precalculus-to-gatsby-state-offers-schools-an-ai-tutor-option/))

### AI in Entertainment:
- Hollywood stars are divided on the use of AI and deepfake technology in films, with concerns about creative control and artistic integrity. ([InsideHook](https://news.google.com/articles/CBMiYWh0dHBzOi8vZGlnaXRhbGNoZXcuY29tLzIwMjQvMDcvMDkvaG9sbHl3b29kLXN0YXJzLXRha2Utc2lkZXMtb24tdXNpbmctYWktYW5kLWRlZXBmYWtlcy1pbi1maWxtcy_SAWFodHRwczovL2RpZ2l0YWxjaGV3LmNvbS8yMDI0LzA3LzA5L2hvbGx5d29vZC1zdGFycy10YWtlLXNpZGVzLW9uLXVzaW5nLWFpLWFuZC1kZWVwZmFrZXMtaW4tZmlsbXMv))

### AI in Technology:
- AI-native networking is transforming businesses by reducing complexity and boosting productivity. ([Frontier Enterprise](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmZyb250aWVyLWVudGVycHJpc2UuY29tL2hvdy1haS1uYXRpdmUtbmV0d29ya2luZy1nZW5lcmF0ZXMtcmVhbC1idXNpbmVzcy1pbXBhY3Qv0gEA))

### AI in Health:
- AI tools are being used to fight against insurance denials, significantly improving approval rates for doctors. ([The New York Times](https://news.google.com/articles/CBMiV2h0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDcvMTAvaGVhbHRoL2RvY3RvcnMtaW5zdXJlcnMtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UuaHRtbNIBAA))

### AI in Security:
- AI-driven chatbots are being used to improve meeting efficiency and professional interactions by leveraging meeting transcripts. ([Geeky Gadgets](https://news.google.com/articles/CBMiRWh0dHBzOi8vd3d3LmhlbHBuZXRzZWN1cml0eS5jb20vMjAyNC8wNy8xMC9qYWtlLWtpbmctZWxhc3RpYy1sbG1zLXNlY3VyaXR5LXJpc2tzL9IBAA))

### AI in Government:
- Arkansas is exploring the use of AI in government operations as part of a growing trend among states. ([Arkansas Democrat Gazette](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3LmFya2Fuc2Fzb25saW5lLmNvbS9uZXdzLzIwMjQvanVsLzE0L2Fya2Fuc2FzLWpvaW5zLW51bWJlci1vZi1zdGF0ZXMtcmVzZWFyY2hpbmcv0gEA))

### AI in Robotics:
- Ukrainian startups are creating low-cost robots to fight against Russian troops, aiming to minimize human casualties. ([Newsday](https://news.google.com/articles/CBMiXGh0dHBzOi8vd3d3Lm5ld3NkYXkuY29tL25ld3MvbmF0aW9uL3VrcmFpbmUtcnVzc2lhLWRyb25lcy1yb2JvdHMtYXJ0aWZpY2lhbC1pbnRlbGxpZ2VuY2UteTk5OTEx0gF0aHR0cHM6Ly9hbXAudGVsZWZvbmljYS5jb20vZW4vY29tbXVuaWNhdGlvbi1yb29tL2Jsb2cvZGF0YS1haS1kcml2ZW4tc29jaWV0eS9hbXAv))

### AI in Data:
- Discover Global Network is leveraging AI to enhance customer experiences and address emerging end-user expectations in the payment space. ([PYMNTS](https://news.google.com/articles/CBMiaGh0dHBzOi8vd3d3LnB5bW50cy5jb20vbmV3cy9wYXltZW50cy1pbm5vdmF0aW9uLzIwMjQvZGlzY292ZXItbG9va3MtdG8tYWktdG8tdW5sb2NrLWN1c3RvbWVyLWV4cGVyaWVuY2Uv0gEA))

### AI in Legal:
- New York State's Department of Financial Services requires insurers to test AI for bias to prevent discriminatory underwriting or pricing decisions. ([Newsday](https://news.google.com/articles/CBMie2h0dHBzOi8vYXJzdGVjaG5pY2EuY29tL3RlY2gtcG9saWN5LzIwMjQvMDcvY291cnQtb3JkZXJlZC1wZW5hbHRpZXMtZm9yLTE1LXRlZW5zLXdoby1jcmVhdGVkLW5ha2VkLWFpLWltYWdlcy1vZi1jbGFzc21hdGVzL9IBAA))

### AI in Hardware:
- LG Energy Solution developed an AI-powered system that shortens the battery cell design process to a day, leading to substantial cost savings and improved operational efficiency. ([Korea JoongAng Daily](https://news.google.com/articles/CBMikQFodHRwczovL2tvcmVham9vbmdhbmdkYWlseS5qb2lucy5jb20vbmV3cy8yMDI0LTA3LTE0L2J1c2luZXNzL2luZHVzdHJ5L05ldy1MRy1FbmVyZ3ktU29sdXRpb24tQUktZGVzaWducy1iYXR0ZXJ5LWNlbGxzLWluLWxlc3MtdGhhbi1hLWRheS8yMDg5Nzc50gEA), [Interesting Engineering](https://news.google.com/articles/CBMiUWh0dHBzOi8vaW50ZXJlc3RpbmdlbmdpbmVlcmluZy5jb20vZW5lcmd5L2xnLWVuZXJneS1zb2x1dGlvbi1haS1iYXR0ZXJ5LWRlc2lnbmluZ9IBAA))

### AI in Consumer Products:
- Apple's iOS 18 update introduces "Apple Intelligence," enhancing user experience on iPhone devices through advanced generative models and personalized context. ([9to5Mac](https://news.google.com/articles/CBMiSmh0dHBzOi8vOXRvNW1hYy5jb20vMjAyNC8wNy8xNS9hcHBsZS1sYWdzLWJlaGluZC1nbG9iYWwtc21hcnRwaG9uZS1tYXJrZXQv0gEA))

### AI in Society:
- AI romantic companions are increasingly being used to combat loneliness but raise concerns about real-world relationship expectations and privacy. ([Live Science](https://news.google.com/articles/CBMiY2h0dHBzOi8vd3d3LmxpdmVzY2llbmNlLmNvbS9oZWFsdGgvcmVsYXRpb25zaGlwcy9wZW9wbGUtYXJlLWZhbGxpbmctaW4tbG92ZS13aXRoLWFpLXNob3VsZC13ZS13b3JyedIBAA))

### AI in Regulation:
- The EU's Artificial Intelligence Act, effective August 1, increases compliance costs for foreign companies, particularly Chinese tech firms. ([TechRadar](https://www.techradar.com/pro/european-ai-rules-could-hamstring-chinese-tech-companies-with-compliance-costs))

### AI in Research:
- AI technology helped uncover a significant copper deposit in Zambia, highlighting AI's potential in revolutionizing mining and resource discovery. ([The New York Times](https://news.google.com/articles/CBMiTmh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDcvMTEvY2xpbWF0ZS9rb2JvbGQtemFtYmlhLWNvcHBlci1haS1taW5pbmcuaHRtbNIBAA))

### AI in Gaming:
- AI-generated board games using models like CodeLLaMa demonstrate AI's potential in creating new gaming experiences. ([Analytics India Magazine](https://analyticsindiamag.com/researchers-make-ai-generated-board-games-using-codellama/))

In [ ]:
response_str = response.choices[0].message.content
response_str = response_str.replace("$", "\\$")
display(Markdown(response_str))


In [ ]:
log("Sending full summary email ")
subject = f'AI news summary {datetime.now().strftime("%H:%M:%S")}'
final_html_str = markdown.markdown(response_str, extensions=['extra'])
display(HTML(final_html_str))
send_gmail(subject, final_html_str)


In [ ]:
log("Finished")


In [ ]:
redirect_dict